# Grammars


## Definitons

We use the following terms:

* The _alphabet_ is the set all of symbols in the input language. For example,
  in this post, we use all ASCII characters as alphabet.
  
* A _terminal_ is a single alphabet symbol. Note that this is slightly different
  from usual definitions (done here for ease of parsing). (Usually a terminal is
  a contiguous sequence of symbols from the alphabet. However, both kinds of
  grammars have a one to one correspondence, and can be converted easily.)
  
  For example, `x` is a terminal symbol.

* A _nonterminal_ is a symbol outside the alphabet whose expansion is _defined_
  in the grammar using _rules_ for expansion.

  For example, `<term>` is a nonterminal in the below grammar.
  
* A _term_ is a terminal or a nonterminal symbol. These are also sometimes called _tokens_.
  We note that a _token_ is also the name used for lexical units in ANTLR. We use both
  interchangeably when there is no ambiguity.

* A _rule_ is a finite sequence of _terms_ (two types of terms: terminals and
  nonterminals) that describe an expansion of a given terminal.

  For example, `[<term>+<expr>]` is one of the expansion rules of the nonterminal `<expr>`.

* A _definition_ is a set of _rules_ that describe the expansion of a given nonterminal.

  For example, `[[<digit>,<digits>],[<digit>]]` is the definition of the nonterminal `<digits>`

* A _context-free grammar_ is  composed of a set of nonterminals and 
  corresponding definitions that define the structure of the nonterminal.

  The grammar given below is an example context-free grammar.

* A terminal _derives_ a string if the string contains only the symbols in the
  terminal. A nonterminal derives a string if the corresponding definition
  derives the string. A definition derives the  string if one of the rules in
  the definition derives the string. A rule derives a string if the sequence
  of terms that make up the rule can derive the string, deriving one substring 
  after another contiguously (also called parsing).

* A *derivation tree* is an ordered tree that describes how an input string is
  derived by the given start symbol. Also called a *parse tree*.
  
* A derivation tree can be collapsed into its string equivalent. Such a string
  can be parsed again by the nonterminal at the root node of the derivation
  tree such that at least one of the resulting derivation trees would be the
  same as the one we started with.

We use the [fuzzingbook](https://www.fuzzingbook.org) *canonical* grammar style.

The format uses a JSON object (or equivalently a Python dictionary) for storing the grammar. The keys in the JSON object correspond to _nonterminals_ in the grammar. The object contains the _definition_ of each nonterminal. 

```python
grammar: {
    "<nonterminal>": definition,
    "<nonterminal>": definition
}
```

The _definition_ is stored as a list of _rules_.
```python
definition: [
    rule,
    rule
]
```
Each _rule_ is again a list of _terms_ which can be either terminal or nonterminal.
```python
rule: [terminal, nonterminal, terminal]
```

Each nonterminal symbol is a string that starts with `<` and ends with `>`, and cannot contain either of these characters inside.

```python
nonterminal: '<...>'
```

A terminal symbol is a string that contains any single character. A relaxation is that it can be any string so long as it does not start with `<` and ends with `>` and it is not empty.

A nonterminal that contains an empty definition in the grammar (an empty list) is considered to be undefined in the grammar.

Here is an example grammar for arithmetic expressions, starting at `<start>`.
A terminal symbol has exactly one character
(Note that we disallow empty string (`''`) as a terminal symbol).

In [ ]:
#%%writefile src/grammars.py
import string
EXPR_START = '<start>'
EXPR_GRAMMAR = {
 '<start>': [['<expr>']],
 '<expr>': [['<term>', '+', '<expr>'],
            ['<term>', '-', '<expr>'],
            ['<term>']],
 '<term>': [['<factor>', '*', '<term>'],
            ['<factor>', '/', '<term>'],
            ['<factor>']],
 '<factor>': [['+', '<factor>'],
              ['-', '<factor>'],
              ['(', '<expr>', ')'],
              ['<integer>', '.', '<integer>'],
              ['<integer>']],
 '<integer>': [['<digit>', '<integer>'], ['<digit>']],
 '<digit>': [[str(i)] for i in string.digits]}



In [ ]:
import ipynb.fs.full.RailRoads as diagram

In [ ]:
# diagram.syntax_diagram(EXPR_GRAMMAR)

In [ ]:
#%%writefile -a src/grammars.py
BEXPR_START = '<start>'
BEXPR_GRAMMAR = {
    '<start>': [['<bexpr>']],
    '<bexpr>': [
        ['<bop>', '(', '<bexprs>', ')'],
        ['<fault>']],
    '<bexprs>' : [['<bexpr>', ',', '<bexprs>'], ['<bexpr>']],
    '<bop>' : [list('and'), list('or'), list('neg')],
    '<fault>': [['<letters>'], []],
    '<letters>': [
        ['<letter>'],
        ['<letter>', '<letters>']],
    '<letter>': [[i] for i in (
        string.ascii_lowercase +
        string.ascii_uppercase +
        string.digits) + '_+*.-']
}


In [ ]:
# diagram.syntax_diagram(BEXPR_GRAMMAR)

In [ ]:
#%%writefile -a src/grammars.py
JSON_START = '<start>'
JSON_GRAMMAR = {
        '<start>': [['<json>']],
        '<json>': [['<element>']],
        '<element>': [['<ws>', '<value>', '<ws>']],
        '<value>': [
           ['<object>'], ['<array>'], ['<string>'], ['<number>'],
           ['true'], ['false'], ['null']],
        '<object>': [['{', '<ws>', '}'], ['{', '<members>', '}']],
        '<members>': [['<member>', '<symbol-2>']],
        '<member>': [['<ws>', '<string>', '<ws>', ':', '<element>']],
        '<array>': [['[', '<ws>', ']'], ['[', '<elements>', ']']],
        '<elements>': [['<element>', '<symbol-1-1>']],
        '<string>': [['"', '<characters>', '"']],
        '<characters>': [['<character-1>']],
        '<character>': [
            ['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'],
            ['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['g'], ['h'], ['i'], ['j'],
            ['k'], ['l'], ['m'], ['n'], ['o'], ['p'], ['q'], ['r'], ['s'], ['t'],
            ['u'], ['v'], ['w'], ['x'], ['y'], ['z'], ['A'], ['B'], ['C'], ['D'],
            ['E'], ['F'], ['G'], ['H'], ['I'], ['J'], ['K'], ['L'], ['M'], ['N'],
            ['O'], ['P'], ['Q'], ['R'], ['S'], ['T'], ['U'], ['V'], ['W'], ['X'],
            ['Y'], ['Z'], ['!'], ['#'], ['$'], ['%'], ['&'], ["'"], ['('], [')'],
            ['*'], ['+'], [','], ['-'], ['.'], ['/'], [':'], [';'], ['<'], ['='],
            ['>'], ['?'], ['@'], ['['], [']'], ['^'], ['_'], ['`'], ['{'], ['|'],
            ['}'], ['~'], [' '], ['\\"'], ['\\\\'], ['\\/'], ['<escaped>']],
        '<number>': [['<int>', '<frac>', '<exp>']],
        '<int>': [
           ['<digit>'], ['<onenine>', '<digits>'],
           ['-', '<digits>'], ['-', '<onenine>', '<digits>']],
        '<digits>': [['<digit-1>']],
        '<digit>': [['0'], ['<onenine>']],
        '<onenine>': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']],
        '<frac>': [['.', '<digits>'], []],
        '<exp>': [[], ['E', '<sign>', '<digits>'], ['e', '<sign>', '<digits>']],
        '<sign>': [[], ['+'], ['-']],
        '<ws>': [['<sp1>', '<ws>'], []],
        '<sp1>': [[' ']], ##[['\n'], ['\r'], ['\t'], ['\x08'], ['\x0c']],
        '<symbol>': [[',', '<members>']],
        '<symbol-1>': [[',', '<elements>']],
        '<symbol-2>': [[], ['<symbol>', '<symbol-2>']],
        '<symbol-1-1>': [[], ['<symbol-1>', '<symbol-1-1>']],
        '<character-1>': [[], ['<character>', '<character-1>']],
        '<digit-1>': [['<digit>'], ['<digit>', '<digit-1>']],
        '<escaped>': [['\\u', '<hex>', '<hex>', '<hex>', '<hex>']],
        '<hex>': [
            ['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'],
            ['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['A'], ['B'], ['C'], ['D'], ['E'],   ['F']]
        }

In [ ]:
# diagram.syntax_diagram(JSON_GRAMMAR)

In [ ]:
SQL_START = '<queries>'

SQL_GRAMMAR = {
 '<queries>': ['<query>*'],
 '<query>': ['<query-1><SEMICOLON_SYMBOL>'],
 '<query-1>': ['<simpleStatement>', '<beginWork>'],
 '<simpleStatement>': ['<alterStatement>',
                       '<createStatement>',
                       '<dropStatement>',
                       '<renameTableStatement>',
                       '<truncateTableStatement>',
                       '<importStatement>',
                       '<callStatement>',
                       '<deleteStatement>',
                       '<doStatement>',
                       '<handlerStatement>',
                       '<insertStatement>',
                       '<loadStatement>',
                       '<replaceStatement>',
                       '<selectStatement>',
                       '<updateStatement>',
                       '<transactionOrLockingStatement>',
                       '<replicationStatement>',
                       '<preparedStatement>',
                       '<cloneStatement>',
                       '<accountManagementStatement>',
                       '<tableAdministrationStatement>',
                       '<uninstallStatement>',
                       '<installStatement>',
                       '<setStatement>',
                       '<showDatabasesStatement>',
                       '<showTablesStatement>',
                       '<showTriggersStatement>',
                       '<showEventsStatement>',
                       '<showTableStatusStatement>',
                       '<showOpenTablesStatement>',
                       '<showParseTreeStatement>',
                       '<showPluginsStatement>',
                       '<showEngineLogsStatement>',
                       '<showEngineMutexStatement>',
                       '<showEngineStatusStatement>',
                       '<showColumnsStatement>',
                       '<showBinaryLogsStatement>',
                       '<showBinaryLogStatusStatement>',
                       '<showReplicasStatement>',
                       '<showBinlogEventsStatement>',
                       '<showRelaylogEventsStatement>',
                       '<showKeysStatement>',
                       '<showEnginesStatement>',
                       '<showCountWarningsStatement>',
                       '<showCountErrorsStatement>',
                       '<showWarningsStatement>',
                       '<showErrorsStatement>',
                       '<showProfilesStatement>',
                       '<showProfileStatement>',
                       '<showStatusStatement>',
                       '<showProcessListStatement>',
                       '<showVariablesStatement>',
                       '<showCharacterSetStatement>',
                       '<showCollationStatement>',
                       '<showPrivilegesStatement>',
                       '<showGrantsStatement>',
                       '<showCreateDatabaseStatement>',
                       '<showCreateTableStatement>',
                       '<showCreateViewStatement>',
                       '<showMasterStatusStatement>',
                       '<showReplicaStatusStatement>',
                       '<showCreateProcedureStatement>',
                       '<showCreateFunctionStatement>',
                       '<showCreateTriggerStatement>',
                       '<showCreateProcedureStatusStatement>',
                       '<showCreateFunctionStatusStatement>',
                       '<showCreateProcedureCodeStatement>',
                       '<showCreateFunctionCodeStatement>',
                       '<showCreateEventStatement>',
                       '<showCreateUserStatement>',
                       '<resourceGroupManagement>',
                       '<otherAdministrativeStatement>',
                       '<utilityStatement>',
                       '<getDiagnosticsStatement>',
                       '<signalStatement>',
                       '<resignalStatement>'],
 '<alterStatement>': ['<ALTER_SYMBOL><alterStatement-1>'],
 '<alterStatement-1>': ['<alterTable>',
                        '<alterDatabase>',
                        '<PROCEDURE_SYMBOL><procedureRef><routineAlterOptions>?',
                        '<FUNCTION_SYMBOL><functionRef><routineAlterOptions>?',
                        '<alterView>',
                        '<alterEvent>',
                        '<alterTablespace>',
                        '<alterUndoTablespace>',
                        '<alterLogfileGroup>',
                        '<alterServer>',
                        '<alterInstanceStatement>'],
 '<alterDatabase>': ['<DATABASE_SYMBOL><schemaRef><alterDatabaseOption>+'],
 '<alterDatabaseOption>': ['<createDatabaseOption>',
                           '<READ_SYMBOL><ONLY_SYMBOL><EQUAL_OPERATOR>?<ternaryOption>'],
 '<alterEvent>': ['<definerClause>?<EVENT_SYMBOL><eventRef><alterEvent-1>?<alterEvent-2>?<alterEvent-3>?<alterEvent-4>?<alterEvent-6>?<alterEvent-7>?'],
 '<alterEvent-1>': ['<ON_SYMBOL><SCHEDULE_SYMBOL><schedule>'],
 '<alterEvent-2>': ['<ON_SYMBOL><COMPLETION_SYMBOL><NOT_SYMBOL>?<PRESERVE_SYMBOL>'],
 '<alterEvent-3>': ['<RENAME_SYMBOL><TO_SYMBOL><identifier>'],
 '<alterEvent-4>': ['<ENABLE_SYMBOL>', '<DISABLE_SYMBOL><alterEvent-5>?'],
 '<alterEvent-5>': ['<ON_SYMBOL><replica>'],
 '<alterEvent-6>': ['<COMMENT_SYMBOL><textLiteral>'],
 '<alterEvent-7>': ['<DO_SYMBOL><compoundStatement>'],
 '<alterLogfileGroup>': ['<LOGFILE_SYMBOL><GROUP_SYMBOL><logfileGroupRef><ADD_SYMBOL><UNDOFILE_SYMBOL><textLiteral><alterLogfileGroupOptions>?'],
 '<alterLogfileGroupOptions>': ['<alterLogfileGroupOption><alterLogfileGroupOptions-1>*'],
 '<alterLogfileGroupOptions-1>': ['<COMMA_SYMBOL>?<alterLogfileGroupOption>'],
 '<alterLogfileGroupOption>': ['<tsOptionInitialSize>',
                               '<tsOptionEngine>',
                               '<tsOptionWait>'],
 '<alterServer>': ['<SERVER_SYMBOL><serverRef><serverOptions>'],
 '<alterTable>': ['<onlineOption>?<TABLE_SYMBOL><tableRef><alterTableActions>?'],
 '<alterTableActions>': ['<alterCommandList><alterTableActions-1>?',
                         '<partitionClause>',
                         '<removePartitioning>',
                         '<alterTableActions-2>?<standaloneAlterCommands>'],
 '<alterTableActions-1>': ['<partitionClause>', '<removePartitioning>'],
 '<alterTableActions-2>': ['<alterCommandsModifierList><COMMA_SYMBOL>'],
 '<alterCommandList>': ['<alterCommandsModifierList>',
                        '<alterCommandList-1>?<alterList>'],
 '<alterCommandList-1>': ['<alterCommandsModifierList><COMMA_SYMBOL>'],
 '<alterCommandsModifierList>': ['<alterCommandsModifier><alterCommandsModifierList-1>*'],
 '<alterCommandsModifierList-1>': ['<COMMA_SYMBOL><alterCommandsModifier>'],
 '<standaloneAlterCommands>': ['<DISCARD_SYMBOL><TABLESPACE_SYMBOL>',
                               '<IMPORT_SYMBOL><TABLESPACE_SYMBOL>',
                               '<alterPartition>',
                               '<standaloneAlterCommands-1>'],
 '<standaloneAlterCommands-1>': ['<SECONDARY_LOAD_SYMBOL>',
                                 '<SECONDARY_UNLOAD_SYMBOL>'],
 '<alterPartition>': ['<ADD_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<alterPartition-1>',
                      '<DROP_SYMBOL><PARTITION_SYMBOL><identifierList>',
                      '<REBUILD_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<allOrPartitionNameList>',
                      '<OPTIMIZE_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<allOrPartitionNameList><noWriteToBinLog>?',
                      '<ANALYZE_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<allOrPartitionNameList>',
                      '<CHECK_SYMBOL><PARTITION_SYMBOL><allOrPartitionNameList><checkOption>*',
                      '<REPAIR_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<allOrPartitionNameList><repairType>*',
                      '<COALESCE_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<real_ulong_number>',
                      '<TRUNCATE_SYMBOL><PARTITION_SYMBOL><allOrPartitionNameList>',
                      '<REORGANIZE_SYMBOL><PARTITION_SYMBOL><noWriteToBinLog>?<alterPartition-2>?',
                      '<EXCHANGE_SYMBOL><PARTITION_SYMBOL><identifier><WITH_SYMBOL><TABLE_SYMBOL><tableRef><withValidation>?',
                      '<DISCARD_SYMBOL><PARTITION_SYMBOL><allOrPartitionNameList><TABLESPACE_SYMBOL>',
                      '<IMPORT_SYMBOL><PARTITION_SYMBOL><allOrPartitionNameList><TABLESPACE_SYMBOL>'],
 '<alterPartition-1>': ['<partitionDefinitions>',
                        '<PARTITIONS_SYMBOL><real_ulong_number>'],
 '<alterPartition-2>': ['<identifierList><INTO_SYMBOL><partitionDefinitions>'],
 '<alterList>': ['<alterList-1><alterList-2>*'],
 '<alterList-1>': ['<alterListItem>', '<createTableOptionsSpaceSeparated>'],
 '<alterList-2>': ['<COMMA_SYMBOL><alterList-3>'],
 '<alterList-3>': ['<alterListItem>',
                   '<alterCommandsModifier>',
                   '<createTableOptionsSpaceSeparated>'],
 '<alterCommandsModifier>': ['<alterAlgorithmOption>',
                             '<alterLockOption>',
                             '<withValidation>'],
 '<alterListItem>': ['<ADD_SYMBOL><COLUMN_SYMBOL>?<alterListItem-1>',
                     '<ADD_SYMBOL><tableConstraintDef>',
                     '<CHANGE_SYMBOL><COLUMN_SYMBOL>?<columnInternalRef><identifier><fieldDefinition><place>?',
                     '<MODIFY_SYMBOL><COLUMN_SYMBOL>?<columnInternalRef><fieldDefinition><place>?',
                     '<DROP_SYMBOL><alterListItem-2>',
                     '<DISABLE_SYMBOL><KEYS_SYMBOL>',
                     '<ENABLE_SYMBOL><KEYS_SYMBOL>',
                     '<ALTER_SYMBOL><COLUMN_SYMBOL>?<columnInternalRef><alterListItem-3>',
                     '<ALTER_SYMBOL><INDEX_SYMBOL><indexRef><visibility>',
                     '<ALTER_SYMBOL><CHECK_SYMBOL><identifier><constraintEnforcement>',
                     '<ALTER_SYMBOL><CONSTRAINT_SYMBOL><identifier><constraintEnforcement>',
                     '<RENAME_SYMBOL><COLUMN_SYMBOL><columnInternalRef><TO_SYMBOL><identifier>',
                     '<RENAME_SYMBOL><alterListItem-5>?<tableName>',
                     '<RENAME_SYMBOL><keyOrIndex><indexRef><TO_SYMBOL><indexName>',
                     '<CONVERT_SYMBOL><TO_SYMBOL><charset><alterListItem-6><collate>?',
                     '<FORCE_SYMBOL>',
                     '<ORDER_SYMBOL><BY_SYMBOL><alterOrderList>'],
 '<alterListItem-1>': ['<identifier><fieldDefinition><checkOrReferences>?<place>?',
                       '<OPEN_PAR_SYMBOL><tableElementList><CLOSE_PAR_SYMBOL>'],
 '<alterListItem-2>': ['<COLUMN_SYMBOL>?<columnInternalRef><restrict>?',
                       '<FOREIGN_SYMBOL><KEY_SYMBOL><columnInternalRef>',
                       '<PRIMARY_SYMBOL><KEY_SYMBOL>',
                       '<keyOrIndex><indexRef>',
                       '<CHECK_SYMBOL><identifier>',
                       '<CONSTRAINT_SYMBOL><identifier>'],
 '<alterListItem-3>': ['<SET_SYMBOL><DEFAULT_SYMBOL><alterListItem-4>',
                       '<DROP_SYMBOL><DEFAULT_SYMBOL>',
                       '<SET_SYMBOL><visibility>'],
 '<alterListItem-4>': ['<exprWithParentheses>', '<signedLiteralOrNull>'],
 '<alterListItem-5>': ['<TO_SYMBOL>', '<AS_SYMBOL>'],
 '<alterListItem-6>': ['<DEFAULT_SYMBOL>', '<charsetName>'],
 '<place>': ['<AFTER_SYMBOL><identifier>', '<FIRST_SYMBOL>'],
 '<restrict>': ['<RESTRICT_SYMBOL>', '<CASCADE_SYMBOL>'],
 '<alterOrderList>': ['<identifier><direction>?<alterOrderList-1>*'],
 '<alterOrderList-1>': ['<COMMA_SYMBOL><identifier><direction>?'],
 '<alterAlgorithmOption>': ['<ALGORITHM_SYMBOL><EQUAL_OPERATOR>?<alterAlgorithmOption-1>'],
 '<alterAlgorithmOption-1>': ['<DEFAULT_SYMBOL>', '<identifier>'],
 '<alterLockOption>': ['<LOCK_SYMBOL><EQUAL_OPERATOR>?<alterLockOption-1>'],
 '<alterLockOption-1>': ['<DEFAULT_SYMBOL>', '<identifier>'],
 '<indexLockAndAlgorithm>': ['<alterAlgorithmOption><alterLockOption>?',
                             '<alterLockOption><alterAlgorithmOption>?'],
 '<withValidation>': ['<withValidation-1><VALIDATION_SYMBOL>'],
 '<withValidation-1>': ['<WITH_SYMBOL>', '<WITHOUT_SYMBOL>'],
 '<removePartitioning>': ['<REMOVE_SYMBOL><PARTITIONING_SYMBOL>'],
 '<allOrPartitionNameList>': ['<ALL_SYMBOL>', '<identifierList>'],
 '<alterTablespace>': ['<TABLESPACE_SYMBOL><tablespaceRef><alterTablespace-1>'],
 '<alterTablespace-1>': ['<alterTablespace-2><DATAFILE_SYMBOL><textLiteral><alterTablespaceOptions>?',
                         '<RENAME_SYMBOL><TO_SYMBOL><identifier>',
                         '<alterTablespaceOptions>'],
 '<alterTablespace-2>': ['<ADD_SYMBOL>', '<DROP_SYMBOL>'],
 '<alterUndoTablespace>': ['<UNDO_SYMBOL><TABLESPACE_SYMBOL><tablespaceRef><SET_SYMBOL><alterUndoTablespace-1><undoTableSpaceOptions>?'],
 '<alterUndoTablespace-1>': ['<ACTIVE_SYMBOL>', '<INACTIVE_SYMBOL>'],
 '<undoTableSpaceOptions>': ['<undoTableSpaceOption><undoTableSpaceOptions-1>*'],
 '<undoTableSpaceOptions-1>': ['<COMMA_SYMBOL>?<undoTableSpaceOption>'],
 '<undoTableSpaceOption>': ['<tsOptionEngine>'],
 '<alterTablespaceOptions>': ['<alterTablespaceOption><alterTablespaceOptions-1>*'],
 '<alterTablespaceOptions-1>': ['<COMMA_SYMBOL>?<alterTablespaceOption>'],
 '<alterTablespaceOption>': ['<INITIAL_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>',
                             '<tsOptionAutoextendSize>',
                             '<tsOptionMaxSize>',
                             '<tsOptionEngine>',
                             '<tsOptionWait>',
                             '<tsOptionEncryption>',
                             '<tsOptionEngineAttribute>'],
 '<alterView>': ['<viewAlgorithm>?<definerClause>?<viewSuid>?<VIEW_SYMBOL><viewRef><viewTail>'],
 '<viewTail>': ['<columnInternalRefList>?<AS_SYMBOL><viewQueryBlock>'],
 '<viewQueryBlock>': ['<queryExpressionWithOptLockingClauses><viewCheckOption>?'],
 '<viewCheckOption>': ['<WITH_SYMBOL><viewCheckOption-1>?<CHECK_SYMBOL><OPTION_SYMBOL>'],
 '<viewCheckOption-1>': ['<CASCADED_SYMBOL>', '<LOCAL_SYMBOL>'],
 '<alterInstanceStatement>': ['<INSTANCE_SYMBOL><ROTATE_SYMBOL><textOrIdentifier><MASTER_SYMBOL><KEY_SYMBOL>',
                              '<alterInstanceStatement-1>'],
 '<alterInstanceStatement-1>': ['<RELOAD_SYMBOL><TLS_SYMBOL><alterInstanceStatement-2>',
                                '<alterInstanceStatement-4><identifier><identifier>',
                                '<RELOAD_SYMBOL><KEYRING_SYMBOL>'],
 '<alterInstanceStatement-2>': ['<NO_SYMBOL><ROLLBACK_SYMBOL><ON_SYMBOL><ERROR_SYMBOL>',
                                '<FOR_SYMBOL><CHANNEL_SYMBOL><identifier><alterInstanceStatement-3>?'],
 '<alterInstanceStatement-3>': ['<NO_SYMBOL><ROLLBACK_SYMBOL><ON_SYMBOL><ERROR_SYMBOL>'],
 '<alterInstanceStatement-4>': ['<ENABLE_SYMBOL>', '<DISABLE_SYMBOL>'],
 '<createStatement>': ['<CREATE_SYMBOL><createStatement-1>'],
 '<createStatement-1>': ['<createDatabase>',
                         '<createTable>',
                         '<createFunction>',
                         '<createProcedure>',
                         '<createUdf>',
                         '<createLogfileGroup>',
                         '<createView>',
                         '<createTrigger>',
                         '<createIndex>',
                         '<createServer>',
                         '<createTablespace>',
                         '<createEvent>',
                         '<createRole>',
                         '<createSpatialReference>',
                         '<createUndoTablespace>'],
 '<createDatabase>': ['<DATABASE_SYMBOL><ifNotExists>?<schemaName><createDatabaseOption>*'],
 '<createDatabaseOption>': ['<defaultCharset>',
                            '<defaultCollation>',
                            '<defaultEncryption>'],
 '<createTable>': ['<TEMPORARY_SYMBOL>?<TABLE_SYMBOL><ifNotExists>?<tableName><createTable-1>'],
 '<createTable-1>': ['<createTable-2>?<createTableOptionsEtc>?',
                     '<LIKE_SYMBOL><tableRef>',
                     '<OPEN_PAR_SYMBOL><LIKE_SYMBOL><tableRef><CLOSE_PAR_SYMBOL>'],
 '<createTable-2>': ['<OPEN_PAR_SYMBOL><tableElementList><CLOSE_PAR_SYMBOL>'],
 '<tableElementList>': ['<tableElement><tableElementList-1>*'],
 '<tableElementList-1>': ['<COMMA_SYMBOL><tableElement>'],
 '<tableElement>': ['<columnDefinition>', '<tableConstraintDef>'],
 '<duplicateAsQe>': ['<duplicateAsQe-1>?<asCreateQueryExpression>'],
 '<duplicateAsQe-1>': ['<REPLACE_SYMBOL>', '<IGNORE_SYMBOL>'],
 '<asCreateQueryExpression>': ['<AS_SYMBOL>?<queryExpressionWithOptLockingClauses>'],
 '<queryExpressionWithOptLockingClauses>': ['<queryExpression><lockingClauseList>?'],
 '<createProcedure>': ['<definerClause>?<PROCEDURE_SYMBOL><ifNotExists>?<procedureName><OPEN_PAR_SYMBOL><createProcedure-1>?<CLOSE_PAR_SYMBOL><routineCreateOption>*<storedRoutineBody>'],
 '<createProcedure-1>': ['<procedureParameter><createProcedure-2>*'],
 '<createProcedure-2>': ['<COMMA_SYMBOL><procedureParameter>'],
 '<routineString>': ['<textStringLiteral>', '<DOLLAR_QUOTED_STRING_TEXT>'],
 '<storedRoutineBody>': ['<compoundStatement>', '<AS_SYMBOL><routineString>'],
 '<createFunction>': ['<definerClause>?<FUNCTION_SYMBOL><ifNotExists>?<functionName><OPEN_PAR_SYMBOL><createFunction-1>?<CLOSE_PAR_SYMBOL><RETURNS_SYMBOL><typeWithOptCollate><routineCreateOption>*<storedRoutineBody>'],
 '<createFunction-1>': ['<functionParameter><createFunction-2>*'],
 '<createFunction-2>': ['<COMMA_SYMBOL><functionParameter>'],
 '<createUdf>': ['<AGGREGATE_SYMBOL>?<FUNCTION_SYMBOL><ifNotExists>?<udfName><RETURNS_SYMBOL><createUdf-1><SONAME_SYMBOL><textLiteral>'],
 '<createUdf-1>': ['<STRING_SYMBOL>',
                   '<INT_SYMBOL>',
                   '<REAL_SYMBOL>',
                   '<DECIMAL_SYMBOL>'],
 '<routineCreateOption>': ['<routineOption>',
                           '<NOT_SYMBOL>?<DETERMINISTIC_SYMBOL>'],
 '<routineAlterOptions>': ['<routineCreateOption>+'],
 '<routineOption>': ['<COMMENT_SYMBOL><textLiteral>',
                     '<LANGUAGE_SYMBOL><routineOption-1>',
                     '<NO_SYMBOL><SQL_SYMBOL>',
                     '<CONTAINS_SYMBOL><SQL_SYMBOL>',
                     '<READS_SYMBOL><SQL_SYMBOL><DATA_SYMBOL>',
                     '<MODIFIES_SYMBOL><SQL_SYMBOL><DATA_SYMBOL>',
                     '<SQL_SYMBOL><SECURITY_SYMBOL><routineOption-2>'],
 '<routineOption-1>': ['<SQL_SYMBOL>', '<identifier>'],
 '<routineOption-2>': ['<DEFINER_SYMBOL>', '<INVOKER_SYMBOL>'],
 '<createIndex>': ['<onlineOption>?<createIndex-1><indexLockAndAlgorithm>?'],
 '<createIndex-1>': ['<UNIQUE_SYMBOL>?<INDEX_SYMBOL><indexName><indexTypeClause>?<createIndexTarget><indexOption>*',
                     '<FULLTEXT_SYMBOL><INDEX_SYMBOL><indexName><createIndexTarget><fulltextIndexOption>*',
                     '<SPATIAL_SYMBOL><INDEX_SYMBOL><indexName><createIndexTarget><spatialIndexOption>*'],
 '<indexNameAndType>': ['<indexName>',
                        '<indexName>?<USING_SYMBOL><indexType>',
                        '<indexName><TYPE_SYMBOL><indexType>'],
 '<createIndexTarget>': ['<ON_SYMBOL><tableRef><keyListWithExpression>'],
 '<createLogfileGroup>': ['<LOGFILE_SYMBOL><GROUP_SYMBOL><logfileGroupName><ADD_SYMBOL><UNDOFILE_SYMBOL><textLiteral><logfileGroupOptions>?'],
 '<logfileGroupOptions>': ['<logfileGroupOption><logfileGroupOptions-1>*'],
 '<logfileGroupOptions-1>': ['<COMMA_SYMBOL>?<logfileGroupOption>'],
 '<logfileGroupOption>': ['<tsOptionInitialSize>',
                          '<tsOptionUndoRedoBufferSize>',
                          '<tsOptionNodegroup>',
                          '<tsOptionEngine>',
                          '<tsOptionWait>',
                          '<tsOptionComment>'],
 '<createServer>': ['<SERVER_SYMBOL><serverName><FOREIGN_SYMBOL><DATA_SYMBOL><WRAPPER_SYMBOL><textOrIdentifier><serverOptions>'],
 '<serverOptions>': ['<OPTIONS_SYMBOL><OPEN_PAR_SYMBOL><serverOption><serverOptions-1>*<CLOSE_PAR_SYMBOL>'],
 '<serverOptions-1>': ['<COMMA_SYMBOL><serverOption>'],
 '<serverOption>': ['<HOST_SYMBOL><textLiteral>',
                    '<DATABASE_SYMBOL><textLiteral>',
                    '<USER_SYMBOL><textLiteral>',
                    '<PASSWORD_SYMBOL><textLiteral>',
                    '<SOCKET_SYMBOL><textLiteral>',
                    '<OWNER_SYMBOL><textLiteral>',
                    '<PORT_SYMBOL><ulong_number>'],
 '<createTablespace>': ['<TABLESPACE_SYMBOL><tablespaceName><tsDataFileName><createTablespace-1>?<tablespaceOptions>?'],
 '<createTablespace-1>': ['<USE_SYMBOL><LOGFILE_SYMBOL><GROUP_SYMBOL><logfileGroupRef>'],
 '<createUndoTablespace>': ['<UNDO_SYMBOL><TABLESPACE_SYMBOL><tablespaceName><ADD_SYMBOL><tsDataFile><undoTableSpaceOptions>?'],
 '<tsDataFileName>': ['<ADD_SYMBOL><tsDataFile>', '<tsDataFileName-1>?'],
 '<tsDataFileName-1>': ['<ADD_SYMBOL><tsDataFile>'],
 '<tsDataFile>': ['<DATAFILE_SYMBOL><textLiteral>'],
 '<tablespaceOptions>': ['<tablespaceOption><tablespaceOptions-1>*'],
 '<tablespaceOptions-1>': ['<COMMA_SYMBOL>?<tablespaceOption>'],
 '<tablespaceOption>': ['<tsOptionInitialSize>',
                        '<tsOptionAutoextendSize>',
                        '<tsOptionMaxSize>',
                        '<tsOptionExtentSize>',
                        '<tsOptionNodegroup>',
                        '<tsOptionEngine>',
                        '<tsOptionWait>',
                        '<tsOptionComment>',
                        '<tsOptionFileblockSize>',
                        '<tsOptionEncryption>'],
 '<tsOptionInitialSize>': ['<INITIAL_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionUndoRedoBufferSize>': ['<tsOptionUndoRedoBufferSize-1><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionUndoRedoBufferSize-1>': ['<UNDO_BUFFER_SIZE_SYMBOL>',
                                    '<REDO_BUFFER_SIZE_SYMBOL>'],
 '<tsOptionAutoextendSize>': ['<AUTOEXTEND_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionMaxSize>': ['<MAX_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionExtentSize>': ['<EXTENT_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionNodegroup>': ['<NODEGROUP_SYMBOL><EQUAL_OPERATOR>?<real_ulong_number>'],
 '<tsOptionEngine>': ['<STORAGE_SYMBOL>?<ENGINE_SYMBOL><EQUAL_OPERATOR>?<engineRef>'],
 '<tsOptionWait>': ['<tsOptionWait-1>'],
 '<tsOptionWait-1>': ['<WAIT_SYMBOL>', '<NO_WAIT_SYMBOL>'],
 '<tsOptionComment>': ['<COMMENT_SYMBOL><EQUAL_OPERATOR>?<textLiteral>'],
 '<tsOptionFileblockSize>': ['<FILE_BLOCK_SIZE_SYMBOL><EQUAL_OPERATOR>?<sizeNumber>'],
 '<tsOptionEncryption>': ['<ENCRYPTION_SYMBOL><EQUAL_OPERATOR>?<textStringLiteral>'],
 '<tsOptionEngineAttribute>': ['<ENGINE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>'],
 '<createView>': ['<viewReplaceOrAlgorithm>?<definerClause>?<viewSuid>?<VIEW_SYMBOL><viewName><viewTail>'],
 '<viewReplaceOrAlgorithm>': ['<OR_SYMBOL><REPLACE_SYMBOL><viewAlgorithm>?',
                              '<viewAlgorithm>'],
 '<viewAlgorithm>': ['<ALGORITHM_SYMBOL><EQUAL_OPERATOR><viewAlgorithm-1>'],
 '<viewAlgorithm-1>': ['<UNDEFINED_SYMBOL>',
                       '<MERGE_SYMBOL>',
                       '<TEMPTABLE_SYMBOL>'],
 '<viewSuid>': ['<SQL_SYMBOL><SECURITY_SYMBOL><viewSuid-1>'],
 '<viewSuid-1>': ['<DEFINER_SYMBOL>', '<INVOKER_SYMBOL>'],
 '<createTrigger>': ['<definerClause>?<TRIGGER_SYMBOL><ifNotExists>?<triggerName><createTrigger-1><createTrigger-2><ON_SYMBOL><tableRef><FOR_SYMBOL><EACH_SYMBOL><ROW_SYMBOL><triggerFollowsPrecedesClause>?<compoundStatement>'],
 '<createTrigger-1>': ['<BEFORE_SYMBOL>', '<AFTER_SYMBOL>'],
 '<createTrigger-2>': ['<INSERT_SYMBOL>', '<UPDATE_SYMBOL>', '<DELETE_SYMBOL>'],
 '<triggerFollowsPrecedesClause>': ['<triggerFollowsPrecedesClause-1><textOrIdentifier>'],
 '<triggerFollowsPrecedesClause-1>': ['<FOLLOWS_SYMBOL>', '<PRECEDES_SYMBOL>'],
 '<createEvent>': ['<definerClause>?<EVENT_SYMBOL><ifNotExists>?<eventName><ON_SYMBOL><SCHEDULE_SYMBOL><schedule><createEvent-1>?<createEvent-2>?<createEvent-4>?<DO_SYMBOL><compoundStatement>'],
 '<createEvent-1>': ['<ON_SYMBOL><COMPLETION_SYMBOL><NOT_SYMBOL>?<PRESERVE_SYMBOL>'],
 '<createEvent-2>': ['<ENABLE_SYMBOL>', '<DISABLE_SYMBOL><createEvent-3>?'],
 '<createEvent-3>': ['<ON_SYMBOL><replica>'],
 '<createEvent-4>': ['<COMMENT_SYMBOL><textLiteral>'],
 '<createRole>': ['<ROLE_SYMBOL><ifNotExists>?<roleList>'],
 '<createSpatialReference>': ['<OR_SYMBOL><REPLACE_SYMBOL><SPATIAL_SYMBOL><REFERENCE_SYMBOL><SYSTEM_SYMBOL><real_ulonglong_number><srsAttribute>*',
                              '<SPATIAL_SYMBOL><REFERENCE_SYMBOL><SYSTEM_SYMBOL><ifNotExists>?<real_ulonglong_number><srsAttribute>*'],
 '<srsAttribute>': ['<NAME_SYMBOL><TEXT_SYMBOL><textStringNoLinebreak>',
                    '<DEFINITION_SYMBOL><TEXT_SYMBOL><textStringNoLinebreak>',
                    '<ORGANIZATION_SYMBOL><textStringNoLinebreak><IDENTIFIED_SYMBOL><BY_SYMBOL><real_ulonglong_number>',
                    '<DESCRIPTION_SYMBOL><TEXT_SYMBOL><textStringNoLinebreak>'],
 '<dropStatement>': ['<DROP_SYMBOL><dropStatement-1>'],
 '<dropStatement-1>': ['<dropDatabase>',
                       '<dropEvent>',
                       '<dropFunction>',
                       '<dropProcedure>',
                       '<dropIndex>',
                       '<dropLogfileGroup>',
                       '<dropServer>',
                       '<dropTable>',
                       '<dropTableSpace>',
                       '<dropTrigger>',
                       '<dropView>',
                       '<dropRole>',
                       '<dropSpatialReference>',
                       '<dropUndoTablespace>'],
 '<dropDatabase>': ['<DATABASE_SYMBOL><ifExists>?<schemaRef>'],
 '<dropEvent>': ['<EVENT_SYMBOL><ifExists>?<eventRef>'],
 '<dropFunction>': ['<FUNCTION_SYMBOL><ifExists>?<functionRef>'],
 '<dropProcedure>': ['<PROCEDURE_SYMBOL><ifExists>?<procedureRef>'],
 '<dropIndex>': ['<onlineOption>?<INDEX_SYMBOL><indexRef><ON_SYMBOL><tableRef><indexLockAndAlgorithm>?'],
 '<dropLogfileGroup>': ['<LOGFILE_SYMBOL><GROUP_SYMBOL><logfileGroupRef><dropLogfileGroup-1>?'],
 '<dropLogfileGroup-1>': ['<dropLogfileGroupOption><dropLogfileGroup-2>*'],
 '<dropLogfileGroup-2>': ['<COMMA_SYMBOL>?<dropLogfileGroupOption>'],
 '<dropLogfileGroupOption>': ['<tsOptionWait>', '<tsOptionEngine>'],
 '<dropServer>': ['<SERVER_SYMBOL><ifExists>?<serverRef>'],
 '<dropTable>': ['<TEMPORARY_SYMBOL>?<dropTable-1><ifExists>?<tableRefList><dropTable-2>?'],
 '<dropTable-1>': ['<TABLE_SYMBOL>', '<TABLES_SYMBOL>'],
 '<dropTable-2>': ['<RESTRICT_SYMBOL>', '<CASCADE_SYMBOL>'],
 '<dropTableSpace>': ['<TABLESPACE_SYMBOL><tablespaceRef><dropTableSpace-1>?'],
 '<dropTableSpace-1>': ['<dropLogfileGroupOption><dropTableSpace-2>*'],
 '<dropTableSpace-2>': ['<COMMA_SYMBOL>?<dropLogfileGroupOption>'],
 '<dropTrigger>': ['<TRIGGER_SYMBOL><ifExists>?<triggerRef>'],
 '<dropView>': ['<VIEW_SYMBOL><ifExists>?<viewRefList><dropView-1>?'],
 '<dropView-1>': ['<RESTRICT_SYMBOL>', '<CASCADE_SYMBOL>'],
 '<dropRole>': ['<ROLE_SYMBOL><ifExists>?<roleList>'],
 '<dropSpatialReference>': ['<SPATIAL_SYMBOL><REFERENCE_SYMBOL><SYSTEM_SYMBOL><ifExists>?<real_ulonglong_number>'],
 '<dropUndoTablespace>': ['<UNDO_SYMBOL><TABLESPACE_SYMBOL><tablespaceRef><undoTableSpaceOptions>?'],
 '<renameTableStatement>': ['<RENAME_SYMBOL><renameTableStatement-1><renamePair><renameTableStatement-2>*'],
 '<renameTableStatement-1>': ['<TABLE_SYMBOL>', '<TABLES_SYMBOL>'],
 '<renameTableStatement-2>': ['<COMMA_SYMBOL><renamePair>'],
 '<renamePair>': ['<tableRef><TO_SYMBOL><tableName>'],
 '<truncateTableStatement>': ['<TRUNCATE_SYMBOL><TABLE_SYMBOL>?<tableRef>'],
 '<importStatement>': ['<IMPORT_SYMBOL><TABLE_SYMBOL><FROM_SYMBOL><textStringLiteralList>'],
 '<callStatement>': ['<CALL_SYMBOL><procedureRef><callStatement-1>?'],
 '<callStatement-1>': ['<OPEN_PAR_SYMBOL><exprList>?<CLOSE_PAR_SYMBOL>'],
 '<deleteStatement>': ['<withClause>?<DELETE_SYMBOL><deleteStatementOption>*<deleteStatement-1>'],
 '<deleteStatement-1>': ['<FROM_SYMBOL><deleteStatement-2>',
                         '<tableAliasRefList><FROM_SYMBOL><tableReferenceList><whereClause>?'],
 '<deleteStatement-2>': ['<tableAliasRefList><USING_SYMBOL><tableReferenceList><whereClause>?',
                         '<tableRef><deleteStatement-3>?<partitionDelete>?<whereClause>?<orderClause>?<simpleLimitClause>?'],
 '<deleteStatement-3>': ['<tableAlias>'],
 '<partitionDelete>': ['<PARTITION_SYMBOL><OPEN_PAR_SYMBOL><identifierList><CLOSE_PAR_SYMBOL>'],
 '<deleteStatementOption>': ['<QUICK_SYMBOL>',
                             '<LOW_PRIORITY_SYMBOL>',
                             '<QUICK_SYMBOL>',
                             '<IGNORE_SYMBOL>'],
 '<doStatement>': ['<DO_SYMBOL><selectItemList>'],
 '<handlerStatement>': ['<HANDLER_SYMBOL><handlerStatement-1>'],
 '<handlerStatement-1>': ['<tableRef><OPEN_SYMBOL><tableAlias>?',
                          '<identifier><handlerStatement-2>'],
 '<handlerStatement-2>': ['<CLOSE_SYMBOL>',
                          '<READ_SYMBOL><handlerReadOrScan><whereClause>?<limitClause>?'],
 '<handlerReadOrScan>': ['<handlerReadOrScan-1>',
                         '<identifier><handlerReadOrScan-2>'],
 '<handlerReadOrScan-1>': ['<FIRST_SYMBOL>', '<NEXT_SYMBOL>'],
 '<handlerReadOrScan-2>': ['<handlerReadOrScan-3>',
                           '<handlerReadOrScan-4><OPEN_PAR_SYMBOL><values><CLOSE_PAR_SYMBOL>'],
 '<handlerReadOrScan-3>': ['<FIRST_SYMBOL>',
                           '<NEXT_SYMBOL>',
                           '<PREV_SYMBOL>',
                           '<LAST_SYMBOL>'],
 '<handlerReadOrScan-4>': ['<EQUAL_OPERATOR>',
                           '<LESS_THAN_OPERATOR>',
                           '<GREATER_THAN_OPERATOR>',
                           '<LESS_OR_EQUAL_OPERATOR>',
                           '<GREATER_OR_EQUAL_OPERATOR>'],
 '<insertStatement>': ['<INSERT_SYMBOL><insertLockOption>?<IGNORE_SYMBOL>?<INTO_SYMBOL>?<tableRef><usePartition>?<insertStatement-1><insertUpdateList>?'],
 '<insertStatement-1>': ['<insertFromConstructor><valuesReference>?',
                         '<SET_SYMBOL><updateList><valuesReference>?',
                         '<insertQueryExpression>'],
 '<insertLockOption>': ['<LOW_PRIORITY_SYMBOL>',
                        '<DELAYED_SYMBOL>',
                        '<HIGH_PRIORITY_SYMBOL>'],
 '<insertFromConstructor>': ['<insertFromConstructor-1>?<insertValues>'],
 '<insertFromConstructor-1>': ['<OPEN_PAR_SYMBOL><fields>?<CLOSE_PAR_SYMBOL>'],
 '<fields>': ['<insertIdentifier><fields-1>*'],
 '<fields-1>': ['<COMMA_SYMBOL><insertIdentifier>'],
 '<insertValues>': ['<insertValues-1><valueList>'],
 '<insertValues-1>': ['<VALUES_SYMBOL>', '<VALUE_SYMBOL>'],
 '<insertQueryExpression>': ['<queryExpression>',
                             '<queryExpressionParens>',
                             '<insertQueryExpression-1>?<queryExpressionWithOptLockingClauses>'],
 '<insertQueryExpression-1>': ['<OPEN_PAR_SYMBOL><fields>?<CLOSE_PAR_SYMBOL>'],
 '<valueList>': ['<OPEN_PAR_SYMBOL><values>?<CLOSE_PAR_SYMBOL><valueList-1>*'],
 '<valueList-1>': ['<COMMA_SYMBOL><OPEN_PAR_SYMBOL><values>?<CLOSE_PAR_SYMBOL>'],
 '<values>': ['<values-1><values-2>*'],
 '<values-1>': ['<expr>', '<DEFAULT_SYMBOL>'],
 '<values-2>': ['<COMMA_SYMBOL><values-3>'],
 '<values-3>': ['<expr>', '<DEFAULT_SYMBOL>'],
 '<valuesReference>': ['<AS_SYMBOL><identifier><columnInternalRefList>?'],
 '<insertUpdateList>': ['<ON_SYMBOL><DUPLICATE_SYMBOL><KEY_SYMBOL><UPDATE_SYMBOL><updateList>'],
 '<loadStatement>': ['<LOAD_SYMBOL><dataOrXml><loadDataLock>?<loadFrom>?<LOCAL_SYMBOL>?<loadSourceType>?<textStringLiteral><sourceCount>?<sourceOrder>?<loadStatement-1>?<INTO_SYMBOL><TABLE_SYMBOL><tableRef><usePartition>?<charsetClause>?<xmlRowsIdentifiedBy>?<fieldsClause>?<linesClause>?<loadDataFileTail><loadParallel>?<loadMemory>?<loadAlgorithm>?'],
 '<loadStatement-1>': ['<REPLACE_SYMBOL>', '<IGNORE_SYMBOL>'],
 '<dataOrXml>': ['<DATA_SYMBOL>', '<XML_SYMBOL>'],
 '<loadDataLock>': ['<LOW_PRIORITY_SYMBOL>', '<CONCURRENT_SYMBOL>'],
 '<loadFrom>': ['<FROM_SYMBOL>'],
 '<loadSourceType>': ['<INFILE_SYMBOL>', '<loadSourceType-1>'],
 '<loadSourceType-1>': ['<URL_SYMBOL>', '<S3_SYMBOL>'],
 '<sourceCount>': ['<sourceCount-1>'],
 '<sourceCount-1>': ['<COUNT_SYMBOL><INT_NUMBER>',
                     '<pureIdentifier><INT_NUMBER>'],
 '<sourceOrder>': ['<IN_SYMBOL><PRIMARY_SYMBOL><KEY_SYMBOL><ORDER_SYMBOL>'],
 '<xmlRowsIdentifiedBy>': ['<ROWS_SYMBOL><IDENTIFIED_SYMBOL><BY_SYMBOL><textString>'],
 '<loadDataFileTail>': ['<loadDataFileTail-1>?<loadDataFileTargetList>?<loadDataFileTail-3>?'],
 '<loadDataFileTail-1>': ['<IGNORE_SYMBOL><INT_NUMBER><loadDataFileTail-2>'],
 '<loadDataFileTail-2>': ['<LINES_SYMBOL>', '<ROWS_SYMBOL>'],
 '<loadDataFileTail-3>': ['<SET_SYMBOL><updateList>'],
 '<loadDataFileTargetList>': ['<OPEN_PAR_SYMBOL><fieldOrVariableList>?<CLOSE_PAR_SYMBOL>'],
 '<fieldOrVariableList>': ['<fieldOrVariableList-1><fieldOrVariableList-2>*'],
 '<fieldOrVariableList-1>': ['<columnRef>',
                             '<AT_SIGN_SYMBOL><textOrIdentifier>',
                             '<AT_AT_SIGN_SYMBOL>'],
 '<fieldOrVariableList-2>': ['<COMMA_SYMBOL><fieldOrVariableList-3>'],
 '<fieldOrVariableList-3>': ['<columnRef>',
                             '<AT_SIGN_SYMBOL><textOrIdentifier>',
                             '<AT_TEXT_SUFFIX>',
                             '<AT_AT_SIGN_SYMBOL>'],
 '<loadAlgorithm>': ['<ALGORITHM_SYMBOL><EQUAL_OPERATOR><BULK_SYMBOL>'],
 '<loadParallel>': ['<PARALLEL_SYMBOL><EQUAL_OPERATOR><INT_NUMBER>'],
 '<loadMemory>': ['<MEMORY_SYMBOL><EQUAL_OPERATOR><sizeNumber>'],
 '<replaceStatement>': ['<REPLACE_SYMBOL><replaceStatement-1>?<INTO_SYMBOL>?<tableRef><usePartition>?<replaceStatement-2>'],
 '<replaceStatement-1>': ['<LOW_PRIORITY_SYMBOL>', '<DELAYED_SYMBOL>'],
 '<replaceStatement-2>': ['<insertFromConstructor>',
                          '<SET_SYMBOL><updateList>',
                          '<insertQueryExpression>'],
 '<selectStatement>': ['<queryExpression><lockingClauseList>?',
                       '<selectStatementWithInto>'],
 '<selectStatementWithInto>': ['<OPEN_PAR_SYMBOL><selectStatementWithInto><CLOSE_PAR_SYMBOL>',
                               '<queryExpression><intoClause><lockingClauseList>?',
                               '<queryExpression><lockingClauseList><intoClause>',
                               '<queryExpressionParens><intoClause>'],
 '<queryExpression>': ['<withClause>?<queryExpressionBody><orderClause>?<limitClause>?'],
 '<queryExpressionBody>': ['<queryExpressionBody-1><queryExpressionBody-2>*'],
 '<queryExpressionBody-1>': ['<queryPrimary>', '<queryExpressionParens>'],
 '<queryExpressionBody-2>': ['<queryExpressionBody-3><unionOption>?<queryExpressionBody>'],
 '<queryExpressionBody-3>': ['<UNION_SYMBOL>', '<queryExpressionBody-4>'],
 '<queryExpressionBody-4>': ['<EXCEPT_SYMBOL>', '<INTERSECT_SYMBOL>'],
 '<queryExpressionParens>': ['<OPEN_PAR_SYMBOL><queryExpressionParens-1><CLOSE_PAR_SYMBOL>'],
 '<queryExpressionParens-1>': ['<queryExpressionParens>',
                               '<queryExpressionWithOptLockingClauses>'],
 '<queryPrimary>': ['<querySpecification>',
                    '<tableValueConstructor>',
                    '<explicitTable>'],
 '<querySpecification>': ['<SELECT_SYMBOL><selectOption>*<selectItemList><intoClause>?<fromClause>?<whereClause>?<groupByClause>?<havingClause>?<windowClause>?<qualifyClause>?'],
 '<subquery>': ['<queryExpressionParens>'],
 '<querySpecOption>': ['<ALL_SYMBOL>',
                       '<DISTINCT_SYMBOL>',
                       '<STRAIGHT_JOIN_SYMBOL>',
                       '<HIGH_PRIORITY_SYMBOL>',
                       '<SQL_SMALL_RESULT_SYMBOL>',
                       '<SQL_BIG_RESULT_SYMBOL>',
                       '<SQL_BUFFER_RESULT_SYMBOL>',
                       '<SQL_CALC_FOUND_ROWS_SYMBOL>'],
 '<limitClause>': ['<LIMIT_SYMBOL><limitOptions>'],
 '<simpleLimitClause>': ['<LIMIT_SYMBOL><limitOption>'],
 '<limitOptions>': ['<limitOption><limitOptions-1>?'],
 '<limitOptions-1>': ['<limitOptions-2><limitOption>'],
 '<limitOptions-2>': ['<COMMA_SYMBOL>', '<OFFSET_SYMBOL>'],
 '<limitOption>': ['<identifier>', '<limitOption-1>'],
 '<limitOption-1>': ['<PARAM_MARKER>',
                     'ULONGLONG_NUMBER',
                     'LONG_NUMBER',
                     '<INT_NUMBER>'],
 '<intoClause>': ['<INTO_SYMBOL><intoClause-1>'],
 '<intoClause-1>': ['<OUTFILE_SYMBOL><textStringLiteral><charsetClause>?<fieldsClause>?<linesClause>?',
                    '<DUMPFILE_SYMBOL><textStringLiteral>',
                    '<intoClause-2><intoClause-3>*'],
 '<intoClause-2>': ['<textOrIdentifier>', '<userVariable>'],
 '<intoClause-3>': ['<COMMA_SYMBOL><intoClause-4>'],
 '<intoClause-4>': ['<textOrIdentifier>', '<userVariable>'],
 '<havingClause>': ['<HAVING_SYMBOL><expr>'],
 '<qualifyClause>': ['<QUALIFY_SYMBOL><expr>'],
 '<windowClause>': ['<WINDOW_SYMBOL><windowDefinition><windowClause-1>*'],
 '<windowClause-1>': ['<COMMA_SYMBOL><windowDefinition>'],
 '<windowDefinition>': ['<windowName><AS_SYMBOL><windowSpec>'],
 '<windowSpec>': ['<OPEN_PAR_SYMBOL><windowSpecDetails><CLOSE_PAR_SYMBOL>'],
 '<windowSpecDetails>': ['<windowName>?<windowSpecDetails-1>?<orderClause>?<windowFrameClause>?'],
 '<windowSpecDetails-1>': ['<PARTITION_SYMBOL><BY_SYMBOL><orderList>'],
 '<windowFrameClause>': ['<windowFrameUnits><windowFrameExtent><windowFrameExclusion>?'],
 '<windowFrameUnits>': ['<ROWS_SYMBOL>', '<RANGE_SYMBOL>', '<GROUPS_SYMBOL>'],
 '<windowFrameExtent>': ['<windowFrameStart>', '<windowFrameBetween>'],
 '<windowFrameStart>': ['<UNBOUNDED_SYMBOL><PRECEDING_SYMBOL>',
                        '<ulonglongNumber><PRECEDING_SYMBOL>',
                        '<PARAM_MARKER><PRECEDING_SYMBOL>',
                        '<INTERVAL_SYMBOL><expr><interval><PRECEDING_SYMBOL>',
                        '<CURRENT_SYMBOL><ROW_SYMBOL>'],
 '<windowFrameBetween>': ['<BETWEEN_SYMBOL><windowFrameBound><AND_SYMBOL><windowFrameBound>'],
 '<windowFrameBound>': ['<windowFrameStart>',
                        '<UNBOUNDED_SYMBOL><FOLLOWING_SYMBOL>',
                        '<ulonglongNumber><FOLLOWING_SYMBOL>',
                        '<PARAM_MARKER><FOLLOWING_SYMBOL>',
                        '<INTERVAL_SYMBOL><expr><interval><FOLLOWING_SYMBOL>'],
 '<windowFrameExclusion>': ['<EXCLUDE_SYMBOL><windowFrameExclusion-1>'],
 '<windowFrameExclusion-1>': ['<CURRENT_SYMBOL><ROW_SYMBOL>',
                              '<GROUP_SYMBOL>',
                              '<TIES_SYMBOL>',
                              '<NO_SYMBOL><OTHERS_SYMBOL>'],
 '<withClause>': ['<WITH_SYMBOL><RECURSIVE_SYMBOL>?<commonTableExpression><withClause-1>*'],
 '<withClause-1>': ['<COMMA_SYMBOL><commonTableExpression>'],
 '<commonTableExpression>': ['<identifier><columnInternalRefList>?<AS_SYMBOL><subquery>'],
 '<groupByClause>': ['<GROUP_SYMBOL><BY_SYMBOL><orderList><olapOption>?',
                     '<GROUP_SYMBOL><BY_SYMBOL><groupByClause-1><OPEN_PAR_SYMBOL><groupList><CLOSE_PAR_SYMBOL>'],
 '<groupByClause-1>': ['<ROLLUP_SYMBOL>', '<CUBE_SYMBOL>'],
 '<olapOption>': ['<WITH_SYMBOL><ROLLUP_SYMBOL>'],
 '<orderClause>': ['<ORDER_SYMBOL><BY_SYMBOL><orderList>'],
 '<direction>': ['<ASC_SYMBOL>', '<DESC_SYMBOL>'],
 '<fromClause>': ['<FROM_SYMBOL><fromClause-1>'],
 '<fromClause-1>': ['<DUAL_SYMBOL>', '<tableReferenceList>'],
 '<tableReferenceList>': ['<tableReference><tableReferenceList-1>*'],
 '<tableReferenceList-1>': ['<COMMA_SYMBOL><tableReference>'],
 '<tableValueConstructor>': ['<VALUES_SYMBOL><rowValueExplicit><tableValueConstructor-1>*'],
 '<tableValueConstructor-1>': ['<COMMA_SYMBOL><rowValueExplicit>'],
 '<explicitTable>': ['<TABLE_SYMBOL><tableRef>'],
 '<rowValueExplicit>': ['<ROW_SYMBOL><OPEN_PAR_SYMBOL><values>?<CLOSE_PAR_SYMBOL>'],
 '<selectOption>': ['<querySpecOption>', '<SQL_NO_CACHE_SYMBOL>'],
 '<lockingClauseList>': ['<lockingClause>+'],
 '<lockingClause>': ['<FOR_SYMBOL><lockStrengh><lockingClause-1>?<lockedRowAction>?',
                     '<LOCK_SYMBOL><IN_SYMBOL><SHARE_SYMBOL><MODE_SYMBOL>'],
 '<lockingClause-1>': ['<OF_SYMBOL><tableAliasRefList>'],
 '<lockStrengh>': ['<UPDATE_SYMBOL>', '<SHARE_SYMBOL>'],
 '<lockedRowAction>': ['<SKIP_SYMBOL><LOCKED_SYMBOL>', '<NOWAIT_SYMBOL>'],
 '<selectItemList>': ['<selectItemList-1><selectItemList-2>*'],
 '<selectItemList-1>': ['<selectItem>', '<MULT_OPERATOR>'],
 '<selectItemList-2>': ['<COMMA_SYMBOL><selectItem>'],
 '<selectItem>': ['<tableWild>', '<expr><selectAlias>?'],
 '<selectAlias>': ['<AS_SYMBOL>?<selectAlias-1>'],
 '<selectAlias-1>': ['<identifier>', '<textStringLiteral>'],
 '<whereClause>': ['<WHERE_SYMBOL><expr>'],
 '<tableReference>': ['<tableReference-1><joinedTable>*'],
 '<tableReference-1>': ['<tableFactor>',
                        '<OPEN_CURLY_SYMBOL><tableReference-2><escapedTableReference><CLOSE_CURLY_SYMBOL>'],
 '<tableReference-2>': ['<identifier>', '<OJ_SYMBOL>'],
 '<escapedTableReference>': ['<tableFactor><joinedTable>*'],
 '<joinedTable>': ['<innerJoinType><tableReference><joinedTable-1>?',
                   '<outerJoinType><tableReference><joinedTable-2>',
                   '<naturalJoinType><tableFactor>'],
 '<joinedTable-1>': ['<ON_SYMBOL><expr>',
                     '<USING_SYMBOL><identifierListWithParentheses>'],
 '<joinedTable-2>': ['<ON_SYMBOL><expr>',
                     '<USING_SYMBOL><identifierListWithParentheses>'],
 '<naturalJoinType>': ['<NATURAL_SYMBOL><INNER_SYMBOL>?<JOIN_SYMBOL>',
                       '<NATURAL_SYMBOL><naturalJoinType-1><OUTER_SYMBOL>?<JOIN_SYMBOL>'],
 '<naturalJoinType-1>': ['<LEFT_SYMBOL>', '<RIGHT_SYMBOL>'],
 '<innerJoinType>': ['<innerJoinType-1>?<JOIN_SYMBOL>',
                     '<STRAIGHT_JOIN_SYMBOL>'],
 '<innerJoinType-1>': ['<INNER_SYMBOL>', '<CROSS_SYMBOL>'],
 '<outerJoinType>': ['<outerJoinType-1><OUTER_SYMBOL>?<JOIN_SYMBOL>'],
 '<outerJoinType-1>': ['<LEFT_SYMBOL>', '<RIGHT_SYMBOL>'],
 '<tableFactor>': ['<singleTable>',
                   '<singleTableParens>',
                   '<derivedTable>',
                   '<tableReferenceListParens>',
                   '<tableFunction>'],
 '<singleTable>': ['<tableRef><usePartition>?<tableAlias>?<indexHintList>?<tablesampleClause>?'],
 '<singleTableParens>': ['<OPEN_PAR_SYMBOL><singleTableParens-1><CLOSE_PAR_SYMBOL>'],
 '<singleTableParens-1>': ['<singleTable>', '<singleTableParens>'],
 '<derivedTable>': ['<subquery><tableAlias>?<columnInternalRefList>?',
                    '<LATERAL_SYMBOL><subquery><tableAlias>?<columnInternalRefList>?'],
 '<tableReferenceListParens>': ['<OPEN_PAR_SYMBOL><tableReferenceListParens-1><CLOSE_PAR_SYMBOL>'],
 '<tableReferenceListParens-1>': ['<tableReferenceList>',
                                  '<tableReferenceListParens>'],
 '<tableFunction>': ['<JSON_TABLE_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><textStringLiteral><columnsClause><CLOSE_PAR_SYMBOL><tableAlias>?'],
 '<columnsClause>': ['<COLUMNS_SYMBOL><OPEN_PAR_SYMBOL><jtColumn><columnsClause-1>*<CLOSE_PAR_SYMBOL>'],
 '<columnsClause-1>': ['<COMMA_SYMBOL><jtColumn>'],
 '<jtColumn>': ['<identifier><FOR_SYMBOL><ORDINALITY_SYMBOL>',
                '<identifier><dataType><jtColumn-1>?<EXISTS_SYMBOL>?<PATH_SYMBOL><textStringLiteral><onEmptyOrErrorJsonTable>?',
                '<NESTED_SYMBOL><PATH_SYMBOL><textStringLiteral><columnsClause>'],
 '<jtColumn-1>': ['<collate>'],
 '<onEmptyOrError>': ['<onEmpty><onError>?', '<onError>'],
 '<onEmptyOrErrorJsonTable>': ['<onEmptyOrError>', '<onError><onEmpty>'],
 '<onEmpty>': ['<jsonOnResponse><ON_SYMBOL><EMPTY_SYMBOL>'],
 '<onError>': ['<jsonOnResponse><ON_SYMBOL><ERROR_SYMBOL>'],
 '<jsonOnResponse>': ['<ERROR_SYMBOL>',
                      '<NULL_SYMBOL>',
                      '<DEFAULT_SYMBOL><textStringLiteral>'],
 '<unionOption>': ['<DISTINCT_SYMBOL>', '<ALL_SYMBOL>'],
 '<tableAlias>': ['<tableAlias-1>?<identifier>'],
 '<tableAlias-1>': ['<AS_SYMBOL>', '<EQUAL_OPERATOR>'],
 '<indexHintList>': ['<indexHint><indexHintList-1>*'],
 '<indexHintList-1>': ['<COMMA_SYMBOL><indexHint>'],
 '<indexHint>': ['<indexHintType><keyOrIndex><indexHintClause>?<OPEN_PAR_SYMBOL><indexList><CLOSE_PAR_SYMBOL>',
                 '<USE_SYMBOL><keyOrIndex><indexHintClause>?<OPEN_PAR_SYMBOL><indexList>?<CLOSE_PAR_SYMBOL>'],
 '<indexHintType>': ['<FORCE_SYMBOL>', '<IGNORE_SYMBOL>'],
 '<keyOrIndex>': ['<KEY_SYMBOL>', '<INDEX_SYMBOL>'],
 '<indexHintClause>': ['<FOR_SYMBOL><indexHintClause-1>'],
 '<indexHintClause-1>': ['<JOIN_SYMBOL>',
                         '<ORDER_SYMBOL><BY_SYMBOL>',
                         '<GROUP_SYMBOL><BY_SYMBOL>'],
 '<indexList>': ['<indexListElement><indexList-1>*'],
 '<indexList-1>': ['<COMMA_SYMBOL><indexListElement>'],
 '<indexListElement>': ['<identifier>', '<PRIMARY_SYMBOL>'],
 '<updateStatement>': ['<withClause>?<UPDATE_SYMBOL><LOW_PRIORITY_SYMBOL>?<IGNORE_SYMBOL>?<tableReferenceList><SET_SYMBOL><updateList><whereClause>?<orderClause>?<simpleLimitClause>?'],
 '<transactionOrLockingStatement>': ['<transactionStatement>',
                                     '<savepointStatement>',
                                     '<lockStatement>',
                                     '<xaStatement>'],
 '<transactionStatement>': ['<START_SYMBOL><TRANSACTION_SYMBOL><startTransactionOptionList>*',
                            '<COMMIT_SYMBOL><WORK_SYMBOL>?<transactionStatement-1>?<transactionStatement-2>?'],
 '<transactionStatement-1>': ['<AND_SYMBOL><NO_SYMBOL>?<CHAIN_SYMBOL>'],
 '<transactionStatement-2>': ['<NO_SYMBOL>?<RELEASE_SYMBOL>'],
 '<beginWork>': ['<BEGIN_SYMBOL><WORK_SYMBOL>?'],
 '<startTransactionOptionList>': ['<WITH_SYMBOL><CONSISTENT_SYMBOL><SNAPSHOT_SYMBOL>',
                                  '<READ_SYMBOL><startTransactionOptionList-1>'],
 '<startTransactionOptionList-1>': ['<WRITE_SYMBOL>', '<ONLY_SYMBOL>'],
 '<savepointStatement>': ['<SAVEPOINT_SYMBOL><identifier>',
                          '<ROLLBACK_SYMBOL><WORK_SYMBOL>?<savepointStatement-1>',
                          '<RELEASE_SYMBOL><SAVEPOINT_SYMBOL><identifier>'],
 '<savepointStatement-1>': ['<TO_SYMBOL><SAVEPOINT_SYMBOL>?<identifier>',
                            '<savepointStatement-2>?<savepointStatement-3>?'],
 '<savepointStatement-2>': ['<AND_SYMBOL><NO_SYMBOL>?<CHAIN_SYMBOL>'],
 '<savepointStatement-3>': ['<NO_SYMBOL>?<RELEASE_SYMBOL>'],
 '<lockStatement>': ['<LOCK_SYMBOL><lockStatement-1><lockItem><lockStatement-2>*',
                     '<LOCK_SYMBOL><INSTANCE_SYMBOL><FOR_SYMBOL><BACKUP_SYMBOL>',
                     '<UNLOCK_SYMBOL><lockStatement-3>'],
 '<lockStatement-1>': ['<TABLES_SYMBOL>', '<TABLE_SYMBOL>'],
 '<lockStatement-2>': ['<COMMA_SYMBOL><lockItem>'],
 '<lockStatement-3>': ['<TABLES_SYMBOL>',
                       '<TABLE_SYMBOL>',
                       '<INSTANCE_SYMBOL>'],
 '<lockItem>': ['<tableRef><tableAlias>?<lockOption>'],
 '<lockOption>': ['<READ_SYMBOL><LOCAL_SYMBOL>?',
                  '<LOW_PRIORITY_SYMBOL>?<WRITE_SYMBOL>'],
 '<xaStatement>': ['<XA_SYMBOL><xaStatement-1>'],
 '<xaStatement-1>': ['<xaStatement-2><xid><xaStatement-3>?',
                     '<END_SYMBOL><xid><xaStatement-4>?',
                     '<PREPARE_SYMBOL><xid>',
                     '<COMMIT_SYMBOL><xid><xaStatement-6>?',
                     '<ROLLBACK_SYMBOL><xid>',
                     '<RECOVER_SYMBOL><xaConvert>?'],
 '<xaStatement-2>': ['<START_SYMBOL>', '<BEGIN_SYMBOL>'],
 '<xaStatement-3>': ['<JOIN_SYMBOL>', '<RESUME_SYMBOL>'],
 '<xaStatement-4>': ['<SUSPEND_SYMBOL><xaStatement-5>?'],
 '<xaStatement-5>': ['<FOR_SYMBOL><MIGRATE_SYMBOL>'],
 '<xaStatement-6>': ['<ONE_SYMBOL><PHASE_SYMBOL>'],
 '<xaConvert>': ['<CONVERT_SYMBOL><XID_SYMBOL>'],
 '<xid>': ['<textString><xid-1>?'],
 '<xid-1>': ['<COMMA_SYMBOL><textString><xid-2>?'],
 '<xid-2>': ['<COMMA_SYMBOL><ulong_number>'],
 '<replicationStatement>': ['<PURGE_SYMBOL><purgeOptions>',
                            '<changeSource>',
                            '<RESET_SYMBOL><resetOption><replicationStatement-1>*',
                            '<RESET_SYMBOL><PERSIST_SYMBOL><ifExistsIdentifier>?',
                            '<startReplicaStatement>',
                            '<stopReplicaStatement>',
                            '<changeReplication>',
                            '<replicationLoad>',
                            '<groupReplication>'],
 '<replicationStatement-1>': ['<COMMA_SYMBOL><resetOption>'],
 '<purgeOptions>': ['<purgeOptions-1><LOGS_SYMBOL><purgeOptions-2>'],
 '<purgeOptions-1>': ['<BINARY_SYMBOL>', '<MASTER_SYMBOL>'],
 '<purgeOptions-2>': ['<TO_SYMBOL><textLiteral>', '<BEFORE_SYMBOL><expr>'],
 '<resetOption>': ['<masterOrBinaryLogsAndGtids><sourceResetOptions>?',
                   '<replica><ALL_SYMBOL>?<channel>?'],
 '<masterOrBinaryLogsAndGtids>': ['<MASTER_SYMBOL>',
                                  '<BINARY_SYMBOL><LOGS_SYMBOL><AND_SYMBOL><GTIDS_SYMBOL>'],
 '<sourceResetOptions>': ['<TO_SYMBOL><real_ulonglong_number>'],
 '<replicationLoad>': ['<LOAD_SYMBOL><replicationLoad-1><FROM_SYMBOL><MASTER_SYMBOL>'],
 '<replicationLoad-1>': ['<DATA_SYMBOL>', '<TABLE_SYMBOL><tableRef>'],
 '<changeReplicationSource>': ['<MASTER_SYMBOL>',
                               '<REPLICATION_SYMBOL><SOURCE_SYMBOL>'],
 '<changeSource>': ['<CHANGE_SYMBOL><changeReplicationSource><TO_SYMBOL><sourceDefinitions><channel>?'],
 '<sourceDefinitions>': ['<sourceDefinition><sourceDefinitions-1>*'],
 '<sourceDefinitions-1>': ['<COMMA_SYMBOL><sourceDefinition>'],
 '<sourceDefinition>': ['<changeReplicationSourceHost><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<NETWORK_NAMESPACE_SYMBOL><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceBind><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceUser><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourcePassword><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourcePort><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceConnectRetry><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceRetryCount><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceDelay><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceSSL><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceSSLCA><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceSSLCApath><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceTLSVersion><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceSSLCert><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceTLSCiphersuites><EQUAL_OPERATOR><sourceTlsCiphersuitesDef>',
                        '<changeReplicationSourceSSLCipher><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceSSLKey><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceSSLVerifyServerCert><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceSSLCLR><EQUAL_OPERATOR><textLiteral>',
                        '<changeReplicationSourceSSLCLRpath><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourcePublicKey><EQUAL_OPERATOR><textStringNoLinebreak>',
                        '<changeReplicationSourceGetSourcePublicKey><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceHeartbeatPeriod><EQUAL_OPERATOR><ulong_number>',
                        '<IGNORE_SERVER_IDS_SYMBOL><EQUAL_OPERATOR><serverIdList>',
                        '<changeReplicationSourceCompressionAlgorithm><EQUAL_OPERATOR><textStringLiteral>',
                        '<changeReplicationSourceZstdCompressionLevel><EQUAL_OPERATOR><ulong_number>',
                        '<changeReplicationSourceAutoPosition><EQUAL_OPERATOR><ulong_number>',
                        '<PRIVILEGE_CHECKS_USER_SYMBOL><EQUAL_OPERATOR><privilegeCheckDef>',
                        '<REQUIRE_ROW_FORMAT_SYMBOL><EQUAL_OPERATOR><ulong_number>',
                        '<REQUIRE_TABLE_PRIMARY_KEY_CHECK_SYMBOL><EQUAL_OPERATOR><tablePrimaryKeyCheckDef>',
                        '<SOURCE_CONNECTION_AUTO_FAILOVER_SYMBOL><EQUAL_OPERATOR><real_ulong_number>',
                        '<ASSIGN_GTIDS_TO_ANONYMOUS_TRANSACTIONS_SYMBOL><EQUAL_OPERATOR><assignGtidsToAnonymousTransactionsDefinition>',
                        '<GTID_ONLY_SYMBOL><EQUAL_OPERATOR><real_ulong_number>',
                        '<sourceFileDef>'],
 '<changeReplicationSourceAutoPosition>': ['<MASTER_AUTO_POSITION_SYMBOL>',
                                           '<SOURCE_AUTO_POSITION_SYMBOL>'],
 '<changeReplicationSourceHost>': ['<MASTER_HOST_SYMBOL>',
                                   '<SOURCE_HOST_SYMBOL>'],
 '<changeReplicationSourceBind>': ['<MASTER_BIND_SYMBOL>',
                                   '<SOURCE_BIND_SYMBOL>'],
 '<changeReplicationSourceUser>': ['<MASTER_USER_SYMBOL>',
                                   '<SOURCE_USER_SYMBOL>'],
 '<changeReplicationSourcePassword>': ['<MASTER_PASSWORD_SYMBOL>',
                                       '<SOURCE_PASSWORD_SYMBOL>'],
 '<changeReplicationSourcePort>': ['<MASTER_PORT_SYMBOL>',
                                   '<SOURCE_PORT_SYMBOL>'],
 '<changeReplicationSourceConnectRetry>': ['<MASTER_CONNECT_RETRY_SYMBOL>',
                                           '<SOURCE_CONNECT_RETRY_SYMBOL>'],
 '<changeReplicationSourceRetryCount>': ['<MASTER_RETRY_COUNT_SYMBOL>',
                                         '<SOURCE_RETRY_COUNT_SYMBOL>'],
 '<changeReplicationSourceDelay>': ['<MASTER_DELAY_SYMBOL>',
                                    '<SOURCE_DELAY_SYMBOL>'],
 '<changeReplicationSourceSSL>': ['<MASTER_SSL_SYMBOL>', '<SOURCE_SSL_SYMBOL>'],
 '<changeReplicationSourceSSLCA>': ['<MASTER_SSL_CA_SYMBOL>',
                                    '<SOURCE_SSL_CA_SYMBOL>'],
 '<changeReplicationSourceSSLCApath>': ['<MASTER_SSL_CAPATH_SYMBOL>',
                                        '<SOURCE_SSL_CAPATH_SYMBOL>'],
 '<changeReplicationSourceSSLCipher>': ['<MASTER_SSL_CIPHER_SYMBOL>',
                                        '<SOURCE_SSL_CIPHER_SYMBOL>'],
 '<changeReplicationSourceSSLCLR>': ['<MASTER_SSL_CRL_SYMBOL>',
                                     '<SOURCE_SSL_CRL_SYMBOL>'],
 '<changeReplicationSourceSSLCLRpath>': ['<MASTER_SSL_CRLPATH_SYMBOL>',
                                         '<SOURCE_SSL_CRLPATH_SYMBOL>'],
 '<changeReplicationSourceSSLKey>': ['<MASTER_SSL_KEY_SYMBOL>',
                                     '<SOURCE_SSL_KEY_SYMBOL>'],
 '<changeReplicationSourceSSLVerifyServerCert>': ['<MASTER_SSL_VERIFY_SERVER_CERT_SYMBOL>',
                                                  '<SOURCE_SSL_VERIFY_SERVER_CERT_SYMBOL>'],
 '<changeReplicationSourceTLSVersion>': ['<MASTER_TLS_VERSION_SYMBOL>',
                                         '<SOURCE_TLS_VERSION_SYMBOL>'],
 '<changeReplicationSourceTLSCiphersuites>': ['<MASTER_TLS_CIPHERSUITES_SYMBOL>',
                                              '<SOURCE_TLS_CIPHERSUITES_SYMBOL>'],
 '<changeReplicationSourceSSLCert>': ['<MASTER_SSL_CERT_SYMBOL>',
                                      '<SOURCE_SSL_CERT_SYMBOL>'],
 '<changeReplicationSourcePublicKey>': ['<MASTER_PUBLIC_KEY_PATH_SYMBOL>',
                                        '<SOURCE_PUBLIC_KEY_PATH_SYMBOL>'],
 '<changeReplicationSourceGetSourcePublicKey>': ['<GET_MASTER_PUBLIC_KEY_SYMBOL>',
                                                 '<GET_SOURCE_PUBLIC_KEY_SYMBOL>'],
 '<changeReplicationSourceHeartbeatPeriod>': ['<MASTER_HEARTBEAT_PERIOD_SYMBOL>',
                                              '<SOURCE_HEARTBEAT_PERIOD_SYMBOL>'],
 '<changeReplicationSourceCompressionAlgorithm>': ['<MASTER_COMPRESSION_ALGORITHM_SYMBOL>',
                                                   '<SOURCE_COMPRESSION_ALGORITHM_SYMBOL>'],
 '<changeReplicationSourceZstdCompressionLevel>': ['<MASTER_ZSTD_COMPRESSION_LEVEL_SYMBOL>',
                                                   '<SOURCE_ZSTD_COMPRESSION_LEVEL_SYMBOL>'],
 '<privilegeCheckDef>': ['<userIdentifierOrText>', '<NULL_SYMBOL>'],
 '<tablePrimaryKeyCheckDef>': ['<STREAM_SYMBOL>',
                               '<ON_SYMBOL>',
                               '<OFF_SYMBOL>',
                               '<GENERATE_SYMBOL>'],
 '<assignGtidsToAnonymousTransactionsDefinition>': ['<OFF_SYMBOL>',
                                                    '<LOCAL_SYMBOL>',
                                                    '<textStringLiteral>'],
 '<sourceTlsCiphersuitesDef>': ['<textStringNoLinebreak>', '<NULL_SYMBOL>'],
 '<sourceFileDef>': ['<sourceLogFile><EQUAL_OPERATOR><textStringNoLinebreak>',
                     '<sourceLogPos><EQUAL_OPERATOR><ulonglongNumber>',
                     '<RELAY_LOG_FILE_SYMBOL><EQUAL_OPERATOR><textStringNoLinebreak>',
                     '<RELAY_LOG_POS_SYMBOL><EQUAL_OPERATOR><ulong_number>'],
 '<sourceLogFile>': ['<MASTER_LOG_FILE_SYMBOL>', '<SOURCE_LOG_FILE_SYMBOL>'],
 '<sourceLogPos>': ['<MASTER_LOG_POS_SYMBOL>', '<SOURCE_LOG_POS_SYMBOL>'],
 '<serverIdList>': ['<OPEN_PAR_SYMBOL><serverIdList-1>?<CLOSE_PAR_SYMBOL>'],
 '<serverIdList-1>': ['<ulong_number><serverIdList-2>*'],
 '<serverIdList-2>': ['<COMMA_SYMBOL><ulong_number>'],
 '<changeReplication>': ['<CHANGE_SYMBOL><REPLICATION_SYMBOL><FILTER_SYMBOL><filterDefinition><changeReplication-1>*<channel>?'],
 '<changeReplication-1>': ['<COMMA_SYMBOL><filterDefinition>'],
 '<filterDefinition>': ['<REPLICATE_DO_DB_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterDbList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_IGNORE_DB_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterDbList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_DO_TABLE_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterTableList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_IGNORE_TABLE_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterTableList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_WILD_DO_TABLE_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterStringList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_WILD_IGNORE_TABLE_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterStringList>?<CLOSE_PAR_SYMBOL>',
                        '<REPLICATE_REWRITE_DB_SYMBOL><EQUAL_OPERATOR><OPEN_PAR_SYMBOL><filterDbPairList>?<CLOSE_PAR_SYMBOL>'],
 '<filterDbList>': ['<schemaRef><filterDbList-1>*'],
 '<filterDbList-1>': ['<COMMA_SYMBOL><schemaRef>'],
 '<filterTableList>': ['<filterTableRef><filterTableList-1>*'],
 '<filterTableList-1>': ['<COMMA_SYMBOL><filterTableRef>'],
 '<filterStringList>': ['<filterWildDbTableString><filterStringList-1>*'],
 '<filterStringList-1>': ['<COMMA_SYMBOL><filterWildDbTableString>'],
 '<filterWildDbTableString>': ['<textStringNoLinebreak>'],
 '<filterDbPairList>': ['<schemaIdentifierPair><filterDbPairList-1>*'],
 '<filterDbPairList-1>': ['<COMMA_SYMBOL><schemaIdentifierPair>'],
 '<startReplicaStatement>': ['<START_SYMBOL><replica><replicaThreadOptions>?<startReplicaStatement-1>?<userOption>?<passwordOption>?<defaultAuthOption>?<pluginDirOption>?<channel>?'],
 '<startReplicaStatement-1>': ['<UNTIL_SYMBOL><replicaUntil>'],
 '<stopReplicaStatement>': ['<STOP_SYMBOL><replica><replicaThreadOptions>?<channel>?'],
 '<replicaUntil>': ['<replicaUntil-1><replicaUntil-3>*'],
 '<replicaUntil-1>': ['<sourceFileDef>',
                      '<replicaUntil-2><EQUAL_OPERATOR><textString>',
                      '<SQL_AFTER_MTS_GAPS_SYMBOL>'],
 '<replicaUntil-2>': ['<SQL_BEFORE_GTIDS_SYMBOL>', '<SQL_AFTER_GTIDS_SYMBOL>'],
 '<replicaUntil-3>': ['<COMMA_SYMBOL><sourceFileDef>'],
 '<userOption>': ['<USER_SYMBOL><EQUAL_OPERATOR><textString>'],
 '<passwordOption>': ['<PASSWORD_SYMBOL><EQUAL_OPERATOR><textString>'],
 '<defaultAuthOption>': ['<DEFAULT_AUTH_SYMBOL><EQUAL_OPERATOR><textString>'],
 '<pluginDirOption>': ['<PLUGIN_DIR_SYMBOL><EQUAL_OPERATOR><textString>'],
 '<replicaThreadOptions>': ['<replicaThreadOption><replicaThreadOptions-1>*'],
 '<replicaThreadOptions-1>': ['<COMMA_SYMBOL><replicaThreadOption>'],
 '<replicaThreadOption>': ['<SQL_THREAD_SYMBOL>', '<RELAY_THREAD_SYMBOL>'],
 '<groupReplication>': ['<groupReplication-1><GROUP_REPLICATION_SYMBOL>'],
 '<groupReplication-1>': ['<START_SYMBOL><groupReplicationStartOptions>?',
                          '<STOP_SYMBOL>'],
 '<groupReplicationStartOptions>': ['<groupReplicationStartOption><groupReplicationStartOptions-1>*'],
 '<groupReplicationStartOptions-1>': ['<COMMA_SYMBOL><groupReplicationStartOption>'],
 '<groupReplicationStartOption>': ['<groupReplicationUser>',
                                   '<groupReplicationPassword>',
                                   '<groupReplicationPluginAuth>'],
 '<groupReplicationUser>': ['<USER_SYMBOL><EQUAL_OPERATOR><textStringNoLinebreak>'],
 '<groupReplicationPassword>': ['<PASSWORD_SYMBOL><EQUAL_OPERATOR><textStringNoLinebreak>'],
 '<groupReplicationPluginAuth>': ['<DEFAULT_AUTH_SYMBOL><EQUAL_OPERATOR><textStringNoLinebreak>'],
 '<replica>': ['<SLAVE_SYMBOL>', '<REPLICA_SYMBOL>'],
 '<preparedStatement>': ['<PREPARE_SYMBOL><identifier><FROM_SYMBOL><preparedStatement-1>',
                         '<executeStatement>',
                         '<preparedStatement-2><PREPARE_SYMBOL><identifier>'],
 '<preparedStatement-1>': ['<textLiteral>', '<userVariable>'],
 '<preparedStatement-2>': ['<DEALLOCATE_SYMBOL>', '<DROP_SYMBOL>'],
 '<executeStatement>': ['<EXECUTE_SYMBOL><identifier><executeStatement-1>?'],
 '<executeStatement-1>': ['<USING_SYMBOL><executeVarList>'],
 '<executeVarList>': ['<userVariable><executeVarList-1>*'],
 '<executeVarList-1>': ['<COMMA_SYMBOL><userVariable>'],
 '<cloneStatement>': ['<CLONE_SYMBOL><cloneStatement-1>'],
 '<cloneStatement-1>': ['<LOCAL_SYMBOL><DATA_SYMBOL><DIRECTORY_SYMBOL><equal>?<textStringLiteral>',
                        '<REMOTE_SYMBOL><cloneStatement-2>?',
                        '<INSTANCE_SYMBOL><FROM_SYMBOL><user><COLON_SYMBOL><ulong_number><IDENTIFIED_SYMBOL><BY_SYMBOL><textStringLiteral><dataDirSSL>?'],
 '<cloneStatement-2>': ['<FOR_SYMBOL><REPLICATION_SYMBOL>'],
 '<dataDirSSL>': ['<ssl>',
                  '<DATA_SYMBOL><DIRECTORY_SYMBOL><equal>?<textStringLiteral><ssl>?'],
 '<ssl>': ['<REQUIRE_SYMBOL><NO_SYMBOL>?<SSL_SYMBOL>'],
 '<accountManagementStatement>': ['<alterUserStatement>',
                                  '<createUserStatement>',
                                  '<dropUserStatement>',
                                  '<grantStatement>',
                                  '<renameUserStatement>',
                                  '<revokeStatement>',
                                  '<setRoleStatement>'],
 '<alterUserStatement>': ['<ALTER_SYMBOL><USER_SYMBOL><ifExists>?<alterUserStatement-1>'],
 '<alterUserStatement-1>': ['<alterUserStatement-2><createUserTail>',
                            '<userFunction><alterUserStatement-3>',
                            '<user><alterUserStatement-5>'],
 '<alterUserStatement-2>': ['<createUserList>', '<alterUserList>'],
 '<alterUserStatement-3>': ['<alterUserStatement-4><replacePassword>?<retainCurrentPassword>?',
                            '<DISCARD_SYMBOL><OLD_SYMBOL><PASSWORD_SYMBOL>',
                            '<userRegistration>?'],
 '<alterUserStatement-4>': ['<identifiedByRandomPassword>',
                            '<identifiedByPassword>'],
 '<alterUserStatement-5>': ['<DEFAULT_SYMBOL><ROLE_SYMBOL><alterUserStatement-6>',
                            '<userRegistration>?'],
 '<alterUserStatement-6>': ['<ALL_SYMBOL>', '<NONE_SYMBOL>', '<roleList>'],
 '<alterUserList>': ['<alterUser><alterUserList-1>*'],
 '<alterUserList-1>': ['<COMMA_SYMBOL><alterUser>'],
 '<alterUser>': ['<oldAlterUser>', '<alterUser-1>'],
 '<alterUser-1>': ['<user><alterUser-2>'],
 '<alterUser-2>': ['<identifiedByPassword><alterUser-3>',
                   '<identifiedByRandomPassword><alterUser-4>',
                   '<identifiedWithPlugin>',
                   '<identifiedWithPluginAsAuth><retainCurrentPassword>?',
                   '<identifiedWithPluginByPassword><alterUser-5>',
                   '<identifiedWithPluginByRandomPassword><retainCurrentPassword>?',
                   '<discardOldPassword>?',
                   '<ADD_SYMBOL><factor><identification><alterUser-6>?',
                   '<MODIFY_SYMBOL><factor><identification><alterUser-7>?',
                   '<DROP_SYMBOL><factor><alterUser-8>?'],
 '<alterUser-3>': ['<REPLACE_SYMBOL><textStringLiteral><retainCurrentPassword>?',
                   '<retainCurrentPassword>?'],
 '<alterUser-4>': ['<retainCurrentPassword>?',
                   '<REPLACE_SYMBOL><textStringLiteral><retainCurrentPassword>?'],
 '<alterUser-5>': ['<REPLACE_SYMBOL><textStringLiteral><retainCurrentPassword>?',
                   '<retainCurrentPassword>?'],
 '<alterUser-6>': ['<ADD_SYMBOL><factor><identification>'],
 '<alterUser-7>': ['<MODIFY_SYMBOL><factor><identification>'],
 '<alterUser-8>': ['<DROP_SYMBOL><factor>'],
 '<oldAlterUser>': ['<user><IDENTIFIED_SYMBOL><BY_SYMBOL><oldAlterUser-1>',
                    '<user><IDENTIFIED_SYMBOL><WITH_SYMBOL><oldAlterUser-3>',
                    '<user><discardOldPassword>?'],
 '<oldAlterUser-1>': ['<textString><REPLACE_SYMBOL><textString><retainCurrentPassword>?',
                      '<textString><retainCurrentPassword>?',
                      '<RANDOM_SYMBOL><PASSWORD_SYMBOL><oldAlterUser-2>?<retainCurrentPassword>?'],
 '<oldAlterUser-2>': ['<REPLACE_SYMBOL><textString>'],
 '<oldAlterUser-3>': ['<textOrIdentifier><oldAlterUser-4>?'],
 '<oldAlterUser-4>': ['<BY_SYMBOL><textString><REPLACE_SYMBOL><textString><retainCurrentPassword>?',
                      '<AS_SYMBOL><textStringHash><retainCurrentPassword>?',
                      '<BY_SYMBOL><textString><retainCurrentPassword>?',
                      '<BY_SYMBOL><RANDOM_SYMBOL><PASSWORD_SYMBOL><retainCurrentPassword>?'],
 '<userFunction>': ['<USER_SYMBOL><parentheses>'],
 '<createUserStatement>': ['<CREATE_SYMBOL><USER_SYMBOL><ifNotExists>?<createUserList><defaultRoleClause>?<createUserTail>'],
 '<createUserTail>': ['<requireClause>?<connectOptions>?<accountLockPasswordExpireOptions>*<createUserTail-1>?'],
 '<createUserTail-1>': ['<userAttributes>'],
 '<userAttributes>': ['<ATTRIBUTE_SYMBOL><textStringLiteral>',
                      '<COMMENT_SYMBOL><textStringLiteral>'],
 '<defaultRoleClause>': ['<DEFAULT_SYMBOL><ROLE_SYMBOL><roleList>'],
 '<requireClause>': ['<REQUIRE_SYMBOL><requireClause-1>'],
 '<requireClause-1>': ['<requireList>', '<requireClause-2>'],
 '<requireClause-2>': ['<SSL_SYMBOL>', '<X509_SYMBOL>', '<NONE_SYMBOL>'],
 '<connectOptions>': ['<WITH_SYMBOL><connectOptions-1>+'],
 '<connectOptions-1>': ['<MAX_QUERIES_PER_HOUR_SYMBOL><ulong_number>',
                        '<MAX_UPDATES_PER_HOUR_SYMBOL><ulong_number>',
                        '<MAX_CONNECTIONS_PER_HOUR_SYMBOL><ulong_number>',
                        '<MAX_USER_CONNECTIONS_SYMBOL><ulong_number>'],
 '<accountLockPasswordExpireOptions>': ['<ACCOUNT_SYMBOL><accountLockPasswordExpireOptions-1>',
                                        '<PASSWORD_SYMBOL><accountLockPasswordExpireOptions-2>',
                                        '<FAILED_LOGIN_ATTEMPTS_SYMBOL><real_ulong_number>',
                                        '<PASSWORD_LOCK_TIME_SYMBOL><accountLockPasswordExpireOptions-7>'],
 '<accountLockPasswordExpireOptions-1>': ['<LOCK_SYMBOL>', '<UNLOCK_SYMBOL>'],
 '<accountLockPasswordExpireOptions-2>': ['<EXPIRE_SYMBOL><accountLockPasswordExpireOptions-3>?',
                                          '<HISTORY_SYMBOL><accountLockPasswordExpireOptions-4>',
                                          '<REUSE_SYMBOL><INTERVAL_SYMBOL><accountLockPasswordExpireOptions-5>',
                                          '<REQUIRE_SYMBOL><CURRENT_SYMBOL><accountLockPasswordExpireOptions-6>?'],
 '<accountLockPasswordExpireOptions-3>': ['<INTERVAL_SYMBOL><real_ulong_number><DAY_SYMBOL>',
                                          '<NEVER_SYMBOL>',
                                          '<DEFAULT_SYMBOL>'],
 '<accountLockPasswordExpireOptions-4>': ['<real_ulong_number>',
                                          '<DEFAULT_SYMBOL>'],
 '<accountLockPasswordExpireOptions-5>': ['<real_ulong_number><DAY_SYMBOL>',
                                          '<DEFAULT_SYMBOL>'],
 '<accountLockPasswordExpireOptions-6>': ['<DEFAULT_SYMBOL>',
                                          '<OPTIONAL_SYMBOL>'],
 '<accountLockPasswordExpireOptions-7>': ['<real_ulong_number>',
                                          '<UNBOUNDED_SYMBOL>'],
 '<dropUserStatement>': ['<DROP_SYMBOL><USER_SYMBOL><ifExists>?<userList>'],
 '<grantStatement>': ['<GRANT_SYMBOL><grantStatement-1>'],
 '<grantStatement-1>': ['<roleOrPrivilegesList><TO_SYMBOL><userList><grantStatement-2>?',
                        '<grantStatement-3><ON_SYMBOL><aclType>?<grantIdentifier><TO_SYMBOL><grantTargetList><versionedRequireClause>?<grantOptions>?<grantAs>?',
                        '<PROXY_SYMBOL><ON_SYMBOL><user><TO_SYMBOL><grantTargetList><grantStatement-4>?'],
 '<grantStatement-2>': ['<WITH_SYMBOL><ADMIN_SYMBOL><OPTION_SYMBOL>'],
 '<grantStatement-3>': ['<roleOrPrivilegesList>',
                        '<ALL_SYMBOL><PRIVILEGES_SYMBOL>?'],
 '<grantStatement-4>': ['<WITH_SYMBOL><GRANT_SYMBOL><OPTION_SYMBOL>'],
 '<grantTargetList>': ['<createUserList>', '<userList>'],
 '<grantOptions>': ['<WITH_SYMBOL><grantOption>'],
 '<exceptRoleList>': ['<EXCEPT_SYMBOL><roleList>'],
 '<withRoles>': ['<WITH_SYMBOL><ROLE_SYMBOL><withRoles-1>'],
 '<withRoles-1>': ['<roleList>',
                   '<ALL_SYMBOL><exceptRoleList>?',
                   '<NONE_SYMBOL>',
                   '<DEFAULT_SYMBOL>'],
 '<grantAs>': ['<AS_SYMBOL><USER_SYMBOL><withRoles>?'],
 '<versionedRequireClause>': ['<requireClause>'],
 '<renameUserStatement>': ['<RENAME_SYMBOL><USER_SYMBOL><user><TO_SYMBOL><user><renameUserStatement-1>*'],
 '<renameUserStatement-1>': ['<COMMA_SYMBOL><user><TO_SYMBOL><user>'],
 '<revokeStatement>': ['<REVOKE_SYMBOL><revokeStatement-1>?<revokeStatement-2><revokeStatement-4>?'],
 '<revokeStatement-1>': ['<ifExists>'],
 '<revokeStatement-2>': ['<roleOrPrivilegesList><FROM_SYMBOL><userList>',
                         '<roleOrPrivilegesList><ON_SYMBOL><aclType>?<grantIdentifier><FROM_SYMBOL><userList>',
                         '<ALL_SYMBOL><PRIVILEGES_SYMBOL>?<revokeStatement-3><FROM_SYMBOL><userList>',
                         '<PROXY_SYMBOL><ON_SYMBOL><user><FROM_SYMBOL><userList>'],
 '<revokeStatement-3>': ['<ON_SYMBOL><aclType>?<grantIdentifier>',
                         '<COMMA_SYMBOL><GRANT_SYMBOL><OPTION_SYMBOL>'],
 '<revokeStatement-4>': ['<ignoreUnknownUser>'],
 '<aclType>': ['<TABLE_SYMBOL>', '<FUNCTION_SYMBOL>', '<PROCEDURE_SYMBOL>'],
 '<roleOrPrivilegesList>': ['<roleOrPrivilege><roleOrPrivilegesList-1>*'],
 '<roleOrPrivilegesList-1>': ['<COMMA_SYMBOL><roleOrPrivilege>'],
 '<roleOrPrivilege>': ['<roleOrPrivilege-1>',
                       '<roleOrPrivilege-3><columnInternalRefList>?',
                       '<roleOrPrivilege-4>',
                       '<GRANT_SYMBOL><OPTION_SYMBOL>',
                       '<SHOW_SYMBOL><DATABASES_SYMBOL>',
                       '<CREATE_SYMBOL><roleOrPrivilege-5>?',
                       '<LOCK_SYMBOL><TABLES_SYMBOL>',
                       '<REPLICATION_SYMBOL><roleOrPrivilege-7>',
                       '<SHOW_SYMBOL><VIEW_SYMBOL>',
                       '<ALTER_SYMBOL><ROUTINE_SYMBOL>?',
                       '<roleOrPrivilege-8><ROLE_SYMBOL>'],
 '<roleOrPrivilege-1>': ['<roleIdentifierOrText><columnInternalRefList>?',
                         '<roleIdentifierOrText><roleOrPrivilege-2>'],
 '<roleOrPrivilege-2>': ['<AT_TEXT_SUFFIX>',
                         '<AT_SIGN_SYMBOL><textOrIdentifier>'],
 '<roleOrPrivilege-3>': ['<SELECT_SYMBOL>',
                         '<INSERT_SYMBOL>',
                         '<UPDATE_SYMBOL>',
                         '<REFERENCES_SYMBOL>'],
 '<roleOrPrivilege-4>': ['<DELETE_SYMBOL>',
                         '<USAGE_SYMBOL>',
                         '<INDEX_SYMBOL>',
                         '<DROP_SYMBOL>',
                         '<EXECUTE_SYMBOL>',
                         '<RELOAD_SYMBOL>',
                         '<SHUTDOWN_SYMBOL>',
                         '<PROCESS_SYMBOL>',
                         '<FILE_SYMBOL>',
                         '<PROXY_SYMBOL>',
                         '<SUPER_SYMBOL>',
                         '<EVENT_SYMBOL>',
                         '<TRIGGER_SYMBOL>'],
 '<roleOrPrivilege-5>': ['<TEMPORARY_SYMBOL><TABLES_SYMBOL>',
                         '<roleOrPrivilege-6>'],
 '<roleOrPrivilege-6>': ['<ROUTINE_SYMBOL>',
                         '<TABLESPACE_SYMBOL>',
                         '<USER_SYMBOL>',
                         '<VIEW_SYMBOL>'],
 '<roleOrPrivilege-7>': ['<CLIENT_SYMBOL>', '<replica>'],
 '<roleOrPrivilege-8>': ['<CREATE_SYMBOL>', '<DROP_SYMBOL>'],
 '<grantIdentifier>': ['<MULT_OPERATOR><grantIdentifier-1>?',
                       '<schemaRef><grantIdentifier-2>?',
                       '<tableRef>',
                       '<schemaRef><DOT_SYMBOL><tableRef>'],
 '<grantIdentifier-1>': ['<DOT_SYMBOL><MULT_OPERATOR>'],
 '<grantIdentifier-2>': ['<DOT_SYMBOL><MULT_OPERATOR>'],
 '<requireList>': ['<requireListElement><requireList-1>*'],
 '<requireList-1>': ['<AND_SYMBOL>?<requireListElement>'],
 '<requireListElement>': ['<CIPHER_SYMBOL><textString>',
                          '<ISSUER_SYMBOL><textString>',
                          '<SUBJECT_SYMBOL><textString>'],
 '<grantOption>': ['<GRANT_SYMBOL><OPTION_SYMBOL>', '<grantOption-1>'],
 '<grantOption-1>': ['<MAX_QUERIES_PER_HOUR_SYMBOL><ulong_number>',
                     '<MAX_UPDATES_PER_HOUR_SYMBOL><ulong_number>',
                     '<MAX_CONNECTIONS_PER_HOUR_SYMBOL><ulong_number>',
                     '<MAX_USER_CONNECTIONS_SYMBOL><ulong_number>'],
 '<setRoleStatement>': ['<SET_SYMBOL><ROLE_SYMBOL><roleList>',
                        '<SET_SYMBOL><ROLE_SYMBOL><setRoleStatement-1>',
                        '<SET_SYMBOL><DEFAULT_SYMBOL><ROLE_SYMBOL><setRoleStatement-2><TO_SYMBOL><roleList>',
                        '<SET_SYMBOL><ROLE_SYMBOL><ALL_SYMBOL><setRoleStatement-3>?'],
 '<setRoleStatement-1>': ['<NONE_SYMBOL>', '<DEFAULT_SYMBOL>'],
 '<setRoleStatement-2>': ['<roleList>', '<NONE_SYMBOL>', '<ALL_SYMBOL>'],
 '<setRoleStatement-3>': ['<EXCEPT_SYMBOL><roleList>'],
 '<roleList>': ['<role><roleList-1>*'],
 '<roleList-1>': ['<COMMA_SYMBOL><role>'],
 '<role>': ['<roleIdentifierOrText><userVariable>?'],
 '<tableAdministrationStatement>': ['<ANALYZE_SYMBOL><noWriteToBinLog>?<TABLE_SYMBOL><tableRefList><histogram>?',
                                    '<CHECK_SYMBOL><TABLE_SYMBOL><tableRefList><checkOption>*',
                                    '<CHECKSUM_SYMBOL><TABLE_SYMBOL><tableRefList><tableAdministrationStatement-1>?',
                                    '<OPTIMIZE_SYMBOL><noWriteToBinLog>?<TABLE_SYMBOL><tableRefList>',
                                    '<REPAIR_SYMBOL><noWriteToBinLog>?<TABLE_SYMBOL><tableRefList><repairType>*'],
 '<tableAdministrationStatement-1>': ['<QUICK_SYMBOL>', '<EXTENDED_SYMBOL>'],
 '<histogramAutoUpdate>': ['<histogramAutoUpdate-1><UPDATE_SYMBOL>'],
 '<histogramAutoUpdate-1>': ['<MANUAL_SYMBOL>', '<AUTO_SYMBOL>'],
 '<histogramUpdateParam>': ['<histogramNumBuckets>?<histogramAutoUpdate>?',
                            '<USING_SYMBOL><DATA_SYMBOL><textStringLiteral>'],
 '<histogramNumBuckets>': ['<WITH_SYMBOL><INT_NUMBER><BUCKETS_SYMBOL>'],
 '<histogram>': ['<UPDATE_SYMBOL><HISTOGRAM_SYMBOL><ON_SYMBOL><identifierList><histogramUpdateParam>',
                 '<DROP_SYMBOL><HISTOGRAM_SYMBOL><ON_SYMBOL><identifierList>'],
 '<checkOption>': ['<FOR_SYMBOL><UPGRADE_SYMBOL>', '<checkOption-1>'],
 '<checkOption-1>': ['<QUICK_SYMBOL>',
                     '<FAST_SYMBOL>',
                     '<MEDIUM_SYMBOL>',
                     '<EXTENDED_SYMBOL>',
                     '<CHANGED_SYMBOL>'],
 '<repairType>': ['<QUICK_SYMBOL>', '<EXTENDED_SYMBOL>', '<USE_FRM_SYMBOL>'],
 '<uninstallStatement>': ['<UNINSTALL_SYMBOL><uninstallStatement-1>'],
 '<uninstallStatement-1>': ['<PLUGIN_SYMBOL><pluginRef>',
                            '<COMPONENT_SYMBOL><componentRef><uninstallStatement-2>*'],
 '<uninstallStatement-2>': ['<COMMA_SYMBOL><componentRef>'],
 '<installStatement>': ['<INSTALL_SYMBOL><installStatement-1>'],
 '<installStatement-1>': ['<PLUGIN_SYMBOL><identifier><SONAME_SYMBOL><textStringLiteral>',
                          '<COMPONENT_SYMBOL><textStringLiteralList><installSetValueList>?'],
 '<installOptionType>': ['<GLOBAL_SYMBOL>', '<PERSIST_SYMBOL>'],
 '<installSetRvalue>': ['<expr>', '<ON_SYMBOL>'],
 '<installSetValue>': ['<installOptionType><lvalueVariable><equal><installSetRvalue>'],
 '<installSetValueList>': ['<SET_SYMBOL><installSetValue><installSetValueList-1>*'],
 '<installSetValueList-1>': ['<COMMA_SYMBOL><installSetValue>'],
 '<setStatement>': ['<SET_SYMBOL><startOptionValueList>'],
 '<startOptionValueList>': ['<optionValueNoOptionType><optionValueListContinued>',
                            '<TRANSACTION_SYMBOL><transactionCharacteristics>',
                            '<optionType><startOptionValueListFollowingOptionType>',
                            '<PASSWORD_SYMBOL><startOptionValueList-1>?<equal><startOptionValueList-2>',
                            '<PASSWORD_SYMBOL><startOptionValueList-3>?<TO_SYMBOL><RANDOM_SYMBOL><replacePassword>?<retainCurrentPassword>?'],
 '<startOptionValueList-1>': ['<FOR_SYMBOL><user>'],
 '<startOptionValueList-2>': ['<textString><replacePassword>?<retainCurrentPassword>?',
                              '<textString><replacePassword>?<retainCurrentPassword>?',
                              '<PASSWORD_SYMBOL><OPEN_PAR_SYMBOL><textString><CLOSE_PAR_SYMBOL>'],
 '<startOptionValueList-3>': ['<FOR_SYMBOL><user>'],
 '<transactionCharacteristics>': ['<transactionAccessMode><isolationLevel>?',
                                  '<isolationLevel><transactionCharacteristics-1>?'],
 '<transactionCharacteristics-1>': ['<COMMA_SYMBOL><transactionAccessMode>'],
 '<transactionAccessMode>': ['<READ_SYMBOL><transactionAccessMode-1>'],
 '<transactionAccessMode-1>': ['<WRITE_SYMBOL>', '<ONLY_SYMBOL>'],
 '<isolationLevel>': ['<ISOLATION_SYMBOL><LEVEL_SYMBOL><isolationLevel-1>'],
 '<isolationLevel-1>': ['<REPEATABLE_SYMBOL><READ_SYMBOL>',
                        '<READ_SYMBOL><isolationLevel-2>',
                        '<SERIALIZABLE_SYMBOL>'],
 '<isolationLevel-2>': ['<COMMITTED_SYMBOL>', '<UNCOMMITTED_SYMBOL>'],
 '<optionValueListContinued>': ['<optionValueListContinued-1>*'],
 '<optionValueListContinued-1>': ['<COMMA_SYMBOL><optionValue>'],
 '<optionValueNoOptionType>': ['<lvalueVariable><equal><setExprOrDefault>',
                               '<charsetClause>',
                               '<userVariable><equal><expr>',
                               '<AT_AT_SIGN_SYMBOL><setVarIdentType>?<lvalueVariable><equal><setExprOrDefault>',
                               '<NAMES_SYMBOL><optionValueNoOptionType-1>'],
 '<optionValueNoOptionType-1>': ['<equal><expr>',
                                 '<charsetName><collate>?',
                                 '<DEFAULT_SYMBOL>'],
 '<optionValue>': ['<optionType><lvalueVariable><equal><setExprOrDefault>',
                   '<optionValueNoOptionType>'],
 '<startOptionValueListFollowingOptionType>': ['<optionValueFollowingOptionType><optionValueListContinued>',
                                               '<TRANSACTION_SYMBOL><transactionCharacteristics>'],
 '<optionValueFollowingOptionType>': ['<lvalueVariable><equal><setExprOrDefault>'],
 '<setExprOrDefault>': ['<expr>',
                        '<DEFAULT_SYMBOL>',
                        '<ON_SYMBOL>',
                        '<ALL_SYMBOL>',
                        '<BINARY_SYMBOL>',
                        '<ROW_SYMBOL>',
                        '<SYSTEM_SYMBOL>'],
 '<showDatabasesStatement>': ['<SHOW_SYMBOL><DATABASES_SYMBOL><likeOrWhere>?'],
 '<showTablesStatement>': ['<SHOW_SYMBOL><showCommandType>?<TABLES_SYMBOL><inDb>?<likeOrWhere>?'],
 '<showTriggersStatement>': ['<SHOW_SYMBOL><FULL_SYMBOL>?<TRIGGERS_SYMBOL><inDb>?<likeOrWhere>?'],
 '<showEventsStatement>': ['<SHOW_SYMBOL><EVENTS_SYMBOL><inDb>?<likeOrWhere>?'],
 '<showTableStatusStatement>': ['<SHOW_SYMBOL><TABLE_SYMBOL><STATUS_SYMBOL><inDb>?<likeOrWhere>?'],
 '<showOpenTablesStatement>': ['<SHOW_SYMBOL><OPEN_SYMBOL><TABLES_SYMBOL><inDb>?<likeOrWhere>?'],
 '<showParseTreeStatement>': ['<SHOW_SYMBOL><PARSE_TREE_SYMBOL><simpleStatement>'],
 '<showPluginsStatement>': ['<SHOW_SYMBOL><PLUGINS_SYMBOL>'],
 '<showEngineLogsStatement>': ['<SHOW_SYMBOL><ENGINE_SYMBOL><engineOrAll><LOGS_SYMBOL>'],
 '<showEngineMutexStatement>': ['<SHOW_SYMBOL><ENGINE_SYMBOL><engineOrAll><MUTEX_SYMBOL>'],
 '<showEngineStatusStatement>': ['<SHOW_SYMBOL><ENGINE_SYMBOL><engineOrAll><STATUS_SYMBOL>'],
 '<showColumnsStatement>': ['<SHOW_SYMBOL><showCommandType>?<COLUMNS_SYMBOL><showColumnsStatement-1><tableRef><inDb>?<likeOrWhere>?'],
 '<showColumnsStatement-1>': ['<FROM_SYMBOL>', '<IN_SYMBOL>'],
 '<showBinaryLogsStatement>': ['<SHOW_SYMBOL><showBinaryLogsStatement-1><LOGS_SYMBOL>'],
 '<showBinaryLogsStatement-1>': ['<BINARY_SYMBOL>', '<MASTER_SYMBOL>'],
 '<showBinaryLogStatusStatement>': ['<SHOW_SYMBOL><BINARY_SYMBOL><LOG_SYMBOL><STATUS_SYMBOL>'],
 '<showReplicasStatement>': ['<SHOW_SYMBOL><showReplicasStatement-1>'],
 '<showReplicasStatement-1>': ['<replica><HOSTS_SYMBOL>', '<REPLICAS_SYMBOL>'],
 '<showBinlogEventsStatement>': ['<SHOW_SYMBOL><BINLOG_SYMBOL><EVENTS_SYMBOL><showBinlogEventsStatement-1>?<showBinlogEventsStatement-2>?<limitClause>?<channel>?'],
 '<showBinlogEventsStatement-1>': ['<IN_SYMBOL><textString>'],
 '<showBinlogEventsStatement-2>': ['<FROM_SYMBOL><ulonglongNumber>'],
 '<showRelaylogEventsStatement>': ['<SHOW_SYMBOL><RELAYLOG_SYMBOL><EVENTS_SYMBOL><showRelaylogEventsStatement-1>?<showRelaylogEventsStatement-2>?<limitClause>?<channel>?'],
 '<showRelaylogEventsStatement-1>': ['<IN_SYMBOL><textString>'],
 '<showRelaylogEventsStatement-2>': ['<FROM_SYMBOL><ulonglongNumber>'],
 '<showKeysStatement>': ['<SHOW_SYMBOL><EXTENDED_SYMBOL>?<showKeysStatement-1><fromOrIn><tableRef><inDb>?<whereClause>?'],
 '<showKeysStatement-1>': ['<INDEX_SYMBOL>',
                           '<INDEXES_SYMBOL>',
                           '<KEYS_SYMBOL>'],
 '<showEnginesStatement>': ['<SHOW_SYMBOL><STORAGE_SYMBOL>?<ENGINES_SYMBOL>'],
 '<showCountWarningsStatement>': ['<SHOW_SYMBOL><COUNT_SYMBOL><OPEN_PAR_SYMBOL><MULT_OPERATOR><CLOSE_PAR_SYMBOL><WARNINGS_SYMBOL>'],
 '<showCountErrorsStatement>': ['<SHOW_SYMBOL><COUNT_SYMBOL><OPEN_PAR_SYMBOL><MULT_OPERATOR><CLOSE_PAR_SYMBOL><ERRORS_SYMBOL>'],
 '<showWarningsStatement>': ['<SHOW_SYMBOL><WARNINGS_SYMBOL><limitClause>?'],
 '<showErrorsStatement>': ['<SHOW_SYMBOL><ERRORS_SYMBOL><limitClause>?'],
 '<showProfilesStatement>': ['<SHOW_SYMBOL><PROFILES_SYMBOL>'],
 '<showProfileStatement>': ['<SHOW_SYMBOL><PROFILE_SYMBOL><profileDefinitions>?<showProfileStatement-1>?<limitClause>?'],
 '<showProfileStatement-1>': ['<FOR_SYMBOL><QUERY_SYMBOL><INT_NUMBER>'],
 '<showStatusStatement>': ['<SHOW_SYMBOL><optionType>?<STATUS_SYMBOL><likeOrWhere>?'],
 '<showProcessListStatement>': ['<SHOW_SYMBOL><FULL_SYMBOL>?<PROCESSLIST_SYMBOL>'],
 '<showVariablesStatement>': ['<SHOW_SYMBOL><optionType>?<VARIABLES_SYMBOL><likeOrWhere>?'],
 '<showCharacterSetStatement>': ['<SHOW_SYMBOL><charset><likeOrWhere>?'],
 '<showCollationStatement>': ['<SHOW_SYMBOL><COLLATION_SYMBOL><likeOrWhere>?'],
 '<showPrivilegesStatement>': ['<SHOW_SYMBOL><PRIVILEGES_SYMBOL>'],
 '<showGrantsStatement>': ['<SHOW_SYMBOL><GRANTS_SYMBOL><showGrantsStatement-1>?'],
 '<showGrantsStatement-1>': ['<FOR_SYMBOL><user><showGrantsStatement-2>?'],
 '<showGrantsStatement-2>': ['<USING_SYMBOL><userList>'],
 '<showCreateDatabaseStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><DATABASE_SYMBOL><ifNotExists>?<schemaRef>'],
 '<showCreateTableStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><TABLE_SYMBOL><tableRef>'],
 '<showCreateViewStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><VIEW_SYMBOL><viewRef>'],
 '<showMasterStatusStatement>': ['<SHOW_SYMBOL><MASTER_SYMBOL><STATUS_SYMBOL>'],
 '<showReplicaStatusStatement>': ['<SHOW_SYMBOL><replica><STATUS_SYMBOL><channel>?'],
 '<showCreateProcedureStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><PROCEDURE_SYMBOL><procedureRef>'],
 '<showCreateFunctionStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><FUNCTION_SYMBOL><functionRef>'],
 '<showCreateTriggerStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><TRIGGER_SYMBOL><triggerRef>'],
 '<showCreateProcedureStatusStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><PROCEDURE_SYMBOL><STATUS_SYMBOL><likeOrWhere>?'],
 '<showCreateFunctionStatusStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><FUNCTION_SYMBOL><STATUS_SYMBOL><likeOrWhere>?'],
 '<showCreateProcedureCodeStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><PROCEDURE_SYMBOL><CODE_SYMBOL><procedureRef>'],
 '<showCreateFunctionCodeStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><FUNCTION_SYMBOL><CODE_SYMBOL><functionRef>'],
 '<showCreateEventStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><EVENT_SYMBOL><eventRef>'],
 '<showCreateUserStatement>': ['<SHOW_SYMBOL><CREATE_SYMBOL><USER_SYMBOL><user>'],
 '<showCommandType>': ['<FULL_SYMBOL>', '<EXTENDED_SYMBOL><FULL_SYMBOL>?'],
 '<engineOrAll>': ['<engineRef>', '<ALL_SYMBOL>'],
 '<fromOrIn>': ['<FROM_SYMBOL>', '<IN_SYMBOL>'],
 '<inDb>': ['<fromOrIn><identifier>'],
 '<profileDefinitions>': ['<profileDefinition><profileDefinitions-1>*'],
 '<profileDefinitions-1>': ['<COMMA_SYMBOL><profileDefinition>'],
 '<profileDefinition>': ['<BLOCK_SYMBOL><IO_SYMBOL>',
                         '<CONTEXT_SYMBOL><SWITCHES_SYMBOL>',
                         '<PAGE_SYMBOL><FAULTS_SYMBOL>',
                         '<profileDefinition-1>'],
 '<profileDefinition-1>': ['<ALL_SYMBOL>',
                           '<CPU_SYMBOL>',
                           '<IPC_SYMBOL>',
                           '<MEMORY_SYMBOL>',
                           '<SOURCE_SYMBOL>',
                           '<SWAPS_SYMBOL>'],
 '<otherAdministrativeStatement>': ['<BINLOG_SYMBOL><textLiteral>',
                                    '<CACHE_SYMBOL><INDEX_SYMBOL><keyCacheListOrParts><IN_SYMBOL><otherAdministrativeStatement-1>',
                                    '<FLUSH_SYMBOL><noWriteToBinLog>?<otherAdministrativeStatement-2>',
                                    '<KILL_SYMBOL><otherAdministrativeStatement-4>?<expr>',
                                    '<LOAD_SYMBOL><INDEX_SYMBOL><INTO_SYMBOL><CACHE_SYMBOL><preloadTail>',
                                    '<SHUTDOWN_SYMBOL>'],
 '<otherAdministrativeStatement-1>': ['<identifier>', '<DEFAULT_SYMBOL>'],
 '<otherAdministrativeStatement-2>': ['<flushTables>',
                                      '<flushOption><otherAdministrativeStatement-3>*'],
 '<otherAdministrativeStatement-3>': ['<COMMA_SYMBOL><flushOption>'],
 '<otherAdministrativeStatement-4>': ['<CONNECTION_SYMBOL>', '<QUERY_SYMBOL>'],
 '<keyCacheListOrParts>': ['<keyCacheList>', '<assignToKeycachePartition>'],
 '<keyCacheList>': ['<assignToKeycache><keyCacheList-1>*'],
 '<keyCacheList-1>': ['<COMMA_SYMBOL><assignToKeycache>'],
 '<assignToKeycache>': ['<tableRef><cacheKeyList>?'],
 '<assignToKeycachePartition>': ['<tableRef><PARTITION_SYMBOL><OPEN_PAR_SYMBOL><allOrPartitionNameList><CLOSE_PAR_SYMBOL><cacheKeyList>?'],
 '<cacheKeyList>': ['<keyOrIndex><OPEN_PAR_SYMBOL><keyUsageList>?<CLOSE_PAR_SYMBOL>'],
 '<keyUsageElement>': ['<identifier>', '<PRIMARY_SYMBOL>'],
 '<keyUsageList>': ['<keyUsageElement><keyUsageList-1>*'],
 '<keyUsageList-1>': ['<COMMA_SYMBOL><keyUsageElement>'],
 '<flushOption>': ['<flushOption-1>',
                   '<logType>?<LOGS_SYMBOL>',
                   '<RELAY_SYMBOL><LOGS_SYMBOL><channel>?',
                   '<OPTIMIZER_COSTS_SYMBOL>'],
 '<flushOption-1>': ['<HOSTS_SYMBOL>',
                     '<PRIVILEGES_SYMBOL>',
                     '<STATUS_SYMBOL>',
                     '<USER_RESOURCES_SYMBOL>'],
 '<logType>': ['<BINARY_SYMBOL>',
               '<ENGINE_SYMBOL>',
               '<ERROR_SYMBOL>',
               '<GENERAL_SYMBOL>',
               '<SLOW_SYMBOL>'],
 '<flushTables>': ['<flushTables-1><flushTables-2>?'],
 '<flushTables-1>': ['<TABLES_SYMBOL>', '<TABLE_SYMBOL>'],
 '<flushTables-2>': ['<WITH_SYMBOL><READ_SYMBOL><LOCK_SYMBOL>',
                     '<identifierList><flushTablesOptions>?'],
 '<flushTablesOptions>': ['<FOR_SYMBOL><EXPORT_SYMBOL>',
                          '<WITH_SYMBOL><READ_SYMBOL><LOCK_SYMBOL>'],
 '<preloadTail>': ['<tableRef><adminPartition><cacheKeyList>?<preloadTail-1>?',
                   '<preloadList>'],
 '<preloadTail-1>': ['<IGNORE_SYMBOL><LEAVES_SYMBOL>'],
 '<preloadList>': ['<preloadKeys><preloadList-1>*'],
 '<preloadList-1>': ['<COMMA_SYMBOL><preloadKeys>'],
 '<preloadKeys>': ['<tableRef><cacheKeyList>?<preloadKeys-1>?'],
 '<preloadKeys-1>': ['<IGNORE_SYMBOL><LEAVES_SYMBOL>'],
 '<adminPartition>': ['<PARTITION_SYMBOL><OPEN_PAR_SYMBOL><allOrPartitionNameList><CLOSE_PAR_SYMBOL>'],
 '<resourceGroupManagement>': ['<createResourceGroup>',
                               '<alterResourceGroup>',
                               '<setResourceGroup>',
                               '<dropResourceGroup>'],
 '<createResourceGroup>': ['<CREATE_SYMBOL><RESOURCE_SYMBOL><GROUP_SYMBOL><identifier><TYPE_SYMBOL><equal>?<createResourceGroup-1><resourceGroupVcpuList>?<resourceGroupPriority>?<resourceGroupEnableDisable>?'],
 '<createResourceGroup-1>': ['<USER_SYMBOL>', '<SYSTEM_SYMBOL>'],
 '<resourceGroupVcpuList>': ['<VCPU_SYMBOL><equal>?<vcpuNumOrRange><resourceGroupVcpuList-1>*'],
 '<resourceGroupVcpuList-1>': ['<COMMA_SYMBOL>?<vcpuNumOrRange>'],
 '<vcpuNumOrRange>': ['<INT_NUMBER><vcpuNumOrRange-1>?'],
 '<vcpuNumOrRange-1>': ['<MINUS_OPERATOR><INT_NUMBER>'],
 '<resourceGroupPriority>': ['<THREAD_PRIORITY_SYMBOL><equal>?<INT_NUMBER>'],
 '<resourceGroupEnableDisable>': ['<ENABLE_SYMBOL>', '<DISABLE_SYMBOL>'],
 '<alterResourceGroup>': ['<ALTER_SYMBOL><RESOURCE_SYMBOL><GROUP_SYMBOL><resourceGroupRef><resourceGroupVcpuList>?<resourceGroupPriority>?<resourceGroupEnableDisable>?<FORCE_SYMBOL>?'],
 '<setResourceGroup>': ['<SET_SYMBOL><RESOURCE_SYMBOL><GROUP_SYMBOL><identifier><setResourceGroup-1>?'],
 '<setResourceGroup-1>': ['<FOR_SYMBOL><threadIdList>'],
 '<threadIdList>': ['<real_ulong_number><threadIdList-1>*'],
 '<threadIdList-1>': ['<COMMA_SYMBOL>?<real_ulong_number>'],
 '<dropResourceGroup>': ['<DROP_SYMBOL><RESOURCE_SYMBOL><GROUP_SYMBOL><resourceGroupRef><FORCE_SYMBOL>?'],
 '<utilityStatement>': ['<describeStatement>',
                        '<explainStatement>',
                        '<helpCommand>',
                        '<useCommand>',
                        '<restartServer>'],
 '<describeStatement>': ['<describeStatement-1><tableRef><describeStatement-2>?'],
 '<describeStatement-1>': ['<EXPLAIN_SYMBOL>',
                           '<DESCRIBE_SYMBOL>',
                           '<DESC_SYMBOL>'],
 '<describeStatement-2>': ['<textString>', '<columnRef>'],
 '<explainStatement>': ['<explainStatement-1><explainOptions>?<explainStatement-2>?<explainableStatement>'],
 '<explainStatement-1>': ['<EXPLAIN_SYMBOL>',
                          '<DESCRIBE_SYMBOL>',
                          '<DESC_SYMBOL>'],
 '<explainStatement-2>': ['<FOR_SYMBOL><DATABASE_SYMBOL><textOrIdentifier>'],
 '<explainOptions>': ['<FORMAT_SYMBOL><EQUAL_OPERATOR><textOrIdentifier><explainOptions-1>?',
                      '<EXTENDED_SYMBOL>',
                      '<ANALYZE_SYMBOL>',
                      '<ANALYZE_SYMBOL><FORMAT_SYMBOL><EQUAL_OPERATOR><textOrIdentifier>'],
 '<explainOptions-1>': ['<explainInto>'],
 '<explainableStatement>': ['<selectStatement>',
                            '<deleteStatement>',
                            '<insertStatement>',
                            '<replaceStatement>',
                            '<updateStatement>',
                            '<FOR_SYMBOL><CONNECTION_SYMBOL><real_ulong_number>'],
 '<explainInto>': ['<INTO_SYMBOL><AT_SIGN_SYMBOL><textOrIdentifier>'],
 '<helpCommand>': ['<HELP_SYMBOL><textOrIdentifier>'],
 '<useCommand>': ['<USE_SYMBOL><schemaRef>'],
 '<restartServer>': ['<RESTART_SYMBOL>'],
 '<expr>': ['<boolPri><expr-1>?',
            '<NOT_SYMBOL><expr>',
            '<expr><expr-3><expr>',
            '<expr><XOR_SYMBOL><expr>',
            '<expr><expr-4><expr>'],
 '<expr-1>': ['<IS_SYMBOL><notRule>?<expr-2>'],
 '<expr-2>': ['<TRUE_SYMBOL>', '<FALSE_SYMBOL>', '<UNKNOWN_SYMBOL>'],
 '<expr-3>': ['<AND_SYMBOL>', '<LOGICAL_AND_OPERATOR>'],
 '<expr-4>': ['<OR_SYMBOL>', '<LOGICAL_OR_OPERATOR>'],
 '<boolPri>': ['<predicate>',
               '<boolPri><IS_SYMBOL><notRule>?<NULL_SYMBOL>',
               '<boolPri><compOp><predicate>',
               '<boolPri><compOp><boolPri-1><subquery>'],
 '<boolPri-1>': ['<ALL_SYMBOL>', '<ANY_SYMBOL>'],
 '<compOp>': ['<EQUAL_OPERATOR>',
              '<NULL_SAFE_EQUAL_OPERATOR>',
              '<GREATER_OR_EQUAL_OPERATOR>',
              '<GREATER_THAN_OPERATOR>',
              '<LESS_OR_EQUAL_OPERATOR>',
              '<LESS_THAN_OPERATOR>',
              '<NOT_EQUAL_OPERATOR>'],
 '<predicate>': ['<bitExpr><predicate-1>?'],
 '<predicate-1>': ['<notRule>?<predicateOperations>',
                   '<MEMBER_SYMBOL><OF_SYMBOL>?<simpleExprWithParentheses>',
                   '<SOUNDS_SYMBOL><LIKE_SYMBOL><bitExpr>'],
 '<predicateOperations>': ['<IN_SYMBOL><predicateOperations-1>',
                           '<BETWEEN_SYMBOL><bitExpr><AND_SYMBOL><predicate>',
                           '<LIKE_SYMBOL><simpleExpr><predicateOperations-2>?',
                           '<REGEXP_SYMBOL><bitExpr>'],
 '<predicateOperations-1>': ['<subquery>',
                             '<OPEN_PAR_SYMBOL><exprList><CLOSE_PAR_SYMBOL>'],
 '<predicateOperations-2>': ['<ESCAPE_SYMBOL><simpleExpr>'],
 '<bitExpr>': ['<simpleExpr>',
               '<bitExpr><BITWISE_XOR_OPERATOR><bitExpr>',
               '<bitExpr><bitExpr-1><bitExpr>',
               '<bitExpr><bitExpr-2><bitExpr>',
               '<bitExpr><bitExpr-3><INTERVAL_SYMBOL><expr><interval>',
               '<bitExpr><bitExpr-4><bitExpr>',
               '<bitExpr><BITWISE_AND_OPERATOR><bitExpr>',
               '<bitExpr><BITWISE_OR_OPERATOR><bitExpr>'],
 '<bitExpr-1>': ['<MULT_OPERATOR>',
                 '<DIV_OPERATOR>',
                 '<MOD_OPERATOR>',
                 '<DIV_SYMBOL>',
                 '<MOD_SYMBOL>'],
 '<bitExpr-2>': ['<PLUS_OPERATOR>', '<MINUS_OPERATOR>'],
 '<bitExpr-3>': ['<PLUS_OPERATOR>', '<MINUS_OPERATOR>'],
 '<bitExpr-4>': ['<SHIFT_LEFT_OPERATOR>', '<SHIFT_RIGHT_OPERATOR>'],
 '<simpleExpr>': ['<columnRef><jsonOperator>?',
                  '<runtimeFunctionCall>',
                  '<functionCall>',
                  '<simpleExpr><COLLATE_SYMBOL><textOrIdentifier>',
                  '<literalOrNull>',
                  '<PARAM_MARKER>',
                  '<rvalueSystemOrUserVariable>',
                  '<inExpressionUserVariableAssignment>',
                  '<sumExpr>',
                  '<groupingOperation>',
                  '<windowFunctionCall>',
                  '<simpleExpr>CONCAT_PIPES_SYMBOL<simpleExpr>',
                  '<simpleExpr-1><simpleExpr>',
                  '<not2Rule><simpleExpr>',
                  '<ROW_SYMBOL>?<OPEN_PAR_SYMBOL><exprList><CLOSE_PAR_SYMBOL>',
                  '<EXISTS_SYMBOL>?<subquery>',
                  '<OPEN_CURLY_SYMBOL><identifier><expr><CLOSE_CURLY_SYMBOL>',
                  '<MATCH_SYMBOL><identListArg><AGAINST_SYMBOL><OPEN_PAR_SYMBOL><bitExpr><fulltextOptions>?<CLOSE_PAR_SYMBOL>',
                  '<BINARY_SYMBOL><simpleExpr>',
                  '<CAST_SYMBOL><OPEN_PAR_SYMBOL><expr><simpleExpr-2>?<AS_SYMBOL><castType><arrayCast>?<CLOSE_PAR_SYMBOL>',
                  '<CAST_SYMBOL><OPEN_PAR_SYMBOL><expr><AT_SYMBOL><TIME_SYMBOL><ZONE_SYMBOL><INTERVAL_SYMBOL>?<textStringLiteral><AS_SYMBOL><DATETIME_SYMBOL><typeDatetimePrecision><CLOSE_PAR_SYMBOL>',
                  '<CASE_SYMBOL><expr>?<simpleExpr-3>+<elseExpression>?<END_SYMBOL>',
                  '<CONVERT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><castType><CLOSE_PAR_SYMBOL>',
                  '<CONVERT_SYMBOL><OPEN_PAR_SYMBOL><expr><USING_SYMBOL><charsetName><CLOSE_PAR_SYMBOL>',
                  '<DEFAULT_SYMBOL><OPEN_PAR_SYMBOL><simpleIdentifier><CLOSE_PAR_SYMBOL>',
                  '<VALUES_SYMBOL><OPEN_PAR_SYMBOL><simpleIdentifier><CLOSE_PAR_SYMBOL>',
                  '<INTERVAL_SYMBOL><expr><interval><PLUS_OPERATOR><expr>'],
 '<simpleExpr-1>': ['<PLUS_OPERATOR>',
                    '<MINUS_OPERATOR>',
                    '<BITWISE_NOT_OPERATOR>'],
 '<simpleExpr-2>': ['<AT_SYMBOL><LOCAL_SYMBOL>'],
 '<simpleExpr-3>': ['<whenExpression><thenExpression>'],
 '<arrayCast>': ['<ARRAY_SYMBOL>'],
 '<jsonOperator>': ['<JSON_SEPARATOR_SYMBOL><textStringLiteral>',
                    '<JSON_UNQUOTED_SEPARATOR_SYMBOL><textStringLiteral>'],
 '<sumExpr>': ['<AVG_SYMBOL><OPEN_PAR_SYMBOL><DISTINCT_SYMBOL>?<inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<sumExpr-1><OPEN_PAR_SYMBOL><inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<jsonFunction>',
               '<ST_COLLECT_SYMBOL><OPEN_PAR_SYMBOL><DISTINCT_SYMBOL>?<inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<COUNT_SYMBOL><OPEN_PAR_SYMBOL><sumExpr-2><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<sumExpr-3><OPEN_PAR_SYMBOL><DISTINCT_SYMBOL>?<inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<sumExpr-4><OPEN_PAR_SYMBOL><inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<SUM_SYMBOL><OPEN_PAR_SYMBOL><DISTINCT_SYMBOL><inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
               '<GROUP_CONCAT_SYMBOL><OPEN_PAR_SYMBOL><DISTINCT_SYMBOL>?<exprList><orderClause>?<sumExpr-5>?<CLOSE_PAR_SYMBOL><windowingClause>?'],
 '<sumExpr-1>': ['<BIT_AND_SYMBOL>', '<BIT_OR_SYMBOL>', '<BIT_XOR_SYMBOL>'],
 '<sumExpr-2>': ['<ALL_SYMBOL>?<MULT_OPERATOR>',
                 '<inSumExpr>',
                 '<DISTINCT_SYMBOL><exprList>'],
 '<sumExpr-3>': ['<MIN_SYMBOL>', '<MAX_SYMBOL>'],
 '<sumExpr-4>': ['<STD_SYMBOL>',
                 '<VARIANCE_SYMBOL>',
                 '<STDDEV_SAMP_SYMBOL>',
                 '<VAR_SAMP_SYMBOL>',
                 '<SUM_SYMBOL>'],
 '<sumExpr-5>': ['<SEPARATOR_SYMBOL><textString>'],
 '<groupingOperation>': ['<GROUPING_SYMBOL><OPEN_PAR_SYMBOL><exprList><CLOSE_PAR_SYMBOL>'],
 '<windowFunctionCall>': ['<windowFunctionCall-1><parentheses><windowingClause>',
                          '<NTILE_SYMBOL><windowFunctionCall-2><windowingClause>',
                          '<windowFunctionCall-3><OPEN_PAR_SYMBOL><expr><leadLagInfo>?<CLOSE_PAR_SYMBOL><nullTreatment>?<windowingClause>',
                          '<windowFunctionCall-4><exprWithParentheses><nullTreatment>?<windowingClause>',
                          '<NTH_VALUE_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><simpleExpr><CLOSE_PAR_SYMBOL><windowFunctionCall-5>?<nullTreatment>?<windowingClause>'],
 '<windowFunctionCall-1>': ['<ROW_NUMBER_SYMBOL>',
                            '<RANK_SYMBOL>',
                            '<DENSE_RANK_SYMBOL>',
                            '<CUME_DIST_SYMBOL>',
                            '<PERCENT_RANK_SYMBOL>'],
 '<windowFunctionCall-2>': ['<OPEN_PAR_SYMBOL><stableInteger><CLOSE_PAR_SYMBOL>',
                            '<simpleExprWithParentheses>'],
 '<windowFunctionCall-3>': ['<LEAD_SYMBOL>', '<LAG_SYMBOL>'],
 '<windowFunctionCall-4>': ['<FIRST_VALUE_SYMBOL>', '<LAST_VALUE_SYMBOL>'],
 '<windowFunctionCall-5>': ['<FROM_SYMBOL><windowFunctionCall-6>'],
 '<windowFunctionCall-6>': ['<FIRST_SYMBOL>', '<LAST_SYMBOL>'],
 '<samplingMethod>': ['<SYSTEM_SYMBOL>', '<BERNOULLI_SYMBOL>'],
 '<samplingPercentage>': ['<ulonglongNumber>',
                          '<AT_SIGN_SYMBOL><textOrIdentifier>',
                          '<PARAM_MARKER>'],
 '<tablesampleClause>': ['<TABLESAMPLE_SYMBOL><samplingMethod><OPEN_PAR_SYMBOL><samplingPercentage><CLOSE_PAR_SYMBOL>'],
 '<windowingClause>': ['<OVER_SYMBOL><windowingClause-1>'],
 '<windowingClause-1>': ['<windowName>', '<windowSpec>'],
 '<leadLagInfo>': ['<COMMA_SYMBOL><leadLagInfo-1><leadLagInfo-2>?'],
 '<leadLagInfo-1>': ['<ulonglongNumber>', '<PARAM_MARKER>', '<stableInteger>'],
 '<leadLagInfo-2>': ['<COMMA_SYMBOL><expr>'],
 '<stableInteger>': ['<int64Literal>', '<paramOrVar>'],
 '<paramOrVar>': ['<PARAM_MARKER>',
                  '<identifier>',
                  '<AT_SIGN_SYMBOL><textOrIdentifier>'],
 '<nullTreatment>': ['<nullTreatment-1><NULLS_SYMBOL>'],
 '<nullTreatment-1>': ['<RESPECT_SYMBOL>', '<IGNORE_SYMBOL>'],
 '<jsonFunction>': ['<JSON_ARRAYAGG_SYMBOL><OPEN_PAR_SYMBOL><inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?',
                    '<JSON_OBJECTAGG_SYMBOL><OPEN_PAR_SYMBOL><inSumExpr><COMMA_SYMBOL><inSumExpr><CLOSE_PAR_SYMBOL><windowingClause>?'],
 '<inSumExpr>': ['<ALL_SYMBOL>?<expr>'],
 '<identListArg>': ['<identList>',
                    '<OPEN_PAR_SYMBOL><identList><CLOSE_PAR_SYMBOL>'],
 '<identList>': ['<simpleIdentifier><identList-1>*'],
 '<identList-1>': ['<COMMA_SYMBOL><simpleIdentifier>'],
 '<fulltextOptions>': ['<IN_SYMBOL><BOOLEAN_SYMBOL><MODE_SYMBOL>',
                       '<IN_SYMBOL><NATURAL_SYMBOL><LANGUAGE_SYMBOL><MODE_SYMBOL><fulltextOptions-1>?',
                       '<WITH_SYMBOL><QUERY_SYMBOL><EXPANSION_SYMBOL>'],
 '<fulltextOptions-1>': ['<WITH_SYMBOL><QUERY_SYMBOL><EXPANSION_SYMBOL>'],
 '<runtimeFunctionCall>': ['<CHAR_SYMBOL><OPEN_PAR_SYMBOL><exprList><runtimeFunctionCall-1>?<CLOSE_PAR_SYMBOL>',
                           '<CURRENT_USER_SYMBOL><parentheses>?',
                           '<DATE_SYMBOL><exprWithParentheses>',
                           '<DAY_SYMBOL><exprWithParentheses>',
                           '<HOUR_SYMBOL><exprWithParentheses>',
                           '<INSERT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><COMMA_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<INTERVAL_SYMBOL><OPEN_PAR_SYMBOL><expr><runtimeFunctionCall-2>+<CLOSE_PAR_SYMBOL>',
                           '<JSON_VALUE_SYMBOL><OPEN_PAR_SYMBOL><simpleExpr><COMMA_SYMBOL><textLiteral><returningType>?<onEmptyOrError><CLOSE_PAR_SYMBOL>',
                           '<LEFT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<MINUTE_SYMBOL><exprWithParentheses>',
                           '<MONTH_SYMBOL><exprWithParentheses>',
                           '<RIGHT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<SECOND_SYMBOL><exprWithParentheses>',
                           '<TIME_SYMBOL><exprWithParentheses>',
                           '<TIMESTAMP_SYMBOL><OPEN_PAR_SYMBOL><expr><runtimeFunctionCall-3>?<CLOSE_PAR_SYMBOL>',
                           '<trimFunction>',
                           '<userFunction>',
                           '<VALUES_SYMBOL><exprWithParentheses>',
                           '<YEAR_SYMBOL><exprWithParentheses>',
                           '<runtimeFunctionCall-4><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><runtimeFunctionCall-5><CLOSE_PAR_SYMBOL>',
                           '<CURDATE_SYMBOL><parentheses>?',
                           '<CURTIME_SYMBOL><timeFunctionParameters>?',
                           '<runtimeFunctionCall-6><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><INTERVAL_SYMBOL><expr><interval><CLOSE_PAR_SYMBOL>',
                           '<EXTRACT_SYMBOL><OPEN_PAR_SYMBOL><interval><FROM_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<GET_FORMAT_SYMBOL><OPEN_PAR_SYMBOL><dateTimeTtype><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<LOG_SYMBOL><OPEN_PAR_SYMBOL><expr><runtimeFunctionCall-7>?<CLOSE_PAR_SYMBOL>',
                           '<NOW_SYMBOL><timeFunctionParameters>?',
                           '<POSITION_SYMBOL><OPEN_PAR_SYMBOL><bitExpr><IN_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<substringFunction>',
                           '<SYSDATE_SYMBOL><timeFunctionParameters>?',
                           '<runtimeFunctionCall-8><OPEN_PAR_SYMBOL><intervalTimeStamp><COMMA_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<UTC_DATE_SYMBOL><parentheses>?',
                           '<UTC_TIME_SYMBOL><timeFunctionParameters>?',
                           '<UTC_TIMESTAMP_SYMBOL><timeFunctionParameters>?',
                           '<ASCII_SYMBOL><exprWithParentheses>',
                           '<CHARSET_SYMBOL><exprWithParentheses>',
                           '<COALESCE_SYMBOL><exprListWithParentheses>',
                           '<COLLATION_SYMBOL><exprWithParentheses>',
                           '<DATABASE_SYMBOL><parentheses>',
                           '<IF_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<FORMAT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><runtimeFunctionCall-9>?<CLOSE_PAR_SYMBOL>',
                           '<MICROSECOND_SYMBOL><exprWithParentheses>',
                           '<MOD_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<PASSWORD_SYMBOL><exprWithParentheses>',
                           '<QUARTER_SYMBOL><exprWithParentheses>',
                           '<REPEAT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<REPLACE_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<REVERSE_SYMBOL><exprWithParentheses>',
                           '<ROW_COUNT_SYMBOL><parentheses>',
                           '<TRUNCATE_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                           '<WEEK_SYMBOL><OPEN_PAR_SYMBOL><expr><runtimeFunctionCall-10>?<CLOSE_PAR_SYMBOL>',
                           '<WEIGHT_STRING_SYMBOL><OPEN_PAR_SYMBOL><expr><runtimeFunctionCall-11><CLOSE_PAR_SYMBOL>',
                           '<geometryFunction>'],
 '<runtimeFunctionCall-1>': ['<USING_SYMBOL><charsetName>'],
 '<runtimeFunctionCall-2>': ['<COMMA_SYMBOL><expr>'],
 '<runtimeFunctionCall-3>': ['<COMMA_SYMBOL><expr>'],
 '<runtimeFunctionCall-4>': ['<ADDDATE_SYMBOL>', '<SUBDATE_SYMBOL>'],
 '<runtimeFunctionCall-5>': ['<expr>', '<INTERVAL_SYMBOL><expr><interval>'],
 '<runtimeFunctionCall-6>': ['<DATE_ADD_SYMBOL>', '<DATE_SUB_SYMBOL>'],
 '<runtimeFunctionCall-7>': ['<COMMA_SYMBOL><expr>'],
 '<runtimeFunctionCall-8>': ['<TIMESTAMPADD_SYMBOL>', '<TIMESTAMPDIFF_SYMBOL>'],
 '<runtimeFunctionCall-9>': ['<COMMA_SYMBOL><expr>'],
 '<runtimeFunctionCall-10>': ['<COMMA_SYMBOL><expr>'],
 '<runtimeFunctionCall-11>': ['<runtimeFunctionCall-12>?',
                              '<AS_SYMBOL><BINARY_SYMBOL><wsNumCodepoints>',
                              '<COMMA_SYMBOL><ulong_number><COMMA_SYMBOL><ulong_number><COMMA_SYMBOL><ulong_number>'],
 '<runtimeFunctionCall-12>': ['<AS_SYMBOL><CHAR_SYMBOL><wsNumCodepoints>'],
 '<returningType>': ['<RETURNING_SYMBOL><castType>'],
 '<geometryFunction>': ['<GEOMETRYCOLLECTION_SYMBOL><OPEN_PAR_SYMBOL><exprList>?<CLOSE_PAR_SYMBOL>',
                        '<LINESTRING_SYMBOL><exprListWithParentheses>',
                        '<MULTILINESTRING_SYMBOL><exprListWithParentheses>',
                        '<MULTIPOINT_SYMBOL><exprListWithParentheses>',
                        '<MULTIPOLYGON_SYMBOL><exprListWithParentheses>',
                        '<POINT_SYMBOL><OPEN_PAR_SYMBOL><expr><COMMA_SYMBOL><expr><CLOSE_PAR_SYMBOL>',
                        '<POLYGON_SYMBOL><exprListWithParentheses>'],
 '<timeFunctionParameters>': ['<OPEN_PAR_SYMBOL><fractionalPrecision>?<CLOSE_PAR_SYMBOL>'],
 '<fractionalPrecision>': ['<INT_NUMBER>'],
 '<dateTimeTtype>': ['<DATE_SYMBOL>',
                     '<TIME_SYMBOL>',
                     '<DATETIME_SYMBOL>',
                     '<TIMESTAMP_SYMBOL>'],
 '<trimFunction>': ['<TRIM_SYMBOL><OPEN_PAR_SYMBOL><trimFunction-1><CLOSE_PAR_SYMBOL>'],
 '<trimFunction-1>': ['<expr><trimFunction-2>?',
                      '<LEADING_SYMBOL><expr>?<FROM_SYMBOL><expr>',
                      '<TRAILING_SYMBOL><expr>?<FROM_SYMBOL><expr>',
                      '<BOTH_SYMBOL><expr>?<FROM_SYMBOL><expr>'],
 '<trimFunction-2>': ['<FROM_SYMBOL><expr>'],
 '<substringFunction>': ['<SUBSTRING_SYMBOL><OPEN_PAR_SYMBOL><expr><substringFunction-1><CLOSE_PAR_SYMBOL>'],
 '<substringFunction-1>': ['<COMMA_SYMBOL><expr><substringFunction-2>?',
                           '<FROM_SYMBOL><expr><substringFunction-3>?'],
 '<substringFunction-2>': ['<COMMA_SYMBOL><expr>'],
 '<substringFunction-3>': ['<FOR_SYMBOL><expr>'],
 '<functionCall>': ['<pureIdentifier><OPEN_PAR_SYMBOL><udfExprList>?<CLOSE_PAR_SYMBOL>',
                    '<qualifiedIdentifier><OPEN_PAR_SYMBOL><exprList>?<CLOSE_PAR_SYMBOL>'],
 '<udfExprList>': ['<udfExpr><udfExprList-1>*'],
 '<udfExprList-1>': ['<COMMA_SYMBOL><udfExpr>'],
 '<udfExpr>': ['<expr><selectAlias>?'],
 '<userVariable>': ['<AT_SIGN_SYMBOL><textOrIdentifier>', '<AT_TEXT_SUFFIX>'],
 '<inExpressionUserVariableAssignment>': ['<userVariable><ASSIGN_OPERATOR><expr>'],
 '<rvalueSystemOrUserVariable>': ['<userVariable>',
                                  '<AT_AT_SIGN_SYMBOL><rvalueSystemVariableType>?<rvalueSystemVariable>'],
 '<lvalueVariable>': ['<lvalueVariable-1>', '<DEFAULT_SYMBOL><dotIdentifier>'],
 '<lvalueVariable-1>': ['<identifier><dotIdentifier>?',
                        '<lValueIdentifier><dotIdentifier>?'],
 '<rvalueSystemVariable>': ['<textOrIdentifier><dotIdentifier>?'],
 '<whenExpression>': ['<WHEN_SYMBOL><expr>'],
 '<thenExpression>': ['<THEN_SYMBOL><expr>'],
 '<elseExpression>': ['<ELSE_SYMBOL><expr>'],
 '<castType>': ['<BINARY_SYMBOL><fieldLength>?',
                '<CHAR_SYMBOL><fieldLength>?<charsetWithOptBinary>?',
                '<nchar><fieldLength>?',
                '<SIGNED_SYMBOL><INT_SYMBOL>?',
                '<UNSIGNED_SYMBOL><INT_SYMBOL>?',
                '<DATE_SYMBOL>',
                '<YEAR_SYMBOL>',
                '<TIME_SYMBOL><typeDatetimePrecision>?',
                '<DATETIME_SYMBOL><typeDatetimePrecision>?',
                '<DECIMAL_SYMBOL><floatOptions>?',
                '<JSON_SYMBOL>',
                '<realType>',
                '<FLOAT_SYMBOL><standardFloatOptions>?',
                '<castType-1>'],
 '<castType-1>': ['<POINT_SYMBOL>',
                  '<LINESTRING_SYMBOL>',
                  '<POLYGON_SYMBOL>',
                  '<MULTIPOINT_SYMBOL>',
                  '<MULTILINESTRING_SYMBOL>',
                  '<MULTIPOLYGON_SYMBOL>',
                  '<GEOMETRYCOLLECTION_SYMBOL>'],
 '<exprList>': ['<expr><exprList-1>*'],
 '<exprList-1>': ['<COMMA_SYMBOL><expr>'],
 '<charset>': ['<CHAR_SYMBOL><SET_SYMBOL>', '<CHARSET_SYMBOL>'],
 '<notRule>': ['<NOT_SYMBOL>', 'NOT2_SYMBOL'],
 '<not2Rule>': ['<LOGICAL_NOT_OPERATOR>', 'NOT2_SYMBOL'],
 '<interval>': ['<intervalTimeStamp>', '<interval-1>'],
 '<interval-1>': ['<SECOND_MICROSECOND_SYMBOL>',
                  '<MINUTE_MICROSECOND_SYMBOL>',
                  '<MINUTE_SECOND_SYMBOL>',
                  '<HOUR_MICROSECOND_SYMBOL>',
                  '<HOUR_SECOND_SYMBOL>',
                  '<HOUR_MINUTE_SYMBOL>',
                  '<DAY_MICROSECOND_SYMBOL>',
                  '<DAY_SECOND_SYMBOL>',
                  '<DAY_MINUTE_SYMBOL>',
                  '<DAY_HOUR_SYMBOL>',
                  '<YEAR_MONTH_SYMBOL>'],
 '<intervalTimeStamp>': ['<MICROSECOND_SYMBOL>',
                         '<SECOND_SYMBOL>',
                         '<MINUTE_SYMBOL>',
                         '<HOUR_SYMBOL>',
                         '<DAY_SYMBOL>',
                         '<WEEK_SYMBOL>',
                         '<MONTH_SYMBOL>',
                         '<QUARTER_SYMBOL>',
                         '<YEAR_SYMBOL>'],
 '<exprListWithParentheses>': ['<OPEN_PAR_SYMBOL><exprList><CLOSE_PAR_SYMBOL>'],
 '<exprWithParentheses>': ['<OPEN_PAR_SYMBOL><expr><CLOSE_PAR_SYMBOL>'],
 '<simpleExprWithParentheses>': ['<OPEN_PAR_SYMBOL><simpleExpr><CLOSE_PAR_SYMBOL>'],
 '<orderList>': ['<orderExpression><orderList-1>*'],
 '<orderList-1>': ['<COMMA_SYMBOL><orderExpression>'],
 '<orderExpression>': ['<expr><direction>?'],
 '<groupList>': ['<groupingExpression><groupList-1>*'],
 '<groupList-1>': ['<COMMA_SYMBOL><groupingExpression>'],
 '<groupingExpression>': ['<expr>'],
 '<channel>': ['<FOR_SYMBOL><CHANNEL_SYMBOL><textStringNoLinebreak>'],
 '<compoundStatement>': ['<simpleStatement>',
                         '<returnStatement>',
                         '<ifStatement>',
                         '<caseStatement>',
                         '<labeledBlock>',
                         '<unlabeledBlock>',
                         '<labeledControl>',
                         '<unlabeledControl>',
                         '<leaveStatement>',
                         '<iterateStatement>',
                         '<cursorOpen>',
                         '<cursorFetch>',
                         '<cursorClose>'],
 '<returnStatement>': ['<RETURN_SYMBOL><expr>'],
 '<ifStatement>': ['<IF_SYMBOL><ifBody><END_SYMBOL><IF_SYMBOL>'],
 '<ifBody>': ['<expr><thenStatement><ifBody-1>?'],
 '<ifBody-1>': ['<ELSEIF_SYMBOL><ifBody>',
                '<ELSE_SYMBOL><compoundStatementList>'],
 '<thenStatement>': ['<THEN_SYMBOL><compoundStatementList>'],
 '<compoundStatementList>': ['<compoundStatementList-1>+'],
 '<compoundStatementList-1>': ['<compoundStatement><SEMICOLON_SYMBOL>'],
 '<caseStatement>': ['<CASE_SYMBOL><expr>?<caseStatement-1>+<elseStatement>?<END_SYMBOL><CASE_SYMBOL>'],
 '<caseStatement-1>': ['<whenExpression><thenStatement>'],
 '<elseStatement>': ['<ELSE_SYMBOL><compoundStatementList>'],
 '<labeledBlock>': ['<label><beginEndBlock><labelRef>?'],
 '<unlabeledBlock>': ['<beginEndBlock>'],
 '<label>': ['<labelIdentifier><COLON_SYMBOL>'],
 '<beginEndBlock>': ['<BEGIN_SYMBOL><spDeclarations>?<compoundStatementList>?<END_SYMBOL>'],
 '<labeledControl>': ['<label><unlabeledControl><labelRef>?'],
 '<unlabeledControl>': ['<loopBlock>', '<whileDoBlock>', '<repeatUntilBlock>'],
 '<loopBlock>': ['<LOOP_SYMBOL><compoundStatementList><END_SYMBOL><LOOP_SYMBOL>'],
 '<whileDoBlock>': ['<WHILE_SYMBOL><expr><DO_SYMBOL><compoundStatementList><END_SYMBOL><WHILE_SYMBOL>'],
 '<repeatUntilBlock>': ['<REPEAT_SYMBOL><compoundStatementList><UNTIL_SYMBOL><expr><END_SYMBOL><REPEAT_SYMBOL>'],
 '<spDeclarations>': ['<spDeclarations-1>+'],
 '<spDeclarations-1>': ['<spDeclaration><SEMICOLON_SYMBOL>'],
 '<spDeclaration>': ['<variableDeclaration>',
                     '<conditionDeclaration>',
                     '<handlerDeclaration>',
                     '<cursorDeclaration>'],
 '<variableDeclaration>': ['<DECLARE_SYMBOL><identifierList><dataType><collate>?<variableDeclaration-1>?'],
 '<variableDeclaration-1>': ['<DEFAULT_SYMBOL><expr>'],
 '<conditionDeclaration>': ['<DECLARE_SYMBOL><identifier><CONDITION_SYMBOL><FOR_SYMBOL><spCondition>'],
 '<spCondition>': ['<ulong_number>', '<sqlstate>'],
 '<sqlstate>': ['<SQLSTATE_SYMBOL><VALUE_SYMBOL>?<textLiteral>'],
 '<handlerDeclaration>': ['<DECLARE_SYMBOL><handlerDeclaration-1><HANDLER_SYMBOL><FOR_SYMBOL><handlerCondition><handlerDeclaration-2>*<compoundStatement>'],
 '<handlerDeclaration-1>': ['<CONTINUE_SYMBOL>',
                            '<EXIT_SYMBOL>',
                            '<UNDO_SYMBOL>'],
 '<handlerDeclaration-2>': ['<COMMA_SYMBOL><handlerCondition>'],
 '<handlerCondition>': ['<spCondition>',
                        '<identifier>',
                        '<SQLWARNING_SYMBOL>',
                        '<notRule><FOUND_SYMBOL>',
                        '<SQLEXCEPTION_SYMBOL>'],
 '<cursorDeclaration>': ['<DECLARE_SYMBOL><identifier><CURSOR_SYMBOL><FOR_SYMBOL><selectStatement>'],
 '<iterateStatement>': ['<ITERATE_SYMBOL><labelRef>'],
 '<leaveStatement>': ['<LEAVE_SYMBOL><labelRef>'],
 '<getDiagnosticsStatement>': ['<GET_SYMBOL><getDiagnosticsStatement-1>?<DIAGNOSTICS_SYMBOL><getDiagnosticsStatement-2>'],
 '<getDiagnosticsStatement-1>': ['<CURRENT_SYMBOL>', '<STACKED_SYMBOL>'],
 '<getDiagnosticsStatement-2>': ['<statementInformationItem><getDiagnosticsStatement-3>*',
                                 '<CONDITION_SYMBOL><signalAllowedExpr><conditionInformationItem><getDiagnosticsStatement-4>*'],
 '<getDiagnosticsStatement-3>': ['<COMMA_SYMBOL><statementInformationItem>'],
 '<getDiagnosticsStatement-4>': ['<COMMA_SYMBOL><conditionInformationItem>'],
 '<signalAllowedExpr>': ['<literal>',
                         '<rvalueSystemOrUserVariable>',
                         '<qualifiedIdentifier>'],
 '<statementInformationItem>': ['<statementInformationItem-1><EQUAL_OPERATOR><statementInformationItem-2>'],
 '<statementInformationItem-1>': ['<userVariable>', '<identifier>'],
 '<statementInformationItem-2>': ['<NUMBER_SYMBOL>', '<ROW_COUNT_SYMBOL>'],
 '<conditionInformationItem>': ['<conditionInformationItem-1><EQUAL_OPERATOR><conditionInformationItem-2>'],
 '<conditionInformationItem-1>': ['<userVariable>', '<identifier>'],
 '<conditionInformationItem-2>': ['<signalInformationItemName>',
                                  '<RETURNED_SQLSTATE_SYMBOL>'],
 '<signalInformationItemName>': ['<CLASS_ORIGIN_SYMBOL>',
                                 '<SUBCLASS_ORIGIN_SYMBOL>',
                                 '<CONSTRAINT_CATALOG_SYMBOL>',
                                 '<CONSTRAINT_SCHEMA_SYMBOL>',
                                 '<CONSTRAINT_NAME_SYMBOL>',
                                 '<CATALOG_NAME_SYMBOL>',
                                 '<SCHEMA_NAME_SYMBOL>',
                                 '<TABLE_NAME_SYMBOL>',
                                 '<COLUMN_NAME_SYMBOL>',
                                 '<CURSOR_NAME_SYMBOL>',
                                 '<MESSAGE_TEXT_SYMBOL>',
                                 '<MYSQL_ERRNO_SYMBOL>'],
 '<signalStatement>': ['<SIGNAL_SYMBOL><signalStatement-1><signalStatement-2>?'],
 '<signalStatement-1>': ['<identifier>', '<sqlstate>'],
 '<signalStatement-2>': ['<SET_SYMBOL><signalInformationItem><signalStatement-3>*'],
 '<signalStatement-3>': ['<COMMA_SYMBOL><signalInformationItem>'],
 '<resignalStatement>': ['<RESIGNAL_SYMBOL><resignalStatement-1>?<resignalStatement-2>?'],
 '<resignalStatement-1>': ['<identifier>', '<sqlstate>'],
 '<resignalStatement-2>': ['<SET_SYMBOL><signalInformationItem><resignalStatement-3>*'],
 '<resignalStatement-3>': ['<COMMA_SYMBOL><signalInformationItem>'],
 '<signalInformationItem>': ['<signalInformationItemName><EQUAL_OPERATOR><signalAllowedExpr>'],
 '<cursorOpen>': ['<OPEN_SYMBOL><identifier>'],
 '<cursorClose>': ['<CLOSE_SYMBOL><identifier>'],
 '<cursorFetch>': ['<FETCH_SYMBOL><cursorFetch-1>?<identifier><INTO_SYMBOL><identifierList>'],
 '<cursorFetch-1>': ['<NEXT_SYMBOL>?<FROM_SYMBOL>'],
 '<schedule>': ['<AT_SYMBOL><expr>',
                '<EVERY_SYMBOL><expr><interval><schedule-1>?<schedule-2>?'],
 '<schedule-1>': ['<STARTS_SYMBOL><expr>'],
 '<schedule-2>': ['<ENDS_SYMBOL><expr>'],
 '<columnDefinition>': ['<columnName><fieldDefinition><checkOrReferences>?'],
 '<checkOrReferences>': ['<checkConstraint>', '<references>'],
 '<checkConstraint>': ['<CHECK_SYMBOL><exprWithParentheses>'],
 '<constraintEnforcement>': ['<NOT_SYMBOL>?<ENFORCED_SYMBOL>'],
 '<tableConstraintDef>': ['<tableConstraintDef-1><indexNameAndType>?<keyListWithExpression><indexOption>*',
                          '<FULLTEXT_SYMBOL><keyOrIndex>?<indexName>?<keyListWithExpression><fulltextIndexOption>*',
                          '<SPATIAL_SYMBOL><keyOrIndex>?<indexName>?<keyListWithExpression><spatialIndexOption>*',
                          '<constraintName>?<tableConstraintDef-2>'],
 '<tableConstraintDef-1>': ['<KEY_SYMBOL>', '<INDEX_SYMBOL>'],
 '<tableConstraintDef-2>': ['<tableConstraintDef-3><indexNameAndType>?<keyListWithExpression><indexOption>*',
                            '<FOREIGN_SYMBOL><KEY_SYMBOL><indexName>?<keyList><references>',
                            '<checkConstraint><constraintEnforcement>?'],
 '<tableConstraintDef-3>': ['<PRIMARY_SYMBOL><KEY_SYMBOL>',
                            '<UNIQUE_SYMBOL><keyOrIndex>?'],
 '<constraintName>': ['<CONSTRAINT_SYMBOL><identifier>?'],
 '<fieldDefinition>': ['<dataType><fieldDefinition-1>'],
 '<fieldDefinition-1>': ['<columnAttribute>*',
                         '<collate>?<fieldDefinition-2>?<AS_SYMBOL><exprWithParentheses><fieldDefinition-3>?<columnAttribute>*'],
 '<fieldDefinition-2>': ['<GENERATED_SYMBOL><ALWAYS_SYMBOL>'],
 '<fieldDefinition-3>': ['<VIRTUAL_SYMBOL>', '<STORED_SYMBOL>'],
 '<columnAttribute>': ['<NOT_SYMBOL>?<nullLiteral>',
                       '<NOT_SYMBOL><SECONDARY_SYMBOL>',
                       '<DEFAULT_SYMBOL><columnAttribute-1>',
                       '<ON_SYMBOL><UPDATE_SYMBOL><NOW_SYMBOL><timeFunctionParameters>?',
                       '<AUTO_INCREMENT_SYMBOL>',
                       '<SERIAL_SYMBOL><DEFAULT_SYMBOL><VALUE_SYMBOL>',
                       '<PRIMARY_SYMBOL>?<KEY_SYMBOL>',
                       '<UNIQUE_SYMBOL><KEY_SYMBOL>?',
                       '<COMMENT_SYMBOL><textLiteral>',
                       '<collate>',
                       '<COLUMN_FORMAT_SYMBOL><columnFormat>',
                       '<STORAGE_SYMBOL><storageMedia>',
                       '<SRID_SYMBOL><real_ulonglong_number>',
                       '<constraintName>?<checkConstraint>',
                       '<constraintEnforcement>',
                       '<ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>',
                       '<SECONDARY_ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>',
                       '<visibility>'],
 '<columnAttribute-1>': ['<nowOrSignedLiteral>', '<exprWithParentheses>'],
 '<columnFormat>': ['<FIXED_SYMBOL>', '<DYNAMIC_SYMBOL>', '<DEFAULT_SYMBOL>'],
 '<storageMedia>': ['<DISK_SYMBOL>', '<MEMORY_SYMBOL>', '<DEFAULT_SYMBOL>'],
 '<now>': ['<NOW_SYMBOL><functionDatetimePrecision>'],
 '<nowOrSignedLiteral>': ['<now>', '<signedLiteralOrNull>'],
 '<references>': ['<REFERENCES_SYMBOL><tableRef><identifierListWithParentheses>?<references-1>?<references-3>?'],
 '<references-1>': ['<MATCH_SYMBOL><references-2>'],
 '<references-2>': ['<FULL_SYMBOL>', '<PARTIAL_SYMBOL>', '<SIMPLE_SYMBOL>'],
 '<references-3>': ['<ON_SYMBOL><UPDATE_SYMBOL><deleteOption><references-4>?',
                    '<ON_SYMBOL><DELETE_SYMBOL><deleteOption><references-5>?'],
 '<references-4>': ['<ON_SYMBOL><DELETE_SYMBOL><deleteOption>'],
 '<references-5>': ['<ON_SYMBOL><UPDATE_SYMBOL><deleteOption>'],
 '<deleteOption>': ['<deleteOption-1>',
                    '<SET_SYMBOL><nullLiteral>',
                    '<SET_SYMBOL><DEFAULT_SYMBOL>',
                    '<NO_SYMBOL><ACTION_SYMBOL>'],
 '<deleteOption-1>': ['<RESTRICT_SYMBOL>', '<CASCADE_SYMBOL>'],
 '<keyList>': ['<OPEN_PAR_SYMBOL><keyPart><keyList-1>*<CLOSE_PAR_SYMBOL>'],
 '<keyList-1>': ['<COMMA_SYMBOL><keyPart>'],
 '<keyPart>': ['<identifier><fieldLength>?<direction>?'],
 '<keyListWithExpression>': ['<OPEN_PAR_SYMBOL><keyPartOrExpression><keyListWithExpression-1>*<CLOSE_PAR_SYMBOL>'],
 '<keyListWithExpression-1>': ['<COMMA_SYMBOL><keyPartOrExpression>'],
 '<keyPartOrExpression>': ['<keyPart>', '<exprWithParentheses><direction>?'],
 '<indexType>': ['<indexType-1>'],
 '<indexType-1>': ['<BTREE_SYMBOL>', '<RTREE_SYMBOL>', '<HASH_SYMBOL>'],
 '<indexOption>': ['<commonIndexOption>', '<indexTypeClause>'],
 '<commonIndexOption>': ['<KEY_BLOCK_SIZE_SYMBOL><EQUAL_OPERATOR>?<ulong_number>',
                         '<COMMENT_SYMBOL><textLiteral>',
                         '<visibility>',
                         '<ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>',
                         '<SECONDARY_ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>'],
 '<visibility>': ['<VISIBLE_SYMBOL>', '<INVISIBLE_SYMBOL>'],
 '<indexTypeClause>': ['<indexTypeClause-1><indexType>'],
 '<indexTypeClause-1>': ['<USING_SYMBOL>', '<TYPE_SYMBOL>'],
 '<fulltextIndexOption>': ['<commonIndexOption>',
                           '<WITH_SYMBOL><PARSER_SYMBOL><identifier>'],
 '<spatialIndexOption>': ['<commonIndexOption>'],
 '<dataType>': ['<dataType-1><fieldLength>?<fieldOptions>?',
                '<dataType-2><precision>?<fieldOptions>?',
                '<dataType-3><floatOptions>?<fieldOptions>?',
                '<BIT_SYMBOL><fieldLength>?',
                '<dataType-4>',
                '<CHAR_SYMBOL><fieldLength>?<charsetWithOptBinary>?',
                '<nchar><fieldLength>?<BINARY_SYMBOL>?',
                '<BINARY_SYMBOL><fieldLength>?',
                '<dataType-5><fieldLength><charsetWithOptBinary>?',
                '<dataType-6><fieldLength><BINARY_SYMBOL>?',
                '<VARBINARY_SYMBOL><fieldLength>',
                '<YEAR_SYMBOL><fieldLength>?<fieldOptions>?',
                '<DATE_SYMBOL>',
                '<TIME_SYMBOL><typeDatetimePrecision>?',
                '<TIMESTAMP_SYMBOL><typeDatetimePrecision>?',
                '<DATETIME_SYMBOL><typeDatetimePrecision>?',
                '<TINYBLOB_SYMBOL>',
                '<BLOB_SYMBOL><fieldLength>?',
                '<dataType-7>',
                '<LONG_SYMBOL><VARBINARY_SYMBOL>',
                '<LONG_SYMBOL><dataType-8>?<charsetWithOptBinary>?',
                '<TINYTEXT_SYMBOL><charsetWithOptBinary>?',
                '<TEXT_SYMBOL><fieldLength>?<charsetWithOptBinary>?',
                '<MEDIUMTEXT_SYMBOL><charsetWithOptBinary>?',
                '<LONGTEXT_SYMBOL><charsetWithOptBinary>?',
                '<ENUM_SYMBOL><stringList><charsetWithOptBinary>?',
                '<SET_SYMBOL><stringList><charsetWithOptBinary>?',
                '<SERIAL_SYMBOL>',
                '<JSON_SYMBOL>',
                '<dataType-9>'],
 '<dataType-1>': ['<INT_SYMBOL>',
                  '<TINYINT_SYMBOL>',
                  '<SMALLINT_SYMBOL>',
                  '<MEDIUMINT_SYMBOL>',
                  '<BIGINT_SYMBOL>'],
 '<dataType-2>': ['<REAL_SYMBOL>', '<DOUBLE_SYMBOL><PRECISION_SYMBOL>?'],
 '<dataType-3>': ['<FLOAT_SYMBOL>',
                  '<DECIMAL_SYMBOL>',
                  '<NUMERIC_SYMBOL>',
                  '<FIXED_SYMBOL>'],
 '<dataType-4>': ['<BOOL_SYMBOL>', '<BOOLEAN_SYMBOL>'],
 '<dataType-5>': ['<CHAR_SYMBOL><VARYING_SYMBOL>', '<VARCHAR_SYMBOL>'],
 '<dataType-6>': ['<NATIONAL_SYMBOL><VARCHAR_SYMBOL>',
                  '<NVARCHAR_SYMBOL>',
                  '<NCHAR_SYMBOL><VARCHAR_SYMBOL>',
                  '<NATIONAL_SYMBOL><CHAR_SYMBOL><VARYING_SYMBOL>',
                  '<NCHAR_SYMBOL><VARYING_SYMBOL>'],
 '<dataType-7>': ['<MEDIUMBLOB_SYMBOL>', '<LONGBLOB_SYMBOL>'],
 '<dataType-8>': ['<CHAR_SYMBOL><VARYING_SYMBOL>', '<VARCHAR_SYMBOL>'],
 '<dataType-9>': ['<GEOMETRY_SYMBOL>',
                  '<GEOMETRYCOLLECTION_SYMBOL>',
                  '<POINT_SYMBOL>',
                  '<MULTIPOINT_SYMBOL>',
                  '<LINESTRING_SYMBOL>',
                  '<MULTILINESTRING_SYMBOL>',
                  '<POLYGON_SYMBOL>',
                  '<MULTIPOLYGON_SYMBOL>'],
 '<nchar>': ['<NCHAR_SYMBOL>', '<NATIONAL_SYMBOL><CHAR_SYMBOL>'],
 '<realType>': ['<REAL_SYMBOL>', '<DOUBLE_SYMBOL><PRECISION_SYMBOL>?'],
 '<fieldLength>': ['<OPEN_PAR_SYMBOL><fieldLength-1><CLOSE_PAR_SYMBOL>'],
 '<fieldLength-1>': ['<real_ulonglong_number>', '<DECIMAL_NUMBER>'],
 '<fieldOptions>': ['<fieldOptions-1>+'],
 '<fieldOptions-1>': ['<SIGNED_SYMBOL>',
                      '<UNSIGNED_SYMBOL>',
                      '<ZEROFILL_SYMBOL>'],
 '<charsetWithOptBinary>': ['<ascii>',
                            '<unicode>',
                            '<BYTE_SYMBOL>',
                            '<charset><charsetName><BINARY_SYMBOL>?',
                            '<BINARY_SYMBOL><charsetWithOptBinary-1>?'],
 '<charsetWithOptBinary-1>': ['<charset><charsetName>'],
 '<ascii>': ['<ASCII_SYMBOL><BINARY_SYMBOL>?', '<BINARY_SYMBOL><ASCII_SYMBOL>'],
 '<unicode>': ['<UNICODE_SYMBOL><BINARY_SYMBOL>?',
               '<BINARY_SYMBOL><UNICODE_SYMBOL>'],
 '<wsNumCodepoints>': ['<OPEN_PAR_SYMBOL><real_ulong_number><CLOSE_PAR_SYMBOL>'],
 '<typeDatetimePrecision>': ['<OPEN_PAR_SYMBOL><INT_NUMBER><CLOSE_PAR_SYMBOL>'],
 '<functionDatetimePrecision>': ['',
                                 '<parentheses>',
                                 '<OPEN_PAR_SYMBOL><INT_NUMBER><CLOSE_PAR_SYMBOL>'],
 '<charsetName>': ['<textOrIdentifier>', '<BINARY_SYMBOL>', '<DEFAULT_SYMBOL>'],
 '<collationName>': ['<textOrIdentifier>',
                     '<DEFAULT_SYMBOL>',
                     '<BINARY_SYMBOL>'],
 '<createTableOptions>': ['<createTableOption><createTableOptions-1>*'],
 '<createTableOptions-1>': ['<COMMA_SYMBOL>?<createTableOption>'],
 '<createTableOptionsEtc>': ['<createTableOptions><createPartitioningEtc>?',
                             '<createPartitioningEtc>'],
 '<createPartitioningEtc>': ['<partitionClause><duplicateAsQe>?',
                             '<duplicateAsQe>'],
 '<createTableOptionsSpaceSeparated>': ['<createTableOption>+'],
 '<createTableOption>': ['<ENGINE_SYMBOL><EQUAL_OPERATOR>?<engineRef>',
                         '<SECONDARY_ENGINE_SYMBOL><equal>?<createTableOption-1>',
                         '<MAX_ROWS_SYMBOL><EQUAL_OPERATOR>?<ulonglongNumber>',
                         '<MIN_ROWS_SYMBOL><EQUAL_OPERATOR>?<ulonglongNumber>',
                         '<AVG_ROW_LENGTH_SYMBOL><EQUAL_OPERATOR>?<ulonglongNumber>',
                         '<PASSWORD_SYMBOL><EQUAL_OPERATOR>?<textStringLiteral>',
                         '<COMMENT_SYMBOL><EQUAL_OPERATOR>?<textStringLiteral>',
                         '<COMPRESSION_SYMBOL><EQUAL_OPERATOR>?<textString>',
                         '<ENCRYPTION_SYMBOL><EQUAL_OPERATOR>?<textString>',
                         '<AUTO_INCREMENT_SYMBOL><EQUAL_OPERATOR>?<ulonglongNumber>',
                         '<PACK_KEYS_SYMBOL><EQUAL_OPERATOR>?<ternaryOption>',
                         '<createTableOption-2><EQUAL_OPERATOR>?<ternaryOption>',
                         '<createTableOption-3><EQUAL_OPERATOR>?<ulong_number>',
                         '<DELAY_KEY_WRITE_SYMBOL><EQUAL_OPERATOR>?<ulong_number>',
                         '<ROW_FORMAT_SYMBOL><EQUAL_OPERATOR>?<createTableOption-4>',
                         '<UNION_SYMBOL><EQUAL_OPERATOR>?<OPEN_PAR_SYMBOL><tableRefList><CLOSE_PAR_SYMBOL>',
                         '<defaultCharset>',
                         '<defaultCollation>',
                         '<INSERT_METHOD_SYMBOL><EQUAL_OPERATOR>?<createTableOption-5>',
                         '<DATA_SYMBOL><DIRECTORY_SYMBOL><EQUAL_OPERATOR>?<textString>',
                         '<INDEX_SYMBOL><DIRECTORY_SYMBOL><EQUAL_OPERATOR>?<textString>',
                         '<TABLESPACE_SYMBOL><EQUAL_OPERATOR>?<identifier>',
                         '<STORAGE_SYMBOL><createTableOption-6>',
                         '<CONNECTION_SYMBOL><EQUAL_OPERATOR>?<textString>',
                         '<KEY_BLOCK_SIZE_SYMBOL><EQUAL_OPERATOR>?<ulonglongNumber>',
                         '<START_SYMBOL><TRANSACTION_SYMBOL>',
                         '<ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>',
                         '<SECONDARY_ENGINE_ATTRIBUTE_SYMBOL><EQUAL_OPERATOR>?<jsonAttribute>',
                         '<tsOptionAutoextendSize>'],
 '<createTableOption-1>': ['<NULL_SYMBOL>', '<textOrIdentifier>'],
 '<createTableOption-2>': ['<STATS_AUTO_RECALC_SYMBOL>',
                           '<STATS_PERSISTENT_SYMBOL>',
                           '<STATS_SAMPLE_PAGES_SYMBOL>'],
 '<createTableOption-3>': ['<CHECKSUM_SYMBOL>', '<TABLE_CHECKSUM_SYMBOL>'],
 '<createTableOption-4>': ['<DEFAULT_SYMBOL>',
                           '<DYNAMIC_SYMBOL>',
                           '<FIXED_SYMBOL>',
                           '<COMPRESSED_SYMBOL>',
                           '<REDUNDANT_SYMBOL>',
                           '<COMPACT_SYMBOL>'],
 '<createTableOption-5>': ['<NO_SYMBOL>', '<FIRST_SYMBOL>', '<LAST_SYMBOL>'],
 '<createTableOption-6>': ['<DISK_SYMBOL>', '<MEMORY_SYMBOL>'],
 '<ternaryOption>': ['<ulong_number>', '<DEFAULT_SYMBOL>'],
 '<defaultCollation>': ['<DEFAULT_SYMBOL>?<COLLATE_SYMBOL><EQUAL_OPERATOR>?<collationName>'],
 '<defaultEncryption>': ['<DEFAULT_SYMBOL>?<ENCRYPTION_SYMBOL><EQUAL_OPERATOR>?<textStringLiteral>'],
 '<defaultCharset>': ['<DEFAULT_SYMBOL>?<charset><EQUAL_OPERATOR>?<charsetName>'],
 '<partitionClause>': ['<PARTITION_SYMBOL><BY_SYMBOL><partitionTypeDef><partitionClause-1>?<subPartitions>?<partitionDefinitions>?'],
 '<partitionClause-1>': ['<PARTITIONS_SYMBOL><real_ulong_number>'],
 '<partitionTypeDef>': ['<LINEAR_SYMBOL>?<KEY_SYMBOL><partitionKeyAlgorithm>?<OPEN_PAR_SYMBOL><identifierList>?<CLOSE_PAR_SYMBOL>',
                        '<LINEAR_SYMBOL>?<HASH_SYMBOL><OPEN_PAR_SYMBOL><bitExpr><CLOSE_PAR_SYMBOL>',
                        '<partitionTypeDef-1><partitionTypeDef-2>'],
 '<partitionTypeDef-1>': ['<RANGE_SYMBOL>', '<LIST_SYMBOL>'],
 '<partitionTypeDef-2>': ['<OPEN_PAR_SYMBOL><bitExpr><CLOSE_PAR_SYMBOL>',
                          '<COLUMNS_SYMBOL><OPEN_PAR_SYMBOL><identifierList>?<CLOSE_PAR_SYMBOL>'],
 '<subPartitions>': ['<SUBPARTITION_SYMBOL><BY_SYMBOL><LINEAR_SYMBOL>?<subPartitions-1><subPartitions-2>?'],
 '<subPartitions-1>': ['<HASH_SYMBOL><OPEN_PAR_SYMBOL><bitExpr><CLOSE_PAR_SYMBOL>',
                       '<KEY_SYMBOL><partitionKeyAlgorithm>?<identifierListWithParentheses>'],
 '<subPartitions-2>': ['<SUBPARTITIONS_SYMBOL><real_ulong_number>'],
 '<partitionKeyAlgorithm>': ['<ALGORITHM_SYMBOL><EQUAL_OPERATOR><real_ulong_number>'],
 '<partitionDefinitions>': ['<OPEN_PAR_SYMBOL><partitionDefinition><partitionDefinitions-1>*<CLOSE_PAR_SYMBOL>'],
 '<partitionDefinitions-1>': ['<COMMA_SYMBOL><partitionDefinition>'],
 '<partitionDefinition>': ['<PARTITION_SYMBOL><identifier><partitionDefinition-1>?<partitionOption>*<partitionDefinition-3>?'],
 '<partitionDefinition-1>': ['<VALUES_SYMBOL><LESS_SYMBOL><THAN_SYMBOL><partitionDefinition-2>',
                             '<VALUES_SYMBOL><IN_SYMBOL><partitionValuesIn>'],
 '<partitionDefinition-2>': ['<partitionValueItemListParen>',
                             '<MAXVALUE_SYMBOL>'],
 '<partitionDefinition-3>': ['<OPEN_PAR_SYMBOL><subpartitionDefinition><partitionDefinition-4>*<CLOSE_PAR_SYMBOL>'],
 '<partitionDefinition-4>': ['<COMMA_SYMBOL><subpartitionDefinition>'],
 '<partitionValuesIn>': ['<partitionValueItemListParen>',
                         '<OPEN_PAR_SYMBOL><partitionValueItemListParen><partitionValuesIn-1>*<CLOSE_PAR_SYMBOL>'],
 '<partitionValuesIn-1>': ['<COMMA_SYMBOL><partitionValueItemListParen>'],
 '<partitionOption>': ['<TABLESPACE_SYMBOL><EQUAL_OPERATOR>?<identifier>',
                       '<STORAGE_SYMBOL>?<ENGINE_SYMBOL><EQUAL_OPERATOR>?<engineRef>',
                       '<NODEGROUP_SYMBOL><EQUAL_OPERATOR>?<real_ulong_number>',
                       '<partitionOption-1><EQUAL_OPERATOR>?<real_ulong_number>',
                       '<partitionOption-2><DIRECTORY_SYMBOL><EQUAL_OPERATOR>?<textLiteral>',
                       '<COMMENT_SYMBOL><EQUAL_OPERATOR>?<textLiteral>'],
 '<partitionOption-1>': ['<MAX_ROWS_SYMBOL>', '<MIN_ROWS_SYMBOL>'],
 '<partitionOption-2>': ['<DATA_SYMBOL>', '<INDEX_SYMBOL>'],
 '<subpartitionDefinition>': ['<SUBPARTITION_SYMBOL><textOrIdentifier><partitionOption>*'],
 '<partitionValueItemListParen>': ['<OPEN_PAR_SYMBOL><partitionValueItem><partitionValueItemListParen-1>*<CLOSE_PAR_SYMBOL>'],
 '<partitionValueItemListParen-1>': ['<COMMA_SYMBOL><partitionValueItem>'],
 '<partitionValueItem>': ['<bitExpr>', '<MAXVALUE_SYMBOL>'],
 '<definerClause>': ['<DEFINER_SYMBOL><EQUAL_OPERATOR><user>'],
 '<ifExists>': ['<IF_SYMBOL><EXISTS_SYMBOL>'],
 '<ifExistsIdentifier>': ['<ifExists><persistedVariableIdentifier>'],
 '<persistedVariableIdentifier>': ['<identifier>',
                                   '<persistedVariableIdentifier-1>'],
 '<persistedVariableIdentifier-1>': ['<qualifiedIdentifier>',
                                     '<DEFAULT_SYMBOL><dotIdentifier>'],
 '<ifNotExists>': ['<IF_SYMBOL><notRule><EXISTS_SYMBOL>'],
 '<ignoreUnknownUser>': ['<IGNORE_SYMBOL><UNKNOWN_SYMBOL><USER_SYMBOL>'],
 '<procedureParameter>': ['<procedureParameter-1>?<functionParameter>'],
 '<procedureParameter-1>': ['<IN_SYMBOL>', '<OUT_SYMBOL>', '<INOUT_SYMBOL>'],
 '<functionParameter>': ['<parameterName><typeWithOptCollate>'],
 '<collate>': ['<COLLATE_SYMBOL><collationName>'],
 '<typeWithOptCollate>': ['<dataType><collate>?'],
 '<schemaIdentifierPair>': ['<OPEN_PAR_SYMBOL><schemaRef><COMMA_SYMBOL><schemaRef><CLOSE_PAR_SYMBOL>'],
 '<viewRefList>': ['<viewRef><viewRefList-1>*'],
 '<viewRefList-1>': ['<COMMA_SYMBOL><viewRef>'],
 '<updateList>': ['<updateElement><updateList-1>*'],
 '<updateList-1>': ['<COMMA_SYMBOL><updateElement>'],
 '<updateElement>': ['<columnRef><EQUAL_OPERATOR><updateElement-1>'],
 '<updateElement-1>': ['<expr>', '<DEFAULT_SYMBOL>'],
 '<charsetClause>': ['<charset><charsetName>'],
 '<fieldsClause>': ['<COLUMNS_SYMBOL><fieldTerm>+'],
 '<fieldTerm>': ['<TERMINATED_SYMBOL><BY_SYMBOL><textString>',
                 '<OPTIONALLY_SYMBOL>?<ENCLOSED_SYMBOL><BY_SYMBOL><textString>',
                 '<ESCAPED_SYMBOL><BY_SYMBOL><textString>'],
 '<linesClause>': ['<LINES_SYMBOL><lineTerm>+'],
 '<lineTerm>': ['<lineTerm-1><BY_SYMBOL><textString>'],
 '<lineTerm-1>': ['<TERMINATED_SYMBOL>', '<STARTING_SYMBOL>'],
 '<userList>': ['<user><userList-1>*'],
 '<userList-1>': ['<COMMA_SYMBOL><user>'],
 '<createUserList>': ['<createUser><createUserList-1>*'],
 '<createUserList-1>': ['<COMMA_SYMBOL><createUser>'],
 '<createUser>': ['<user><createUser-1>?'],
 '<createUser-1>': ['<identification><createUserWithMfa>?',
                    '<identifiedWithPlugin><initialAuth>?',
                    '<createUserWithMfa>'],
 '<createUserWithMfa>': ['<AND_SYMBOL><identification><createUserWithMfa-1>?'],
 '<createUserWithMfa-1>': ['<AND_SYMBOL><identification>'],
 '<identification>': ['<identifiedByPassword>',
                      '<identifiedByRandomPassword>',
                      '<identifiedWithPlugin>',
                      '<identifiedWithPluginAsAuth>',
                      '<identifiedWithPluginByPassword>',
                      '<identifiedWithPluginByRandomPassword>'],
 '<identifiedByPassword>': ['<IDENTIFIED_SYMBOL><BY_SYMBOL><textStringLiteral>'],
 '<identifiedByRandomPassword>': ['<IDENTIFIED_SYMBOL><BY_SYMBOL><RANDOM_SYMBOL><PASSWORD_SYMBOL>'],
 '<identifiedWithPlugin>': ['<IDENTIFIED_SYMBOL><WITH_SYMBOL><textOrIdentifier>'],
 '<identifiedWithPluginAsAuth>': ['<IDENTIFIED_SYMBOL><WITH_SYMBOL><textOrIdentifier><AS_SYMBOL><textStringHash>'],
 '<identifiedWithPluginByPassword>': ['<IDENTIFIED_SYMBOL><WITH_SYMBOL><textOrIdentifier><BY_SYMBOL><textStringLiteral>'],
 '<identifiedWithPluginByRandomPassword>': ['<IDENTIFIED_SYMBOL><WITH_SYMBOL><textOrIdentifier><BY_SYMBOL><RANDOM_SYMBOL><PASSWORD_SYMBOL>'],
 '<initialAuth>': ['<INITIAL_SYMBOL><AUTHENTICATION_SYMBOL><initialAuth-1>'],
 '<initialAuth-1>': ['<identifiedByRandomPassword>',
                     '<identifiedWithPluginAsAuth>',
                     '<identifiedByPassword>'],
 '<retainCurrentPassword>': ['<RETAIN_SYMBOL><CURRENT_SYMBOL><PASSWORD_SYMBOL>'],
 '<discardOldPassword>': ['<DISCARD_SYMBOL><OLD_SYMBOL><PASSWORD_SYMBOL>'],
 '<userRegistration>': ['<factor><INITIATE_SYMBOL><REGISTRATION_SYMBOL>',
                        '<factor><UNREGISTER_SYMBOL>',
                        '<factor><FINISH_SYMBOL><REGISTRATION_SYMBOL><SET_SYMBOL><CHALLENGE_RESPONSE_SYMBOL><AS_SYMBOL><textStringHash>'],
 '<factor>': ['<numLiteral><FACTOR_SYMBOL>'],
 '<replacePassword>': ['<REPLACE_SYMBOL><textString>'],
 '<userIdentifierOrText>': ['<textOrIdentifier><userVariable>?'],
 '<user>': ['<userIdentifierOrText>', '<CURRENT_USER_SYMBOL><parentheses>?'],
 '<likeClause>': ['<LIKE_SYMBOL><textStringLiteral>'],
 '<likeOrWhere>': ['<likeClause>', '<whereClause>'],
 '<onlineOption>': ['<ONLINE_SYMBOL>', '<OFFLINE_SYMBOL>'],
 '<noWriteToBinLog>': ['<LOCAL_SYMBOL>', '<NO_WRITE_TO_BINLOG_SYMBOL>'],
 '<usePartition>': ['<PARTITION_SYMBOL><identifierListWithParentheses>'],
 '<fieldIdentifier>': ['<dotIdentifier>',
                       '<qualifiedIdentifier><dotIdentifier>?'],
 '<columnName>': ['<identifier>'],
 '<columnInternalRef>': ['<identifier>'],
 '<columnInternalRefList>': ['<OPEN_PAR_SYMBOL><columnInternalRef><columnInternalRefList-1>*<CLOSE_PAR_SYMBOL>'],
 '<columnInternalRefList-1>': ['<COMMA_SYMBOL><columnInternalRef>'],
 '<columnRef>': ['<fieldIdentifier>'],
 '<insertIdentifier>': ['<columnRef>', '<tableWild>'],
 '<indexName>': ['<identifier>'],
 '<indexRef>': ['<fieldIdentifier>'],
 '<tableWild>': ['<identifier><DOT_SYMBOL><tableWild-1>?<MULT_OPERATOR>'],
 '<tableWild-1>': ['<identifier><DOT_SYMBOL>'],
 '<schemaName>': ['<identifier>'],
 '<schemaRef>': ['<identifier>'],
 '<procedureName>': ['<qualifiedIdentifier>'],
 '<procedureRef>': ['<qualifiedIdentifier>'],
 '<functionName>': ['<qualifiedIdentifier>'],
 '<functionRef>': ['<qualifiedIdentifier>'],
 '<triggerName>': ['<qualifiedIdentifier>'],
 '<triggerRef>': ['<qualifiedIdentifier>'],
 '<viewName>': ['<qualifiedIdentifier>', '<dotIdentifier>'],
 '<viewRef>': ['<qualifiedIdentifier>', '<dotIdentifier>'],
 '<tablespaceName>': ['<identifier>'],
 '<tablespaceRef>': ['<identifier>'],
 '<logfileGroupName>': ['<identifier>'],
 '<logfileGroupRef>': ['<identifier>'],
 '<eventName>': ['<qualifiedIdentifier>'],
 '<eventRef>': ['<qualifiedIdentifier>'],
 '<udfName>': ['<identifier>'],
 '<serverName>': ['<textOrIdentifier>'],
 '<serverRef>': ['<textOrIdentifier>'],
 '<engineRef>': ['<textOrIdentifier>'],
 '<tableName>': ['<qualifiedIdentifier>', '<dotIdentifier>'],
 '<filterTableRef>': ['<schemaRef><dotIdentifier>'],
 '<tableRefWithWildcard>': ['<identifier><tableRefWithWildcard-1>?'],
 '<tableRefWithWildcard-1>': ['<DOT_SYMBOL><MULT_OPERATOR>',
                              '<dotIdentifier><tableRefWithWildcard-2>?'],
 '<tableRefWithWildcard-2>': ['<DOT_SYMBOL><MULT_OPERATOR>'],
 '<tableRef>': ['<qualifiedIdentifier>', '<dotIdentifier>'],
 '<tableRefList>': ['<tableRef><tableRefList-1>*'],
 '<tableRefList-1>': ['<COMMA_SYMBOL><tableRef>'],
 '<tableAliasRefList>': ['<tableRefWithWildcard><tableAliasRefList-1>*'],
 '<tableAliasRefList-1>': ['<COMMA_SYMBOL><tableRefWithWildcard>'],
 '<parameterName>': ['<identifier>'],
 '<labelIdentifier>': ['<pureIdentifier>', '<labelKeyword>'],
 '<labelRef>': ['<labelIdentifier>'],
 '<roleIdentifier>': ['<pureIdentifier>', '<roleKeyword>'],
 '<pluginRef>': ['<identifier>'],
 '<componentRef>': ['<textStringLiteral>'],
 '<resourceGroupRef>': ['<identifier>'],
 '<windowName>': ['<identifier>'],
 '<pureIdentifier>': ['<pureIdentifier-1>', '<DOUBLE_QUOTED_TEXT>'],
 '<pureIdentifier-1>': ['<IDENTIFIER>', '<BACK_TICK_QUOTED_ID>'],
 '<identifier>': ['<pureIdentifier>', '<identifierKeyword>'],
 '<identifierList>': ['<identifier><identifierList-1>*'],
 '<identifierList-1>': ['<COMMA_SYMBOL><identifier>'],
 '<identifierListWithParentheses>': ['<OPEN_PAR_SYMBOL><identifierList><CLOSE_PAR_SYMBOL>'],
 '<qualifiedIdentifier>': ['<identifier><dotIdentifier>?'],
 '<simpleIdentifier>': ['<identifier><simpleIdentifier-1>?'],
 '<simpleIdentifier-1>': ['<dotIdentifier><dotIdentifier>?'],
 '<dotIdentifier>': ['<DOT_SYMBOL><identifier>'],
 '<ulong_number>': ['<INT_NUMBER>',
                    '<HEX_NUMBER>',
                    'LONG_NUMBER',
                    'ULONGLONG_NUMBER',
                    '<DECIMAL_NUMBER>',
                    '<FLOAT_NUMBER>'],
 '<real_ulong_number>': ['<INT_NUMBER>',
                         '<HEX_NUMBER>',
                         'LONG_NUMBER',
                         'ULONGLONG_NUMBER'],
 '<ulonglongNumber>': ['<INT_NUMBER>',
                       'LONG_NUMBER',
                       'ULONGLONG_NUMBER',
                       '<DECIMAL_NUMBER>',
                       '<FLOAT_NUMBER>'],
 '<real_ulonglong_number>': ['<INT_NUMBER>',
                             '<HEX_NUMBER>',
                             'ULONGLONG_NUMBER',
                             'LONG_NUMBER'],
 '<signedLiteral>': ['<literal>',
                     '<PLUS_OPERATOR><ulong_number>',
                     '<MINUS_OPERATOR><ulong_number>'],
 '<signedLiteralOrNull>': ['<signedLiteral>', '<nullAsLiteral>'],
 '<literal>': ['<textLiteral>',
               '<numLiteral>',
               '<temporalLiteral>',
               '<nullLiteral>',
               '<boolLiteral>',
               '<UNDERSCORE_CHARSET>?<literal-1>'],
 '<literal-1>': ['<HEX_NUMBER>', '<BIN_NUMBER>'],
 '<literalOrNull>': ['<literal>', '<nullAsLiteral>'],
 '<nullAsLiteral>': ['<NULL_SYMBOL>'],
 '<stringList>': ['<OPEN_PAR_SYMBOL><textString><stringList-1>*<CLOSE_PAR_SYMBOL>'],
 '<stringList-1>': ['<COMMA_SYMBOL><textString>'],
 '<textStringLiteral>': ['<SINGLE_QUOTED_TEXT>', '<DOUBLE_QUOTED_TEXT>'],
 '<textString>': ['<textStringLiteral>', '<HEX_NUMBER>', '<BIN_NUMBER>'],
 '<textStringHash>': ['<textStringLiteral>', '<HEX_NUMBER>'],
 '<textLiteral>': ['<textLiteral-1><textStringLiteral>*'],
 '<textLiteral-1>': ['<UNDERSCORE_CHARSET>?<textStringLiteral>',
                     '<NCHAR_TEXT>'],
 '<textStringNoLinebreak>': ['<textStringLiteral>'],
 '<textStringLiteralList>': ['<textStringLiteral><textStringLiteralList-1>*'],
 '<textStringLiteralList-1>': ['<COMMA_SYMBOL><textStringLiteral>'],
 '<numLiteral>': ['<int64Literal>', '<DECIMAL_NUMBER>', '<FLOAT_NUMBER>'],
 '<boolLiteral>': ['<TRUE_SYMBOL>', '<FALSE_SYMBOL>'],
 '<nullLiteral>': ['<NULL_SYMBOL>', '<NULL2_SYMBOL>'],
 '<int64Literal>': ['<INT_NUMBER>', 'LONG_NUMBER', 'ULONGLONG_NUMBER'],
 '<temporalLiteral>': ['<DATE_SYMBOL><SINGLE_QUOTED_TEXT>',
                       '<TIME_SYMBOL><SINGLE_QUOTED_TEXT>',
                       '<TIMESTAMP_SYMBOL><SINGLE_QUOTED_TEXT>'],
 '<floatOptions>': ['<fieldLength>', '<precision>'],
 '<standardFloatOptions>': ['<precision>'],
 '<precision>': ['<OPEN_PAR_SYMBOL><INT_NUMBER><COMMA_SYMBOL><INT_NUMBER><CLOSE_PAR_SYMBOL>'],
 '<textOrIdentifier>': ['<identifier>', '<textStringLiteral>'],
 '<lValueIdentifier>': ['<pureIdentifier>', '<lValueKeyword>'],
 '<roleIdentifierOrText>': ['<roleIdentifier>', '<textStringLiteral>'],
 '<sizeNumber>': ['<real_ulonglong_number>', '<pureIdentifier>'],
 '<parentheses>': ['<OPEN_PAR_SYMBOL><CLOSE_PAR_SYMBOL>'],
 '<equal>': ['<EQUAL_OPERATOR>', '<ASSIGN_OPERATOR>'],
 '<optionType>': ['<PERSIST_SYMBOL>',
                  '<PERSIST_ONLY_SYMBOL>',
                  '<GLOBAL_SYMBOL>',
                  '<LOCAL_SYMBOL>',
                  '<SESSION_SYMBOL>'],
 '<rvalueSystemVariableType>': ['<GLOBAL_SYMBOL><DOT_SYMBOL>',
                                '<LOCAL_SYMBOL><DOT_SYMBOL>',
                                '<SESSION_SYMBOL><DOT_SYMBOL>'],
 '<setVarIdentType>': ['<setVarIdentType-1><DOT_SYMBOL>'],
 '<setVarIdentType-1>': ['<PERSIST_SYMBOL>',
                         '<PERSIST_ONLY_SYMBOL>',
                         '<GLOBAL_SYMBOL>',
                         '<LOCAL_SYMBOL>',
                         '<SESSION_SYMBOL>'],
 '<jsonAttribute>': ['<textStringLiteral>'],
 '<identifierKeyword>': ['<identifierKeyword-1>', '<identifierKeyword-2>'],
 '<identifierKeyword-1>': ['<labelKeyword>',
                           '<roleOrIdentifierKeyword>',
                           '<EXECUTE_SYMBOL>',
                           '<SHUTDOWN_SYMBOL>',
                           '<RESTART_SYMBOL>'],
 '<identifierKeyword-2>': ['<identifierKeywordsUnambiguous>',
                           '<identifierKeywordsAmbiguous1RolesAndLabels>',
                           '<identifierKeywordsAmbiguous2Labels>',
                           '<identifierKeywordsAmbiguous3Roles>',
                           '<identifierKeywordsAmbiguous4SystemVariables>'],
 '<identifierKeywordsAmbiguous1RolesAndLabels>': ['<EXECUTE_SYMBOL>',
                                                  '<RESTART_SYMBOL>',
                                                  '<SHUTDOWN_SYMBOL>'],
 '<identifierKeywordsAmbiguous2Labels>': ['<ASCII_SYMBOL>',
                                          '<BEGIN_SYMBOL>',
                                          '<BYTE_SYMBOL>',
                                          '<CACHE_SYMBOL>',
                                          '<CHARSET_SYMBOL>',
                                          '<CHECKSUM_SYMBOL>',
                                          '<CLONE_SYMBOL>',
                                          '<COMMENT_SYMBOL>',
                                          '<COMMIT_SYMBOL>',
                                          '<CONTAINS_SYMBOL>',
                                          '<DEALLOCATE_SYMBOL>',
                                          '<DO_SYMBOL>',
                                          '<END_SYMBOL>',
                                          '<FLUSH_SYMBOL>',
                                          '<FOLLOWS_SYMBOL>',
                                          '<HANDLER_SYMBOL>',
                                          '<HELP_SYMBOL>',
                                          '<IMPORT_SYMBOL>',
                                          '<INSTALL_SYMBOL>',
                                          '<LANGUAGE_SYMBOL>',
                                          '<NO_SYMBOL>',
                                          '<PRECEDES_SYMBOL>',
                                          '<PREPARE_SYMBOL>',
                                          '<REPAIR_SYMBOL>',
                                          '<RESET_SYMBOL>',
                                          '<ROLLBACK_SYMBOL>',
                                          '<SAVEPOINT_SYMBOL>',
                                          '<SIGNED_SYMBOL>',
                                          '<SLAVE_SYMBOL>',
                                          '<START_SYMBOL>',
                                          '<STOP_SYMBOL>',
                                          '<TRUNCATE_SYMBOL>',
                                          '<UNICODE_SYMBOL>',
                                          '<UNINSTALL_SYMBOL>',
                                          '<XA_SYMBOL>'],
 '<labelKeyword>': ['<labelKeyword-1>', '<labelKeyword-2>'],
 '<labelKeyword-1>': ['<roleOrLabelKeyword>',
                      '<EVENT_SYMBOL>',
                      '<FILE_SYMBOL>',
                      '<NONE_SYMBOL>',
                      '<PROCESS_SYMBOL>',
                      '<PROXY_SYMBOL>',
                      '<RELOAD_SYMBOL>',
                      '<REPLICATION_SYMBOL>',
                      '<RESOURCE_SYMBOL>',
                      '<SUPER_SYMBOL>'],
 '<labelKeyword-2>': ['<identifierKeywordsUnambiguous>',
                      '<identifierKeywordsAmbiguous3Roles>',
                      '<identifierKeywordsAmbiguous4SystemVariables>'],
 '<identifierKeywordsAmbiguous3Roles>': ['<EVENT_SYMBOL>',
                                         '<FILE_SYMBOL>',
                                         '<NONE_SYMBOL>',
                                         '<PROCESS_SYMBOL>',
                                         '<PROXY_SYMBOL>',
                                         '<RELOAD_SYMBOL>',
                                         '<REPLICATION_SYMBOL>',
                                         '<RESOURCE_SYMBOL>',
                                         '<SUPER_SYMBOL>'],
 '<identifierKeywordsUnambiguous>': ['<identifierKeywordsUnambiguous-1>',
                                     '<identifierKeywordsUnambiguous-2>',
                                     '<identifierKeywordsUnambiguous-3>'],
 '<identifierKeywordsUnambiguous-1>': ['<ACTION_SYMBOL>',
                                       '<ACCOUNT_SYMBOL>',
                                       '<ACTIVE_SYMBOL>',
                                       '<ADDDATE_SYMBOL>',
                                       '<ADMIN_SYMBOL>',
                                       '<AFTER_SYMBOL>',
                                       '<AGAINST_SYMBOL>',
                                       '<AGGREGATE_SYMBOL>',
                                       '<ALGORITHM_SYMBOL>',
                                       '<ALWAYS_SYMBOL>',
                                       '<ANY_SYMBOL>',
                                       '<AT_SYMBOL>',
                                       '<ATTRIBUTE_SYMBOL>',
                                       '<AUTHENTICATION_SYMBOL>',
                                       '<AUTOEXTEND_SIZE_SYMBOL>',
                                       '<AUTO_INCREMENT_SYMBOL>',
                                       '<AVG_ROW_LENGTH_SYMBOL>',
                                       '<AVG_SYMBOL>',
                                       '<BACKUP_SYMBOL>',
                                       '<BINLOG_SYMBOL>',
                                       '<BIT_SYMBOL>',
                                       '<BLOCK_SYMBOL>',
                                       '<BOOLEAN_SYMBOL>',
                                       '<BOOL_SYMBOL>',
                                       '<BTREE_SYMBOL>',
                                       '<BUCKETS_SYMBOL>',
                                       '<CASCADED_SYMBOL>',
                                       '<CATALOG_NAME_SYMBOL>',
                                       '<CHAIN_SYMBOL>',
                                       '<CHALLENGE_RESPONSE_SYMBOL>',
                                       '<CHANGED_SYMBOL>',
                                       '<CHANNEL_SYMBOL>',
                                       '<CIPHER_SYMBOL>',
                                       '<CLASS_ORIGIN_SYMBOL>',
                                       '<CLIENT_SYMBOL>',
                                       '<CLOSE_SYMBOL>',
                                       '<COALESCE_SYMBOL>',
                                       '<CODE_SYMBOL>',
                                       '<COLLATION_SYMBOL>',
                                       '<COLUMNS_SYMBOL>',
                                       '<COLUMN_FORMAT_SYMBOL>',
                                       '<COLUMN_NAME_SYMBOL>',
                                       '<COMMITTED_SYMBOL>',
                                       '<COMPACT_SYMBOL>',
                                       '<COMPLETION_SYMBOL>',
                                       '<COMPONENT_SYMBOL>',
                                       '<COMPRESSED_SYMBOL>',
                                       '<COMPRESSION_SYMBOL>',
                                       '<CONCURRENT_SYMBOL>',
                                       '<CONNECTION_SYMBOL>',
                                       '<CONSISTENT_SYMBOL>',
                                       '<CONSTRAINT_CATALOG_SYMBOL>',
                                       '<CONSTRAINT_NAME_SYMBOL>',
                                       '<CONSTRAINT_SCHEMA_SYMBOL>',
                                       '<CONTEXT_SYMBOL>',
                                       '<CPU_SYMBOL>',
                                       '<CURRENT_SYMBOL>',
                                       '<CURSOR_NAME_SYMBOL>',
                                       '<DATAFILE_SYMBOL>',
                                       '<DATA_SYMBOL>',
                                       '<DATETIME_SYMBOL>',
                                       '<DATE_SYMBOL>',
                                       '<DAY_SYMBOL>',
                                       '<DEFAULT_AUTH_SYMBOL>',
                                       '<DEFINER_SYMBOL>',
                                       '<DEFINITION_SYMBOL>',
                                       '<DELAY_KEY_WRITE_SYMBOL>',
                                       '<DESCRIPTION_SYMBOL>',
                                       '<DIAGNOSTICS_SYMBOL>',
                                       '<DIRECTORY_SYMBOL>',
                                       '<DISABLE_SYMBOL>',
                                       '<DISCARD_SYMBOL>',
                                       '<DISK_SYMBOL>',
                                       '<DUMPFILE_SYMBOL>',
                                       '<DUPLICATE_SYMBOL>',
                                       '<DYNAMIC_SYMBOL>',
                                       '<ENABLE_SYMBOL>',
                                       '<ENCRYPTION_SYMBOL>',
                                       '<ENDS_SYMBOL>',
                                       '<ENFORCED_SYMBOL>',
                                       '<ENGINES_SYMBOL>',
                                       '<ENGINE_SYMBOL>',
                                       '<ENUM_SYMBOL>',
                                       '<ERRORS_SYMBOL>',
                                       '<ERROR_SYMBOL>',
                                       '<ESCAPE_SYMBOL>',
                                       '<EVENTS_SYMBOL>',
                                       '<EVERY_SYMBOL>',
                                       '<EXCHANGE_SYMBOL>',
                                       '<EXCLUDE_SYMBOL>',
                                       '<EXPANSION_SYMBOL>',
                                       '<EXPIRE_SYMBOL>',
                                       '<EXPORT_SYMBOL>',
                                       '<EXTENDED_SYMBOL>',
                                       '<EXTENT_SIZE_SYMBOL>',
                                       '<FACTOR_SYMBOL>',
                                       '<FAST_SYMBOL>',
                                       '<FAULTS_SYMBOL>',
                                       '<FILE_BLOCK_SIZE_SYMBOL>',
                                       '<FILTER_SYMBOL>',
                                       '<FINISH_SYMBOL>',
                                       '<FIRST_SYMBOL>',
                                       '<FIXED_SYMBOL>',
                                       '<FOLLOWING_SYMBOL>',
                                       '<FORMAT_SYMBOL>',
                                       '<FOUND_SYMBOL>',
                                       '<FULL_SYMBOL>',
                                       '<GENERAL_SYMBOL>',
                                       '<GEOMETRYCOLLECTION_SYMBOL>',
                                       '<GEOMETRY_SYMBOL>',
                                       '<GET_FORMAT_SYMBOL>',
                                       '<GET_MASTER_PUBLIC_KEY_SYMBOL>',
                                       '<GET_SOURCE_PUBLIC_KEY_SYMBOL>',
                                       '<GRANTS_SYMBOL>',
                                       '<GROUP_REPLICATION_SYMBOL>',
                                       '<GTID_ONLY_SYMBOL>',
                                       '<HASH_SYMBOL>',
                                       '<HISTOGRAM_SYMBOL>',
                                       '<HISTORY_SYMBOL>',
                                       '<HOSTS_SYMBOL>',
                                       '<HOST_SYMBOL>',
                                       '<HOUR_SYMBOL>',
                                       '<IDENTIFIED_SYMBOL>',
                                       '<IGNORE_SERVER_IDS_SYMBOL>',
                                       '<INACTIVE_SYMBOL>',
                                       '<INDEXES_SYMBOL>',
                                       '<INITIAL_SIZE_SYMBOL>',
                                       '<INITIAL_SYMBOL>',
                                       '<INITIATE_SYMBOL>',
                                       '<INSERT_METHOD_SYMBOL>',
                                       '<INSTANCE_SYMBOL>',
                                       '<INVISIBLE_SYMBOL>',
                                       '<INVOKER_SYMBOL>',
                                       '<IO_SYMBOL>',
                                       '<IPC_SYMBOL>',
                                       '<ISOLATION_SYMBOL>',
                                       '<ISSUER_SYMBOL>',
                                       '<JSON_SYMBOL>',
                                       '<JSON_VALUE_SYMBOL>',
                                       '<KEY_BLOCK_SIZE_SYMBOL>',
                                       '<KEYRING_SYMBOL>',
                                       '<LAST_SYMBOL>',
                                       '<LEAVES_SYMBOL>',
                                       '<LESS_SYMBOL>',
                                       '<LEVEL_SYMBOL>',
                                       '<LINESTRING_SYMBOL>',
                                       '<LIST_SYMBOL>',
                                       '<LOCKED_SYMBOL>',
                                       '<LOCKS_SYMBOL>',
                                       '<LOGFILE_SYMBOL>',
                                       '<LOGS_SYMBOL>',
                                       '<MASTER_AUTO_POSITION_SYMBOL>',
                                       '<MASTER_COMPRESSION_ALGORITHM_SYMBOL>',
                                       '<MASTER_CONNECT_RETRY_SYMBOL>',
                                       '<MASTER_DELAY_SYMBOL>',
                                       '<MASTER_HEARTBEAT_PERIOD_SYMBOL>',
                                       '<MASTER_HOST_SYMBOL>',
                                       '<NETWORK_NAMESPACE_SYMBOL>',
                                       '<MASTER_LOG_FILE_SYMBOL>',
                                       '<MASTER_LOG_POS_SYMBOL>',
                                       '<MASTER_PASSWORD_SYMBOL>',
                                       '<MASTER_PORT_SYMBOL>',
                                       '<MASTER_PUBLIC_KEY_PATH_SYMBOL>',
                                       '<MASTER_RETRY_COUNT_SYMBOL>',
                                       '<MASTER_SSL_CAPATH_SYMBOL>',
                                       '<MASTER_SSL_CA_SYMBOL>',
                                       '<MASTER_SSL_CERT_SYMBOL>',
                                       '<MASTER_SSL_CIPHER_SYMBOL>',
                                       '<MASTER_SSL_CRLPATH_SYMBOL>',
                                       '<MASTER_SSL_CRL_SYMBOL>',
                                       '<MASTER_SSL_KEY_SYMBOL>',
                                       '<MASTER_SSL_SYMBOL>',
                                       '<MASTER_SYMBOL>',
                                       '<MASTER_TLS_CIPHERSUITES_SYMBOL>',
                                       '<MASTER_TLS_VERSION_SYMBOL>',
                                       '<MASTER_USER_SYMBOL>',
                                       '<MASTER_ZSTD_COMPRESSION_LEVEL_SYMBOL>',
                                       '<MAX_CONNECTIONS_PER_HOUR_SYMBOL>',
                                       '<MAX_QUERIES_PER_HOUR_SYMBOL>',
                                       '<MAX_ROWS_SYMBOL>',
                                       '<MAX_SIZE_SYMBOL>',
                                       '<MAX_UPDATES_PER_HOUR_SYMBOL>',
                                       '<MAX_USER_CONNECTIONS_SYMBOL>',
                                       '<MEDIUM_SYMBOL>',
                                       '<MEMORY_SYMBOL>',
                                       '<MERGE_SYMBOL>',
                                       '<MESSAGE_TEXT_SYMBOL>',
                                       '<MICROSECOND_SYMBOL>',
                                       '<MIGRATE_SYMBOL>',
                                       '<MINUTE_SYMBOL>',
                                       '<MIN_ROWS_SYMBOL>',
                                       '<MODE_SYMBOL>',
                                       '<MODIFY_SYMBOL>',
                                       '<MONTH_SYMBOL>',
                                       '<MULTILINESTRING_SYMBOL>',
                                       '<MULTIPOINT_SYMBOL>',
                                       '<MULTIPOLYGON_SYMBOL>',
                                       '<MUTEX_SYMBOL>',
                                       '<MYSQL_ERRNO_SYMBOL>',
                                       '<NAMES_SYMBOL>',
                                       '<NAME_SYMBOL>',
                                       '<NATIONAL_SYMBOL>',
                                       '<NCHAR_SYMBOL>',
                                       '<NDBCLUSTER_SYMBOL>',
                                       '<NESTED_SYMBOL>',
                                       '<NEVER_SYMBOL>',
                                       '<NEW_SYMBOL>',
                                       '<NEXT_SYMBOL>',
                                       '<NODEGROUP_SYMBOL>',
                                       '<NOWAIT_SYMBOL>',
                                       '<NO_WAIT_SYMBOL>',
                                       '<NULLS_SYMBOL>',
                                       '<NUMBER_SYMBOL>',
                                       '<NVARCHAR_SYMBOL>',
                                       '<OFFSET_SYMBOL>',
                                       '<OJ_SYMBOL>',
                                       '<OLD_SYMBOL>',
                                       '<ONE_SYMBOL>',
                                       '<ONLY_SYMBOL>',
                                       '<OPEN_SYMBOL>',
                                       '<OPTIONAL_SYMBOL>',
                                       '<OPTIONS_SYMBOL>',
                                       '<ORDINALITY_SYMBOL>',
                                       '<ORGANIZATION_SYMBOL>',
                                       '<OTHERS_SYMBOL>',
                                       '<OWNER_SYMBOL>',
                                       '<PACK_KEYS_SYMBOL>',
                                       '<PAGE_SYMBOL>',
                                       '<PARSER_SYMBOL>',
                                       '<PARTIAL_SYMBOL>',
                                       '<PARTITIONING_SYMBOL>',
                                       '<PARTITIONS_SYMBOL>',
                                       '<PASSWORD_SYMBOL>',
                                       '<PATH_SYMBOL>',
                                       '<PHASE_SYMBOL>',
                                       '<PLUGINS_SYMBOL>',
                                       '<PLUGIN_DIR_SYMBOL>',
                                       '<PLUGIN_SYMBOL>',
                                       '<POINT_SYMBOL>',
                                       '<POLYGON_SYMBOL>',
                                       '<PORT_SYMBOL>',
                                       '<PRECEDING_SYMBOL>',
                                       '<PRESERVE_SYMBOL>',
                                       '<PREV_SYMBOL>',
                                       '<PRIVILEGES_SYMBOL>',
                                       '<PRIVILEGE_CHECKS_USER_SYMBOL>',
                                       '<PROCESSLIST_SYMBOL>',
                                       '<PROFILES_SYMBOL>',
                                       '<PROFILE_SYMBOL>',
                                       '<QUARTER_SYMBOL>',
                                       '<QUERY_SYMBOL>',
                                       '<QUICK_SYMBOL>',
                                       '<READ_ONLY_SYMBOL>',
                                       '<REBUILD_SYMBOL>',
                                       '<RECOVER_SYMBOL>',
                                       '<REDO_BUFFER_SIZE_SYMBOL>',
                                       '<REDUNDANT_SYMBOL>',
                                       '<REFERENCE_SYMBOL>',
                                       '<REGISTRATION_SYMBOL>',
                                       '<RELAY_SYMBOL>',
                                       '<RELAYLOG_SYMBOL>',
                                       '<RELAY_LOG_FILE_SYMBOL>',
                                       '<RELAY_LOG_POS_SYMBOL>',
                                       '<RELAY_THREAD_SYMBOL>',
                                       '<REMOVE_SYMBOL>',
                                       '<ASSIGN_GTIDS_TO_ANONYMOUS_TRANSACTIONS_SYMBOL>',
                                       '<REORGANIZE_SYMBOL>',
                                       '<REPEATABLE_SYMBOL>',
                                       '<REPLICAS_SYMBOL>',
                                       '<REPLICATE_DO_DB_SYMBOL>',
                                       '<REPLICATE_DO_TABLE_SYMBOL>',
                                       '<REPLICATE_IGNORE_DB_SYMBOL>',
                                       '<REPLICATE_IGNORE_TABLE_SYMBOL>',
                                       '<REPLICATE_REWRITE_DB_SYMBOL>',
                                       '<REPLICATE_WILD_DO_TABLE_SYMBOL>',
                                       '<REPLICATE_WILD_IGNORE_TABLE_SYMBOL>',
                                       '<REPLICA_SYMBOL>',
                                       '<USER_RESOURCES_SYMBOL>',
                                       '<RESPECT_SYMBOL>',
                                       '<RESTORE_SYMBOL>',
                                       '<RESUME_SYMBOL>',
                                       '<RETAIN_SYMBOL>',
                                       '<RETURNED_SQLSTATE_SYMBOL>',
                                       '<RETURNING_SYMBOL>',
                                       '<RETURNS_SYMBOL>',
                                       '<REUSE_SYMBOL>',
                                       '<REVERSE_SYMBOL>',
                                       '<ROLE_SYMBOL>',
                                       '<ROLLUP_SYMBOL>',
                                       '<ROTATE_SYMBOL>',
                                       '<ROUTINE_SYMBOL>',
                                       '<ROW_COUNT_SYMBOL>',
                                       '<ROW_FORMAT_SYMBOL>',
                                       '<RTREE_SYMBOL>',
                                       '<SCHEDULE_SYMBOL>',
                                       '<SCHEMA_NAME_SYMBOL>',
                                       '<SECONDARY_ENGINE_SYMBOL>',
                                       '<SECONDARY_ENGINE_ATTRIBUTE_SYMBOL>',
                                       '<SECONDARY_LOAD_SYMBOL>',
                                       '<SECONDARY_SYMBOL>',
                                       '<SECONDARY_UNLOAD_SYMBOL>',
                                       '<SECOND_SYMBOL>',
                                       '<SECURITY_SYMBOL>',
                                       '<SERIALIZABLE_SYMBOL>',
                                       '<SERIAL_SYMBOL>',
                                       '<SERVER_SYMBOL>',
                                       '<SHARE_SYMBOL>',
                                       '<SIMPLE_SYMBOL>',
                                       '<SKIP_SYMBOL>',
                                       '<SLOW_SYMBOL>',
                                       '<SNAPSHOT_SYMBOL>',
                                       '<SOCKET_SYMBOL>',
                                       '<SONAME_SYMBOL>',
                                       '<SOUNDS_SYMBOL>',
                                       '<SOURCE_AUTO_POSITION_SYMBOL>',
                                       '<SOURCE_BIND_SYMBOL>',
                                       '<SOURCE_COMPRESSION_ALGORITHM_SYMBOL>',
                                       '<SOURCE_CONNECTION_AUTO_FAILOVER_SYMBOL>',
                                       '<SOURCE_CONNECT_RETRY_SYMBOL>',
                                       '<SOURCE_DELAY_SYMBOL>',
                                       '<SOURCE_HEARTBEAT_PERIOD_SYMBOL>',
                                       '<SOURCE_HOST_SYMBOL>',
                                       '<SOURCE_LOG_FILE_SYMBOL>',
                                       '<SOURCE_LOG_POS_SYMBOL>',
                                       '<SOURCE_PASSWORD_SYMBOL>',
                                       '<SOURCE_PORT_SYMBOL>',
                                       '<SOURCE_PUBLIC_KEY_PATH_SYMBOL>',
                                       '<SOURCE_RETRY_COUNT_SYMBOL>',
                                       '<SOURCE_SSL_CAPATH_SYMBOL>',
                                       '<SOURCE_SSL_CA_SYMBOL>',
                                       '<SOURCE_SSL_CERT_SYMBOL>',
                                       '<SOURCE_SSL_CIPHER_SYMBOL>',
                                       '<SOURCE_SSL_CRLPATH_SYMBOL>',
                                       '<SOURCE_SSL_CRL_SYMBOL>',
                                       '<SOURCE_SSL_KEY_SYMBOL>',
                                       '<SOURCE_SSL_SYMBOL>',
                                       '<SOURCE_SSL_VERIFY_SERVER_CERT_SYMBOL>',
                                       '<SOURCE_SYMBOL>',
                                       '<SOURCE_TLS_CIPHERSUITES_SYMBOL>',
                                       '<SOURCE_TLS_VERSION_SYMBOL>',
                                       '<SOURCE_USER_SYMBOL>',
                                       '<SOURCE_ZSTD_COMPRESSION_LEVEL_SYMBOL>',
                                       '<SQL_AFTER_GTIDS_SYMBOL>',
                                       '<SQL_AFTER_MTS_GAPS_SYMBOL>',
                                       '<SQL_BEFORE_GTIDS_SYMBOL>',
                                       '<SQL_BUFFER_RESULT_SYMBOL>',
                                       '<SQL_NO_CACHE_SYMBOL>',
                                       '<SQL_THREAD_SYMBOL>',
                                       '<SRID_SYMBOL>',
                                       '<STACKED_SYMBOL>',
                                       '<STARTS_SYMBOL>',
                                       '<STATS_AUTO_RECALC_SYMBOL>',
                                       '<STATS_PERSISTENT_SYMBOL>',
                                       '<STATS_SAMPLE_PAGES_SYMBOL>',
                                       '<STATUS_SYMBOL>',
                                       '<STORAGE_SYMBOL>',
                                       '<STRING_SYMBOL>',
                                       '<ST_COLLECT_SYMBOL>',
                                       '<SUBCLASS_ORIGIN_SYMBOL>',
                                       '<SUBDATE_SYMBOL>',
                                       '<SUBJECT_SYMBOL>',
                                       '<SUBPARTITIONS_SYMBOL>',
                                       '<SUBPARTITION_SYMBOL>',
                                       '<SUSPEND_SYMBOL>',
                                       '<SWAPS_SYMBOL>',
                                       '<SWITCHES_SYMBOL>',
                                       '<TABLES_SYMBOL>',
                                       '<TABLESPACE_SYMBOL>',
                                       '<TABLE_CHECKSUM_SYMBOL>',
                                       '<TABLE_NAME_SYMBOL>',
                                       '<TEMPORARY_SYMBOL>',
                                       '<TEMPTABLE_SYMBOL>',
                                       '<TEXT_SYMBOL>',
                                       '<THAN_SYMBOL>',
                                       '<THREAD_PRIORITY_SYMBOL>',
                                       '<TIES_SYMBOL>',
                                       '<TIMESTAMPADD_SYMBOL>',
                                       '<TIMESTAMPDIFF_SYMBOL>',
                                       '<TIMESTAMP_SYMBOL>',
                                       '<TIME_SYMBOL>',
                                       '<TLS_SYMBOL>',
                                       '<TRANSACTION_SYMBOL>',
                                       '<TRIGGERS_SYMBOL>',
                                       '<TYPES_SYMBOL>',
                                       '<TYPE_SYMBOL>',
                                       '<UNBOUNDED_SYMBOL>',
                                       '<UNCOMMITTED_SYMBOL>',
                                       '<UNDEFINED_SYMBOL>',
                                       '<UNDOFILE_SYMBOL>',
                                       '<UNDO_BUFFER_SIZE_SYMBOL>',
                                       '<UNKNOWN_SYMBOL>',
                                       '<UNREGISTER_SYMBOL>',
                                       '<UNTIL_SYMBOL>',
                                       '<UPGRADE_SYMBOL>',
                                       '<USER_SYMBOL>',
                                       '<USE_FRM_SYMBOL>',
                                       '<VALIDATION_SYMBOL>',
                                       '<VALUE_SYMBOL>',
                                       '<VARIABLES_SYMBOL>',
                                       '<VCPU_SYMBOL>',
                                       '<VIEW_SYMBOL>',
                                       '<VISIBLE_SYMBOL>',
                                       '<WAIT_SYMBOL>',
                                       '<WARNINGS_SYMBOL>',
                                       '<WEEK_SYMBOL>',
                                       '<WEIGHT_STRING_SYMBOL>',
                                       '<WITHOUT_SYMBOL>',
                                       '<WORK_SYMBOL>',
                                       '<WRAPPER_SYMBOL>',
                                       '<X509_SYMBOL>',
                                       '<XID_SYMBOL>',
                                       '<XML_SYMBOL>',
                                       '<YEAR_SYMBOL>',
                                       '<ZONE_SYMBOL>'],
 '<identifierKeywordsUnambiguous-2>': ['<ARRAY_SYMBOL>',
                                       '<FAILED_LOGIN_ATTEMPTS_SYMBOL>',
                                       '<MASTER_COMPRESSION_ALGORITHM_SYMBOL>',
                                       '<MASTER_TLS_CIPHERSUITES_SYMBOL>',
                                       '<MASTER_ZSTD_COMPRESSION_LEVEL_SYMBOL>',
                                       '<MEMBER_SYMBOL>',
                                       '<OFF_SYMBOL>',
                                       '<PASSWORD_LOCK_TIME_SYMBOL>',
                                       '<PRIVILEGE_CHECKS_USER_SYMBOL>',
                                       '<RANDOM_SYMBOL>',
                                       '<REQUIRE_ROW_FORMAT_SYMBOL>',
                                       '<REQUIRE_TABLE_PRIMARY_KEY_CHECK_SYMBOL>',
                                       '<STREAM_SYMBOL>',
                                       '<TIMESTAMP_SYMBOL>',
                                       '<TIME_SYMBOL>'],
 '<identifierKeywordsUnambiguous-3>': ['<BULK_SYMBOL>',
                                       '<GENERATE_SYMBOL>',
                                       '<GTIDS_SYMBOL>',
                                       '<LOG_SYMBOL>',
                                       '<PARSE_TREE_SYMBOL>',
                                       '<S3_SYMBOL>',
                                       '<BERNOULLI_SYMBOL>'],
 '<roleKeyword>': ['<roleKeyword-1>', '<roleKeyword-2>'],
 '<roleKeyword-1>': ['<roleOrLabelKeyword>', '<roleOrIdentifierKeyword>'],
 '<roleKeyword-2>': ['<identifierKeywordsUnambiguous>',
                     '<identifierKeywordsAmbiguous2Labels>',
                     '<identifierKeywordsAmbiguous4SystemVariables>'],
 '<lValueKeyword>': ['<identifierKeywordsUnambiguous>',
                     '<identifierKeywordsAmbiguous1RolesAndLabels>',
                     '<identifierKeywordsAmbiguous2Labels>',
                     '<identifierKeywordsAmbiguous3Roles>'],
 '<identifierKeywordsAmbiguous4SystemVariables>': ['<GLOBAL_SYMBOL>',
                                                   '<LOCAL_SYMBOL>',
                                                   '<PERSIST_SYMBOL>',
                                                   '<PERSIST_ONLY_SYMBOL>',
                                                   '<SESSION_SYMBOL>'],
 '<roleOrIdentifierKeyword>': ['<roleOrIdentifierKeyword-1>'],
 '<roleOrIdentifierKeyword-1>': ['<ACCOUNT_SYMBOL>',
                                 '<ASCII_SYMBOL>',
                                 '<ALWAYS_SYMBOL>',
                                 '<BACKUP_SYMBOL>',
                                 '<BEGIN_SYMBOL>',
                                 '<BYTE_SYMBOL>',
                                 '<CACHE_SYMBOL>',
                                 '<CHARSET_SYMBOL>',
                                 '<CHECKSUM_SYMBOL>',
                                 '<CLONE_SYMBOL>',
                                 '<CLOSE_SYMBOL>',
                                 '<COMMENT_SYMBOL>',
                                 '<COMMIT_SYMBOL>',
                                 '<CONTAINS_SYMBOL>',
                                 '<DEALLOCATE_SYMBOL>',
                                 '<DO_SYMBOL>',
                                 '<END_SYMBOL>',
                                 '<FLUSH_SYMBOL>',
                                 '<FOLLOWS_SYMBOL>',
                                 '<FORMAT_SYMBOL>',
                                 '<GROUP_REPLICATION_SYMBOL>',
                                 '<HANDLER_SYMBOL>',
                                 '<HELP_SYMBOL>',
                                 '<HOST_SYMBOL>',
                                 '<INSTALL_SYMBOL>',
                                 '<INVISIBLE_SYMBOL>',
                                 '<LANGUAGE_SYMBOL>',
                                 '<NO_SYMBOL>',
                                 '<OPEN_SYMBOL>',
                                 '<OPTIONS_SYMBOL>',
                                 '<OWNER_SYMBOL>',
                                 '<PARSER_SYMBOL>',
                                 '<PARTITION_SYMBOL>',
                                 '<PORT_SYMBOL>',
                                 '<PRECEDES_SYMBOL>',
                                 '<PREPARE_SYMBOL>',
                                 '<REMOVE_SYMBOL>',
                                 '<REPAIR_SYMBOL>',
                                 '<RESET_SYMBOL>',
                                 '<RESTORE_SYMBOL>',
                                 '<ROLE_SYMBOL>',
                                 '<ROLLBACK_SYMBOL>',
                                 '<SAVEPOINT_SYMBOL>',
                                 '<SECONDARY_SYMBOL>',
                                 '<SECONDARY_ENGINE_SYMBOL>',
                                 '<SECONDARY_LOAD_SYMBOL>',
                                 '<SECONDARY_UNLOAD_SYMBOL>',
                                 '<SECURITY_SYMBOL>',
                                 '<SERVER_SYMBOL>',
                                 '<SIGNED_SYMBOL>',
                                 '<SOCKET_SYMBOL>',
                                 '<SLAVE_SYMBOL>',
                                 '<SONAME_SYMBOL>',
                                 '<START_SYMBOL>',
                                 '<STOP_SYMBOL>',
                                 '<TRUNCATE_SYMBOL>',
                                 '<UNICODE_SYMBOL>',
                                 '<UNINSTALL_SYMBOL>',
                                 '<UPGRADE_SYMBOL>',
                                 '<VISIBLE_SYMBOL>',
                                 '<WRAPPER_SYMBOL>',
                                 '<XA_SYMBOL>'],
 '<roleOrLabelKeyword>': ['<roleOrLabelKeyword-1>', '<ADMIN_SYMBOL>'],
 '<roleOrLabelKeyword-1>': ['<ACTION_SYMBOL>',
                            '<ACTIVE_SYMBOL>',
                            '<ADDDATE_SYMBOL>',
                            '<AFTER_SYMBOL>',
                            '<AGAINST_SYMBOL>',
                            '<AGGREGATE_SYMBOL>',
                            '<ALGORITHM_SYMBOL>',
                            '<ANY_SYMBOL>',
                            '<AT_SYMBOL>',
                            '<AUTO_INCREMENT_SYMBOL>',
                            '<AUTOEXTEND_SIZE_SYMBOL>',
                            '<AVG_ROW_LENGTH_SYMBOL>',
                            '<AVG_SYMBOL>',
                            '<BINLOG_SYMBOL>',
                            '<BIT_SYMBOL>',
                            '<BLOCK_SYMBOL>',
                            '<BOOL_SYMBOL>',
                            '<BOOLEAN_SYMBOL>',
                            '<BTREE_SYMBOL>',
                            '<BUCKETS_SYMBOL>',
                            '<CASCADED_SYMBOL>',
                            '<CATALOG_NAME_SYMBOL>',
                            '<CHAIN_SYMBOL>',
                            '<CHANGED_SYMBOL>',
                            '<CHANNEL_SYMBOL>',
                            '<CIPHER_SYMBOL>',
                            '<CLIENT_SYMBOL>',
                            '<CLASS_ORIGIN_SYMBOL>',
                            '<COALESCE_SYMBOL>',
                            '<CODE_SYMBOL>',
                            '<COLLATION_SYMBOL>',
                            '<COLUMN_NAME_SYMBOL>',
                            '<COLUMN_FORMAT_SYMBOL>',
                            '<COLUMNS_SYMBOL>',
                            '<COMMITTED_SYMBOL>',
                            '<COMPACT_SYMBOL>',
                            '<COMPLETION_SYMBOL>',
                            '<COMPONENT_SYMBOL>',
                            '<COMPRESSED_SYMBOL>',
                            '<COMPRESSION_SYMBOL>',
                            '<CONCURRENT_SYMBOL>',
                            '<CONNECTION_SYMBOL>',
                            '<CONSISTENT_SYMBOL>',
                            '<CONSTRAINT_CATALOG_SYMBOL>',
                            '<CONSTRAINT_SCHEMA_SYMBOL>',
                            '<CONSTRAINT_NAME_SYMBOL>',
                            '<CONTEXT_SYMBOL>',
                            '<CPU_SYMBOL>',
                            '<CURRENT_SYMBOL>',
                            '<CURSOR_NAME_SYMBOL>',
                            '<DATA_SYMBOL>',
                            '<DATAFILE_SYMBOL>',
                            '<DATETIME_SYMBOL>',
                            '<DATE_SYMBOL>',
                            '<DAY_SYMBOL>',
                            '<DEFAULT_AUTH_SYMBOL>',
                            '<DEFINER_SYMBOL>',
                            '<DELAY_KEY_WRITE_SYMBOL>',
                            '<DESCRIPTION_SYMBOL>',
                            '<DIAGNOSTICS_SYMBOL>',
                            '<DIRECTORY_SYMBOL>',
                            '<DISABLE_SYMBOL>',
                            '<DISCARD_SYMBOL>',
                            '<DISK_SYMBOL>',
                            '<DUMPFILE_SYMBOL>',
                            '<DUPLICATE_SYMBOL>',
                            '<DYNAMIC_SYMBOL>',
                            '<ENCRYPTION_SYMBOL>',
                            '<ENDS_SYMBOL>',
                            '<ENUM_SYMBOL>',
                            '<ENGINE_SYMBOL>',
                            '<ENGINES_SYMBOL>',
                            '<ENGINE_ATTRIBUTE_SYMBOL>',
                            '<ERROR_SYMBOL>',
                            '<ERRORS_SYMBOL>',
                            '<ESCAPE_SYMBOL>',
                            '<EVENTS_SYMBOL>',
                            '<EVERY_SYMBOL>',
                            '<EXCLUDE_SYMBOL>',
                            '<EXPANSION_SYMBOL>',
                            '<EXPORT_SYMBOL>',
                            '<EXTENDED_SYMBOL>',
                            '<EXTENT_SIZE_SYMBOL>',
                            '<FAULTS_SYMBOL>',
                            '<FAST_SYMBOL>',
                            '<FOLLOWING_SYMBOL>',
                            '<FOUND_SYMBOL>',
                            '<ENABLE_SYMBOL>',
                            '<FULL_SYMBOL>',
                            '<FILE_BLOCK_SIZE_SYMBOL>',
                            '<FILTER_SYMBOL>',
                            '<FIRST_SYMBOL>',
                            '<FIXED_SYMBOL>',
                            '<GENERAL_SYMBOL>',
                            '<GEOMETRY_SYMBOL>',
                            '<GEOMETRYCOLLECTION_SYMBOL>',
                            '<GET_FORMAT_SYMBOL>',
                            '<GRANTS_SYMBOL>',
                            '<GLOBAL_SYMBOL>',
                            '<HASH_SYMBOL>',
                            '<HISTOGRAM_SYMBOL>',
                            '<HISTORY_SYMBOL>',
                            '<HOSTS_SYMBOL>',
                            '<HOUR_SYMBOL>',
                            '<IDENTIFIED_SYMBOL>',
                            '<IGNORE_SERVER_IDS_SYMBOL>',
                            '<INVOKER_SYMBOL>',
                            '<INDEXES_SYMBOL>',
                            '<INITIAL_SIZE_SYMBOL>',
                            '<INSTANCE_SYMBOL>',
                            '<INACTIVE_SYMBOL>',
                            '<IO_SYMBOL>',
                            '<IPC_SYMBOL>',
                            '<ISOLATION_SYMBOL>',
                            '<ISSUER_SYMBOL>',
                            '<INSERT_METHOD_SYMBOL>',
                            '<JSON_SYMBOL>',
                            '<KEY_BLOCK_SIZE_SYMBOL>',
                            '<LAST_SYMBOL>',
                            '<LEAVES_SYMBOL>',
                            '<LESS_SYMBOL>',
                            '<LEVEL_SYMBOL>',
                            '<LINESTRING_SYMBOL>',
                            '<LIST_SYMBOL>',
                            '<LOCAL_SYMBOL>',
                            '<LOCKED_SYMBOL>',
                            '<LOCKS_SYMBOL>',
                            '<LOGFILE_SYMBOL>',
                            '<LOGS_SYMBOL>',
                            '<MAX_ROWS_SYMBOL>',
                            '<MASTER_SYMBOL>',
                            '<MASTER_HEARTBEAT_PERIOD_SYMBOL>',
                            '<MASTER_HOST_SYMBOL>',
                            '<MASTER_PORT_SYMBOL>',
                            '<MASTER_LOG_FILE_SYMBOL>',
                            '<MASTER_LOG_POS_SYMBOL>',
                            '<MASTER_USER_SYMBOL>',
                            '<MASTER_PASSWORD_SYMBOL>',
                            '<MASTER_PUBLIC_KEY_PATH_SYMBOL>',
                            '<MASTER_CONNECT_RETRY_SYMBOL>',
                            '<MASTER_RETRY_COUNT_SYMBOL>',
                            '<MASTER_DELAY_SYMBOL>',
                            '<MASTER_SSL_SYMBOL>',
                            '<MASTER_SSL_CA_SYMBOL>',
                            '<MASTER_SSL_CAPATH_SYMBOL>',
                            '<MASTER_TLS_VERSION_SYMBOL>',
                            '<MASTER_SSL_CERT_SYMBOL>',
                            '<MASTER_SSL_CIPHER_SYMBOL>',
                            '<MASTER_SSL_CRL_SYMBOL>',
                            '<MASTER_SSL_CRLPATH_SYMBOL>',
                            '<MASTER_SSL_KEY_SYMBOL>',
                            '<MASTER_AUTO_POSITION_SYMBOL>',
                            '<MAX_CONNECTIONS_PER_HOUR_SYMBOL>',
                            '<MAX_QUERIES_PER_HOUR_SYMBOL>',
                            '<MAX_SIZE_SYMBOL>',
                            '<MAX_UPDATES_PER_HOUR_SYMBOL>',
                            '<MAX_USER_CONNECTIONS_SYMBOL>',
                            '<MEDIUM_SYMBOL>',
                            '<MEMORY_SYMBOL>',
                            '<MERGE_SYMBOL>',
                            '<MESSAGE_TEXT_SYMBOL>',
                            '<MICROSECOND_SYMBOL>',
                            '<MIGRATE_SYMBOL>',
                            '<MINUTE_SYMBOL>',
                            '<MIN_ROWS_SYMBOL>',
                            '<MODIFY_SYMBOL>',
                            '<MODE_SYMBOL>',
                            '<MONTH_SYMBOL>',
                            '<MULTILINESTRING_SYMBOL>',
                            '<MULTIPOINT_SYMBOL>',
                            '<MULTIPOLYGON_SYMBOL>',
                            '<MUTEX_SYMBOL>',
                            '<MYSQL_ERRNO_SYMBOL>',
                            '<NAME_SYMBOL>',
                            '<NAMES_SYMBOL>',
                            '<NATIONAL_SYMBOL>',
                            '<NCHAR_SYMBOL>',
                            '<NDBCLUSTER_SYMBOL>',
                            '<NESTED_SYMBOL>',
                            '<NEVER_SYMBOL>',
                            '<NEXT_SYMBOL>',
                            '<NEW_SYMBOL>',
                            '<NO_WAIT_SYMBOL>',
                            '<NODEGROUP_SYMBOL>',
                            '<NULLS_SYMBOL>',
                            '<NOWAIT_SYMBOL>',
                            '<NUMBER_SYMBOL>',
                            '<NVARCHAR_SYMBOL>',
                            '<OFFSET_SYMBOL>',
                            '<OLD_SYMBOL>',
                            '<ONE_SYMBOL>',
                            '<OPTIONAL_SYMBOL>',
                            '<ORDINALITY_SYMBOL>',
                            '<ORGANIZATION_SYMBOL>',
                            '<OTHERS_SYMBOL>',
                            '<PACK_KEYS_SYMBOL>',
                            '<PAGE_SYMBOL>',
                            '<PARTIAL_SYMBOL>',
                            '<PARTITIONING_SYMBOL>',
                            '<PARTITIONS_SYMBOL>',
                            '<PASSWORD_SYMBOL>',
                            '<PATH_SYMBOL>',
                            '<PHASE_SYMBOL>',
                            '<PLUGIN_DIR_SYMBOL>',
                            '<PLUGIN_SYMBOL>',
                            '<PLUGINS_SYMBOL>',
                            '<POINT_SYMBOL>',
                            '<POLYGON_SYMBOL>',
                            '<PRECEDING_SYMBOL>',
                            '<PRESERVE_SYMBOL>',
                            '<PREV_SYMBOL>',
                            '<THREAD_PRIORITY_SYMBOL>',
                            '<PRIVILEGES_SYMBOL>',
                            '<PROCESSLIST_SYMBOL>',
                            '<PROFILE_SYMBOL>',
                            '<PROFILES_SYMBOL>',
                            '<QUARTER_SYMBOL>',
                            '<QUERY_SYMBOL>',
                            '<QUICK_SYMBOL>',
                            '<READ_ONLY_SYMBOL>',
                            '<REBUILD_SYMBOL>',
                            '<RECOVER_SYMBOL>',
                            '<REDO_BUFFER_SIZE_SYMBOL>',
                            '<REDUNDANT_SYMBOL>',
                            '<RELAY_SYMBOL>',
                            '<RELAYLOG_SYMBOL>',
                            '<RELAY_LOG_FILE_SYMBOL>',
                            '<RELAY_LOG_POS_SYMBOL>',
                            '<RELAY_THREAD_SYMBOL>',
                            '<REMOTE_SYMBOL>',
                            '<REORGANIZE_SYMBOL>',
                            '<REPEATABLE_SYMBOL>',
                            '<REPLICATE_DO_DB_SYMBOL>',
                            '<REPLICATE_IGNORE_DB_SYMBOL>',
                            '<REPLICATE_DO_TABLE_SYMBOL>',
                            '<REPLICATE_IGNORE_TABLE_SYMBOL>',
                            '<REPLICATE_WILD_DO_TABLE_SYMBOL>',
                            '<REPLICATE_WILD_IGNORE_TABLE_SYMBOL>',
                            '<REPLICATE_REWRITE_DB_SYMBOL>',
                            '<USER_RESOURCES_SYMBOL>',
                            '<RESPECT_SYMBOL>',
                            '<RESUME_SYMBOL>',
                            '<RETAIN_SYMBOL>',
                            '<RETURNED_SQLSTATE_SYMBOL>',
                            '<RETURNS_SYMBOL>',
                            '<REUSE_SYMBOL>',
                            '<REVERSE_SYMBOL>',
                            '<ROLLUP_SYMBOL>',
                            '<ROTATE_SYMBOL>',
                            '<ROUTINE_SYMBOL>',
                            '<ROW_COUNT_SYMBOL>',
                            '<ROW_FORMAT_SYMBOL>',
                            '<RTREE_SYMBOL>',
                            '<SCHEDULE_SYMBOL>',
                            '<SCHEMA_NAME_SYMBOL>',
                            '<SECOND_SYMBOL>',
                            '<SERIAL_SYMBOL>',
                            '<SERIALIZABLE_SYMBOL>',
                            '<SESSION_SYMBOL>',
                            '<SHARE_SYMBOL>',
                            '<SIMPLE_SYMBOL>',
                            '<SKIP_SYMBOL>',
                            '<SLOW_SYMBOL>',
                            '<SNAPSHOT_SYMBOL>',
                            '<SOUNDS_SYMBOL>',
                            '<SOURCE_SYMBOL>',
                            '<SQL_AFTER_GTIDS_SYMBOL>',
                            '<SQL_AFTER_MTS_GAPS_SYMBOL>',
                            '<SQL_BEFORE_GTIDS_SYMBOL>',
                            '<SQL_BUFFER_RESULT_SYMBOL>',
                            '<SQL_NO_CACHE_SYMBOL>',
                            '<SQL_THREAD_SYMBOL>',
                            '<SRID_SYMBOL>',
                            '<STACKED_SYMBOL>',
                            '<STARTS_SYMBOL>',
                            '<STATS_AUTO_RECALC_SYMBOL>',
                            '<STATS_PERSISTENT_SYMBOL>',
                            '<STATS_SAMPLE_PAGES_SYMBOL>',
                            '<STATUS_SYMBOL>',
                            '<STORAGE_SYMBOL>',
                            '<STRING_SYMBOL>',
                            '<SUBCLASS_ORIGIN_SYMBOL>',
                            '<SUBDATE_SYMBOL>',
                            '<SUBJECT_SYMBOL>',
                            '<SUBPARTITION_SYMBOL>',
                            '<SUBPARTITIONS_SYMBOL>',
                            '<SUPER_SYMBOL>',
                            '<SUSPEND_SYMBOL>',
                            '<SWAPS_SYMBOL>',
                            '<SWITCHES_SYMBOL>',
                            '<TABLE_NAME_SYMBOL>',
                            '<TABLES_SYMBOL>',
                            '<TABLE_CHECKSUM_SYMBOL>',
                            '<TABLESPACE_SYMBOL>',
                            '<TEMPORARY_SYMBOL>',
                            '<TEMPTABLE_SYMBOL>',
                            '<TEXT_SYMBOL>',
                            '<THAN_SYMBOL>',
                            '<TIES_SYMBOL>',
                            '<TRANSACTION_SYMBOL>',
                            '<TRIGGERS_SYMBOL>',
                            '<TIMESTAMP_SYMBOL>',
                            '<TIMESTAMPADD_SYMBOL>',
                            '<TIMESTAMPDIFF_SYMBOL>',
                            '<TIME_SYMBOL>',
                            '<TYPES_SYMBOL>',
                            '<TYPE_SYMBOL>',
                            '<UDF_RETURNS_SYMBOL>',
                            '<UNBOUNDED_SYMBOL>',
                            '<UNCOMMITTED_SYMBOL>',
                            '<UNDEFINED_SYMBOL>',
                            '<UNDO_BUFFER_SIZE_SYMBOL>',
                            '<UNDOFILE_SYMBOL>',
                            '<UNKNOWN_SYMBOL>',
                            '<UNTIL_SYMBOL>',
                            '<USER_SYMBOL>',
                            '<USE_FRM_SYMBOL>',
                            '<VARIABLES_SYMBOL>',
                            '<VCPU_SYMBOL>',
                            '<VIEW_SYMBOL>',
                            '<VALUE_SYMBOL>',
                            '<WARNINGS_SYMBOL>',
                            '<WAIT_SYMBOL>',
                            '<WEEK_SYMBOL>',
                            '<WORK_SYMBOL>',
                            '<WEIGHT_STRING_SYMBOL>',
                            '<X509_SYMBOL>',
                            '<XID_SYMBOL>',
                            '<XML_SYMBOL>',
                            '<YEAR_SYMBOL>'],
 '<EQUAL_OPERATOR>': ['='],
 '<ASSIGN_OPERATOR>': [':='],
 '<NULL_SAFE_EQUAL_OPERATOR>': ['<LESS_THAN_OPERATOR>=<GREATER_THAN_OPERATOR>'],
 '<GREATER_OR_EQUAL_OPERATOR>': ['>='],
 '<GREATER_THAN_OPERATOR>': ['>'],
 '<LESS_OR_EQUAL_OPERATOR>': ['<='],
 '<LESS_THAN_OPERATOR>': ['<'],
 '<NOT_EQUAL_OPERATOR>': ['!='],
 '<PLUS_OPERATOR>': ['+'],
 '<MINUS_OPERATOR>': ['-'],
 '<MULT_OPERATOR>': ['*'],
 '<DIV_OPERATOR>': ['/'],
 '<MOD_OPERATOR>': ['%'],
 '<LOGICAL_NOT_OPERATOR>': ['!'],
 '<BITWISE_NOT_OPERATOR>': ['~'],
 '<SHIFT_LEFT_OPERATOR>': ['<<'],
 '<SHIFT_RIGHT_OPERATOR>': ['>>'],
 '<LOGICAL_AND_OPERATOR>': ['&&'],
 '<BITWISE_AND_OPERATOR>': ['&'],
 '<BITWISE_XOR_OPERATOR>': ['^'],
 '<LOGICAL_OR_OPERATOR>': ['||'],
 '<BITWISE_OR_OPERATOR>': ['|'],
 '<DOT_SYMBOL>': ['.'],
 '<COMMA_SYMBOL>': [','],
 '<SEMICOLON_SYMBOL>': [';'],
 '<COLON_SYMBOL>': [':'],
 '<OPEN_PAR_SYMBOL>': ['('],
 '<CLOSE_PAR_SYMBOL>': [')'],
 '<OPEN_CURLY_SYMBOL>': ['{'],
 '<CLOSE_CURLY_SYMBOL>': ['}'],
 '<UNDERLINE_SYMBOL>': ['_'],
 '<JSON_SEPARATOR_SYMBOL>': ['->'],
 '<JSON_UNQUOTED_SEPARATOR_SYMBOL>': ['->>'],
 '<AT_SIGN_SYMBOL>': ['@'],
 '<AT_TEXT_SUFFIX>': ['@<SIMPLE_IDENTIFIER>'],
 '<AT_AT_SIGN_SYMBOL>': ['@@'],
 '<NULL2_SYMBOL>': ['\\\\N'],
 '<PARAM_MARKER>': ['?'],
 '<DIGIT>': ['<[0-9]>'],
 '<DIGITS>': ['<DIGIT>+'],
 '<HEXDIGIT>': ['<[0-9a-fA-F]>'],
 '<HEX_NUMBER>': ['<HEX_NUMBER-1>', '<HEX_NUMBER-2>'],
 '<HEX_NUMBER-1>': ['0x<HEXDIGIT>+'],
 '<HEX_NUMBER-2>': ["x\\'<HEXDIGIT>+\\'"],
 '<BIN_NUMBER>': ['<BIN_NUMBER-1>', '<BIN_NUMBER-2>'],
 '<BIN_NUMBER-1>': ['0b<[01]>+'],
 '<BIN_NUMBER-2>': ["b\\'<[01]>+\\'"],
 '<INT_NUMBER>': ['<DIGITS>'],
 '<DECIMAL_NUMBER>': ['<DIGITS>?<DOT_SYMBOL><DIGITS>'],
 '<FLOAT_NUMBER>': ['<FLOAT_NUMBER-1>?<DIGITS><[eE]><FLOAT_NUMBER-2>?<DIGITS>'],
 '<FLOAT_NUMBER-1>': ['<DIGITS>?<DOT_SYMBOL>'],
 '<FLOAT_NUMBER-2>': ['<MINUS_OPERATOR>', '<PLUS_OPERATOR>'],
 '<ACCOUNT_SYMBOL>': ['ACCOUNT'],
 '<ACTION_SYMBOL>': ['ACTION'],
 '<ADD_SYMBOL>': ['ADD'],
 '<ADDDATE_SYMBOL>': ['ADDDATE'],
 '<AFTER_SYMBOL>': ['AFTER'],
 '<AGAINST_SYMBOL>': ['AGAINST'],
 '<AGGREGATE_SYMBOL>': ['AGGREGATE'],
 '<ALGORITHM_SYMBOL>': ['ALGORITHM'],
 '<ALL_SYMBOL>': ['ALL'],
 '<ALTER_SYMBOL>': ['ALTER'],
 '<ALWAYS_SYMBOL>': ['ALWAYS'],
 '<ANALYZE_SYMBOL>': ['ANALYZE'],
 '<AND_SYMBOL>': ['AND'],
 '<ANY_SYMBOL>': ['ANY'],
 '<AS_SYMBOL>': ['AS'],
 '<ASC_SYMBOL>': ['ASC'],
 '<ASCII_SYMBOL>': ['ASCII'],
 '<AT_SYMBOL>': ['AT'],
 '<AUTOEXTEND_SIZE_SYMBOL>': ['AUTOEXTEND_SIZE'],
 '<AUTO_INCREMENT_SYMBOL>': ['AUTO_INCREMENT'],
 '<AVG_ROW_LENGTH_SYMBOL>': ['AVG_ROW_LENGTH'],
 '<AVG_SYMBOL>': ['AVG'],
 '<BACKUP_SYMBOL>': ['BACKUP'],
 '<BEFORE_SYMBOL>': ['BEFORE'],
 '<BEGIN_SYMBOL>': ['BEGIN'],
 '<BETWEEN_SYMBOL>': ['BETWEEN'],
 '<BIGINT_SYMBOL>': ['BIGINT'],
 '<BINARY_SYMBOL>': ['BINARY'],
 '<BINLOG_SYMBOL>': ['BINLOG'],
 '<BIT_AND_SYMBOL>': ['BIT_AND'],
 '<BIT_OR_SYMBOL>': ['BIT_OR'],
 '<BIT_SYMBOL>': ['BIT'],
 '<BIT_XOR_SYMBOL>': ['BIT_XOR'],
 '<BLOB_SYMBOL>': ['BLOB'],
 '<BLOCK_SYMBOL>': ['BLOCK'],
 '<BOOLEAN_SYMBOL>': ['BOOLEAN'],
 '<BOOL_SYMBOL>': ['BOOL'],
 '<BOTH_SYMBOL>': ['BOTH'],
 '<BTREE_SYMBOL>': ['BTREE'],
 '<BY_SYMBOL>': ['BY'],
 '<BYTE_SYMBOL>': ['BYTE'],
 '<CACHE_SYMBOL>': ['CACHE'],
 '<CALL_SYMBOL>': ['CALL'],
 '<CASCADE_SYMBOL>': ['CASCADE'],
 '<CASCADED_SYMBOL>': ['CASCADED'],
 '<CASE_SYMBOL>': ['CASE'],
 '<CAST_SYMBOL>': ['CAST'],
 '<CATALOG_NAME_SYMBOL>': ['CATALOG_NAME'],
 '<CHAIN_SYMBOL>': ['CHAIN'],
 '<CHANGE_SYMBOL>': ['CHANGE'],
 '<CHANGED_SYMBOL>': ['CHANGED'],
 '<CHANNEL_SYMBOL>': ['CHANNEL'],
 '<CHARSET_SYMBOL>': ['CHARSET'],
 '<CHAR_SYMBOL>': ['CHAR'],
 '<CHECKSUM_SYMBOL>': ['CHECKSUM'],
 '<CHECK_SYMBOL>': ['CHECK'],
 '<CIPHER_SYMBOL>': ['CIPHER'],
 '<CLASS_ORIGIN_SYMBOL>': ['CLASS_ORIGIN'],
 '<CLIENT_SYMBOL>': ['CLIENT'],
 '<CLOSE_SYMBOL>': ['CLOSE'],
 '<COALESCE_SYMBOL>': ['COALESCE'],
 '<CODE_SYMBOL>': ['CODE'],
 '<COLLATE_SYMBOL>': ['COLLATE'],
 '<COLLATION_SYMBOL>': ['COLLATION'],
 '<COLUMNS_SYMBOL>': ['COLUMNS'],
 '<COLUMN_SYMBOL>': ['COLUMN'],
 '<COLUMN_NAME_SYMBOL>': ['COLUMN_NAME'],
 '<COLUMN_FORMAT_SYMBOL>': ['COLUMN_FORMAT'],
 '<COMMENT_SYMBOL>': ['COMMENT'],
 '<COMMITTED_SYMBOL>': ['COMMITTED'],
 '<COMMIT_SYMBOL>': ['COMMIT'],
 '<COMPACT_SYMBOL>': ['COMPACT'],
 '<COMPLETION_SYMBOL>': ['COMPLETION'],
 '<COMPRESSED_SYMBOL>': ['COMPRESSED'],
 '<COMPRESSION_SYMBOL>': ['COMPRESSION'],
 '<CONCURRENT_SYMBOL>': ['CONCURRENT'],
 '<CONDITION_SYMBOL>': ['CONDITION'],
 '<CONNECTION_SYMBOL>': ['CONNECTION'],
 '<CONSISTENT_SYMBOL>': ['CONSISTENT'],
 '<CONSTRAINT_SYMBOL>': ['CONSTRAINT'],
 '<CONSTRAINT_CATALOG_SYMBOL>': ['CONSTRAINT_CATALOG'],
 '<CONSTRAINT_NAME_SYMBOL>': ['CONSTRAINT_NAME'],
 '<CONSTRAINT_SCHEMA_SYMBOL>': ['CONSTRAINT_SCHEMA'],
 '<CONTAINS_SYMBOL>': ['CONTAINS'],
 '<CONTEXT_SYMBOL>': ['CONTEXT'],
 '<CONTINUE_SYMBOL>': ['CONTINUE'],
 '<CONVERT_SYMBOL>': ['CONVERT'],
 '<COUNT_SYMBOL>': ['COUNT'],
 '<CPU_SYMBOL>': ['CPU'],
 '<CREATE_SYMBOL>': ['CREATE'],
 '<CROSS_SYMBOL>': ['CROSS'],
 '<CUBE_SYMBOL>': ['CUBE'],
 '<CURDATE_SYMBOL>': ['CURDATE'],
 '<CURRENT_SYMBOL>': ['CURRENT'],
 '<CURRENT_USER_SYMBOL>': ['CURRENT_USER'],
 '<CURSOR_SYMBOL>': ['CURSOR'],
 '<CURSOR_NAME_SYMBOL>': ['CURSOR_NAME'],
 '<CURTIME_SYMBOL>': ['CURTIME'],
 '<DATABASE_SYMBOL>': ['DATABASE'],
 '<DATABASES_SYMBOL>': ['DATABASES'],
 '<DATAFILE_SYMBOL>': ['DATAFILE'],
 '<DATA_SYMBOL>': ['DATA'],
 '<DATETIME_SYMBOL>': ['DATETIME'],
 '<DATE_ADD_SYMBOL>': ['DATE_ADD'],
 '<DATE_SUB_SYMBOL>': ['DATE_SUB'],
 '<DATE_SYMBOL>': ['DATE'],
 '<DAY_HOUR_SYMBOL>': ['DAY_HOUR'],
 '<DAY_MICROSECOND_SYMBOL>': ['DAY_MICROSECOND'],
 '<DAY_MINUTE_SYMBOL>': ['DAY_MINUTE'],
 '<DAY_SECOND_SYMBOL>': ['DAY_SECOND'],
 '<DAY_SYMBOL>': ['DAY'],
 '<DEALLOCATE_SYMBOL>': ['DEALLOCATE'],
 '<DECIMAL_SYMBOL>': ['DECIMAL'],
 '<DECLARE_SYMBOL>': ['DECLARE'],
 '<DEFAULT_SYMBOL>': ['DEFAULT'],
 '<DEFAULT_AUTH_SYMBOL>': ['DEFAULT_AUTH'],
 '<DEFINER_SYMBOL>': ['DEFINER'],
 '<DELAYED_SYMBOL>': ['DELAYED'],
 '<DELAY_KEY_WRITE_SYMBOL>': ['DELAY_KEY_WRITE'],
 '<DELETE_SYMBOL>': ['DELETE'],
 '<DESC_SYMBOL>': ['DESC'],
 '<DESCRIBE_SYMBOL>': ['DESCRIBE'],
 '<DETERMINISTIC_SYMBOL>': ['DETERMINISTIC'],
 '<DIAGNOSTICS_SYMBOL>': ['DIAGNOSTICS'],
 '<DIRECTORY_SYMBOL>': ['DIRECTORY'],
 '<DISABLE_SYMBOL>': ['DISABLE'],
 '<DISCARD_SYMBOL>': ['DISCARD'],
 '<DISK_SYMBOL>': ['DISK'],
 '<DISTINCT_SYMBOL>': ['DISTINCT'],
 '<DIV_SYMBOL>': ['DIV'],
 '<DOUBLE_SYMBOL>': ['DOUBLE'],
 '<DO_SYMBOL>': ['DO'],
 '<DROP_SYMBOL>': ['DROP'],
 '<DUAL_SYMBOL>': ['DUAL'],
 '<DUMPFILE_SYMBOL>': ['DUMPFILE'],
 '<DUPLICATE_SYMBOL>': ['DUPLICATE'],
 '<DYNAMIC_SYMBOL>': ['DYNAMIC'],
 '<EACH_SYMBOL>': ['EACH'],
 '<ELSE_SYMBOL>': ['ELSE'],
 '<ELSEIF_SYMBOL>': ['ELSEIF'],
 '<ENABLE_SYMBOL>': ['ENABLE'],
 '<ENCLOSED_SYMBOL>': ['ENCLOSED'],
 '<ENCRYPTION_SYMBOL>': ['ENCRYPTION'],
 '<END_SYMBOL>': ['END'],
 '<ENDS_SYMBOL>': ['ENDS'],
 '<ENGINES_SYMBOL>': ['ENGINES'],
 '<ENGINE_SYMBOL>': ['ENGINE'],
 '<ENUM_SYMBOL>': ['ENUM'],
 '<ERROR_SYMBOL>': ['ERROR'],
 '<ERRORS_SYMBOL>': ['ERRORS'],
 '<ESCAPED_SYMBOL>': ['ESCAPED'],
 '<ESCAPE_SYMBOL>': ['ESCAPE'],
 '<EVENTS_SYMBOL>': ['EVENTS'],
 '<EVENT_SYMBOL>': ['EVENT'],
 '<EVERY_SYMBOL>': ['EVERY'],
 '<EXCHANGE_SYMBOL>': ['EXCHANGE'],
 '<EXECUTE_SYMBOL>': ['EXECUTE'],
 '<EXISTS_SYMBOL>': ['EXISTS'],
 '<EXIT_SYMBOL>': ['EXIT'],
 '<EXPANSION_SYMBOL>': ['EXPANSION'],
 '<EXPIRE_SYMBOL>': ['EXPIRE'],
 '<EXPLAIN_SYMBOL>': ['EXPLAIN'],
 '<EXPORT_SYMBOL>': ['EXPORT'],
 '<EXTENDED_SYMBOL>': ['EXTENDED'],
 '<EXTENT_SIZE_SYMBOL>': ['EXTENT_SIZE'],
 '<EXTRACT_SYMBOL>': ['EXTRACT'],
 '<FALSE_SYMBOL>': ['FALSE'],
 '<FAST_SYMBOL>': ['FAST'],
 '<FAULTS_SYMBOL>': ['FAULTS'],
 '<FETCH_SYMBOL>': ['FETCH'],
 '<FILE_SYMBOL>': ['FILE'],
 '<FILE_BLOCK_SIZE_SYMBOL>': ['FILE_BLOCK_SIZE'],
 '<FILTER_SYMBOL>': ['FILTER'],
 '<FIRST_SYMBOL>': ['FIRST'],
 '<FIXED_SYMBOL>': ['FIXED'],
 '<FLOAT_SYMBOL>': ['FLOAT'],
 '<FLUSH_SYMBOL>': ['FLUSH'],
 '<FOLLOWS_SYMBOL>': ['FOLLOWS'],
 '<FORCE_SYMBOL>': ['FORCE'],
 '<FOREIGN_SYMBOL>': ['FOREIGN'],
 '<FOR_SYMBOL>': ['FOR'],
 '<FORMAT_SYMBOL>': ['FORMAT'],
 '<FOUND_SYMBOL>': ['FOUND'],
 '<FROM_SYMBOL>': ['FROM'],
 '<FULL_SYMBOL>': ['FULL'],
 '<FULLTEXT_SYMBOL>': ['FULLTEXT'],
 '<FUNCTION_SYMBOL>': ['FUNCTION'],
 '<GET_SYMBOL>': ['GET'],
 '<GENERAL_SYMBOL>': ['GENERAL'],
 '<GENERATED_SYMBOL>': ['GENERATED'],
 '<GROUP_REPLICATION_SYMBOL>': ['GROUP_REPLICATION'],
 '<GEOMETRYCOLLECTION_SYMBOL>': ['GEOMETRYCOLLECTION'],
 '<GEOMETRY_SYMBOL>': ['GEOMETRY'],
 '<GET_FORMAT_SYMBOL>': ['GET_FORMAT'],
 '<GLOBAL_SYMBOL>': ['GLOBAL'],
 '<GRANT_SYMBOL>': ['GRANT'],
 '<GRANTS_SYMBOL>': ['GRANTS'],
 '<GROUP_SYMBOL>': ['GROUP'],
 '<GROUP_CONCAT_SYMBOL>': ['GROUP_CONCAT'],
 '<HANDLER_SYMBOL>': ['HANDLER'],
 '<HASH_SYMBOL>': ['HASH'],
 '<HAVING_SYMBOL>': ['HAVING'],
 '<HELP_SYMBOL>': ['HELP'],
 '<HIGH_PRIORITY_SYMBOL>': ['HIGH_PRIORITY'],
 '<HOST_SYMBOL>': ['HOST'],
 '<HOSTS_SYMBOL>': ['HOSTS'],
 '<HOUR_MICROSECOND_SYMBOL>': ['HOUR_MICROSECOND'],
 '<HOUR_MINUTE_SYMBOL>': ['HOUR_MINUTE'],
 '<HOUR_SECOND_SYMBOL>': ['HOUR_SECOND'],
 '<HOUR_SYMBOL>': ['HOUR'],
 '<IDENTIFIED_SYMBOL>': ['IDENTIFIED'],
 '<IF_SYMBOL>': ['IF'],
 '<IGNORE_SYMBOL>': ['IGNORE'],
 '<IGNORE_SERVER_IDS_SYMBOL>': ['IGNORE_SERVER_IDS'],
 '<IMPORT_SYMBOL>': ['IMPORT'],
 '<INDEXES_SYMBOL>': ['INDEXES'],
 '<INDEX_SYMBOL>': ['INDEX'],
 '<INFILE_SYMBOL>': ['INFILE'],
 '<INITIAL_SIZE_SYMBOL>': ['INITIAL_SIZE'],
 '<INNER_SYMBOL>': ['INNER'],
 '<INOUT_SYMBOL>': ['INOUT'],
 '<INSERT_SYMBOL>': ['INSERT'],
 '<INSERT_METHOD_SYMBOL>': ['INSERT_METHOD'],
 '<INSTANCE_SYMBOL>': ['INSTANCE'],
 '<INSTALL_SYMBOL>': ['INSTALL'],
 '<INTERVAL_SYMBOL>': ['INTERVAL'],
 '<INTO_SYMBOL>': ['INTO'],
 '<INT_SYMBOL>': ['INT'],
 '<INVOKER_SYMBOL>': ['INVOKER'],
 '<IN_SYMBOL>': ['IN'],
 '<IO_SYMBOL>': ['IO'],
 '<IPC_SYMBOL>': ['IPC'],
 '<IS_SYMBOL>': ['IS'],
 '<ISOLATION_SYMBOL>': ['ISOLATION'],
 '<ISSUER_SYMBOL>': ['ISSUER'],
 '<ITERATE_SYMBOL>': ['ITERATE'],
 '<JOIN_SYMBOL>': ['JOIN'],
 '<JSON_SYMBOL>': ['JSON'],
 '<KEYS_SYMBOL>': ['KEYS'],
 '<KEY_BLOCK_SIZE_SYMBOL>': ['KEY_BLOCK_SIZE'],
 '<KEY_SYMBOL>': ['KEY'],
 '<KILL_SYMBOL>': ['KILL'],
 '<LANGUAGE_SYMBOL>': ['LANGUAGE'],
 '<LAST_SYMBOL>': ['LAST'],
 '<LEADING_SYMBOL>': ['LEADING'],
 '<LEAVES_SYMBOL>': ['LEAVES'],
 '<LEAVE_SYMBOL>': ['LEAVE'],
 '<LEFT_SYMBOL>': ['LEFT'],
 '<LESS_SYMBOL>': ['LESS'],
 '<LEVEL_SYMBOL>': ['LEVEL'],
 '<LIKE_SYMBOL>': ['LIKE'],
 '<LIMIT_SYMBOL>': ['LIMIT'],
 '<LINEAR_SYMBOL>': ['LINEAR'],
 '<LINES_SYMBOL>': ['LINES'],
 '<LINESTRING_SYMBOL>': ['LINESTRING'],
 '<LIST_SYMBOL>': ['LIST'],
 '<LOAD_SYMBOL>': ['LOAD'],
 '<LOCAL_SYMBOL>': ['LOCAL'],
 '<LOCKS_SYMBOL>': ['LOCKS'],
 '<LOCK_SYMBOL>': ['LOCK'],
 '<LOGFILE_SYMBOL>': ['LOGFILE'],
 '<LOGS_SYMBOL>': ['LOGS'],
 '<LONGBLOB_SYMBOL>': ['LONGBLOB'],
 '<LONGTEXT_SYMBOL>': ['LONGTEXT'],
 '<LONG_SYMBOL>': ['LONG'],
 '<LOOP_SYMBOL>': ['LOOP'],
 '<LOW_PRIORITY_SYMBOL>': ['LOW_PRIORITY'],
 '<MASTER_AUTO_POSITION_SYMBOL>': ['MASTER_AUTO_POSITION'],
 '<MASTER_BIND_SYMBOL>': ['MASTER_BIND'],
 '<MASTER_CONNECT_RETRY_SYMBOL>': ['MASTER_CONNECT_RETRY'],
 '<MASTER_DELAY_SYMBOL>': ['MASTER_DELAY'],
 '<MASTER_HOST_SYMBOL>': ['MASTER_HOST'],
 '<MASTER_LOG_FILE_SYMBOL>': ['MASTER_LOG_FILE'],
 '<MASTER_LOG_POS_SYMBOL>': ['MASTER_LOG_POS'],
 '<MASTER_PASSWORD_SYMBOL>': ['MASTER_PASSWORD'],
 '<MASTER_PORT_SYMBOL>': ['MASTER_PORT'],
 '<MASTER_RETRY_COUNT_SYMBOL>': ['MASTER_RETRY_COUNT'],
 '<MASTER_SSL_CAPATH_SYMBOL>': ['MASTER_SSL_CAPATH'],
 '<MASTER_SSL_CA_SYMBOL>': ['MASTER_SSL_CA'],
 '<MASTER_SSL_CERT_SYMBOL>': ['MASTER_SSL_CERT'],
 '<MASTER_SSL_CIPHER_SYMBOL>': ['MASTER_SSL_CIPHER'],
 '<MASTER_SSL_CRL_SYMBOL>': ['MASTER_SSL_CRL'],
 '<MASTER_SSL_CRLPATH_SYMBOL>': ['MASTER_SSL_CRLPATH'],
 '<MASTER_SSL_KEY_SYMBOL>': ['MASTER_SSL_KEY'],
 '<MASTER_SSL_SYMBOL>': ['MASTER_SSL'],
 '<MASTER_SSL_VERIFY_SERVER_CERT_SYMBOL>': ['MASTER_SSL_VERIFY_SERVER_CERT'],
 '<MASTER_SYMBOL>': ['MASTER'],
 '<MASTER_TLS_VERSION_SYMBOL>': ['MASTER_TLS_VERSION'],
 '<MASTER_USER_SYMBOL>': ['MASTER_USER'],
 '<MASTER_HEARTBEAT_PERIOD_SYMBOL>': ['MASTER_HEARTBEAT_PERIOD'],
 '<MATCH_SYMBOL>': ['MATCH'],
 '<MAX_CONNECTIONS_PER_HOUR_SYMBOL>': ['MAX_CONNECTIONS_PER_HOUR'],
 '<MAX_QUERIES_PER_HOUR_SYMBOL>': ['MAX_QUERIES_PER_HOUR'],
 '<MAX_ROWS_SYMBOL>': ['MAX_ROWS'],
 '<MAX_SIZE_SYMBOL>': ['MAX_SIZE'],
 '<MAX_SYMBOL>': ['MAX'],
 '<MAX_UPDATES_PER_HOUR_SYMBOL>': ['MAX_UPDATES_PER_HOUR'],
 '<MAX_USER_CONNECTIONS_SYMBOL>': ['MAX_USER_CONNECTIONS'],
 '<MAXVALUE_SYMBOL>': ['MAXVALUE'],
 '<MEDIUMBLOB_SYMBOL>': ['MEDIUMBLOB'],
 '<MEDIUMINT_SYMBOL>': ['MEDIUMINT'],
 '<MEDIUMTEXT_SYMBOL>': ['MEDIUMTEXT'],
 '<MEDIUM_SYMBOL>': ['MEDIUM'],
 '<MEMORY_SYMBOL>': ['MEMORY'],
 '<MERGE_SYMBOL>': ['MERGE'],
 '<MESSAGE_TEXT_SYMBOL>': ['MESSAGE_TEXT'],
 '<MICROSECOND_SYMBOL>': ['MICROSECOND'],
 '<MIGRATE_SYMBOL>': ['MIGRATE'],
 '<MINUTE_MICROSECOND_SYMBOL>': ['MINUTE_MICROSECOND'],
 '<MINUTE_SECOND_SYMBOL>': ['MINUTE_SECOND'],
 '<MINUTE_SYMBOL>': ['MINUTE'],
 '<MIN_ROWS_SYMBOL>': ['MIN_ROWS'],
 '<MIN_SYMBOL>': ['MIN'],
 '<MODE_SYMBOL>': ['MODE'],
 '<MODIFIES_SYMBOL>': ['MODIFIES'],
 '<MODIFY_SYMBOL>': ['MODIFY'],
 '<MOD_SYMBOL>': ['MOD'],
 '<MONTH_SYMBOL>': ['MONTH'],
 '<MULTILINESTRING_SYMBOL>': ['MULTILINESTRING'],
 '<MULTIPOINT_SYMBOL>': ['MULTIPOINT'],
 '<MULTIPOLYGON_SYMBOL>': ['MULTIPOLYGON'],
 '<MUTEX_SYMBOL>': ['MUTEX'],
 '<MYSQL_ERRNO_SYMBOL>': ['MYSQL_ERRNO'],
 '<NAMES_SYMBOL>': ['NAMES'],
 '<NAME_SYMBOL>': ['NAME'],
 '<NATIONAL_SYMBOL>': ['NATIONAL'],
 '<NATURAL_SYMBOL>': ['NATURAL'],
 '<NCHAR_SYMBOL>': ['NCHAR'],
 '<NDBCLUSTER_SYMBOL>': ['NDBCLUSTER'],
 '<NEVER_SYMBOL>': ['NEVER'],
 '<NEW_SYMBOL>': ['NEW'],
 '<NEXT_SYMBOL>': ['NEXT'],
 '<NODEGROUP_SYMBOL>': ['NODEGROUP'],
 '<NONE_SYMBOL>': ['NONE'],
 '<NOT_SYMBOL>': ['NOT'],
 '<NOW_SYMBOL>': ['NOW'],
 '<NO_SYMBOL>': ['NO'],
 '<NO_WAIT_SYMBOL>': ['NO_WAIT'],
 '<NO_WRITE_TO_BINLOG_SYMBOL>': ['NO_WRITE_TO_BINLOG'],
 '<NULL_SYMBOL>': ['NULL'],
 '<NUMBER_SYMBOL>': ['NUMBER'],
 '<NUMERIC_SYMBOL>': ['NUMERIC'],
 '<NVARCHAR_SYMBOL>': ['NVARCHAR'],
 '<OFFLINE_SYMBOL>': ['OFFLINE'],
 '<OFFSET_SYMBOL>': ['OFFSET'],
 '<ON_SYMBOL>': ['ON'],
 '<ONE_SYMBOL>': ['ONE'],
 '<ONLINE_SYMBOL>': ['ONLINE'],
 '<ONLY_SYMBOL>': ['ONLY'],
 '<OPEN_SYMBOL>': ['OPEN'],
 '<OPTIMIZE_SYMBOL>': ['OPTIMIZE'],
 '<OPTIMIZER_COSTS_SYMBOL>': ['OPTIMIZER_COSTS'],
 '<OPTIONS_SYMBOL>': ['OPTIONS'],
 '<OPTION_SYMBOL>': ['OPTION'],
 '<OPTIONALLY_SYMBOL>': ['OPTIONALLY'],
 '<ORDER_SYMBOL>': ['ORDER'],
 '<OR_SYMBOL>': ['OR'],
 '<OUTER_SYMBOL>': ['OUTER'],
 '<OUTFILE_SYMBOL>': ['OUTFILE'],
 '<OUT_SYMBOL>': ['OUT'],
 '<OWNER_SYMBOL>': ['OWNER'],
 '<PACK_KEYS_SYMBOL>': ['PACK_KEYS'],
 '<PAGE_SYMBOL>': ['PAGE'],
 '<PARSER_SYMBOL>': ['PARSER'],
 '<PARTIAL_SYMBOL>': ['PARTIAL'],
 '<PARTITIONING_SYMBOL>': ['PARTITIONING'],
 '<PARTITIONS_SYMBOL>': ['PARTITIONS'],
 '<PARTITION_SYMBOL>': ['PARTITION'],
 '<PASSWORD_SYMBOL>': ['PASSWORD'],
 '<PHASE_SYMBOL>': ['PHASE'],
 '<PLUGINS_SYMBOL>': ['PLUGINS'],
 '<PLUGIN_DIR_SYMBOL>': ['PLUGIN_DIR'],
 '<PLUGIN_SYMBOL>': ['PLUGIN'],
 '<POINT_SYMBOL>': ['POINT'],
 '<POLYGON_SYMBOL>': ['POLYGON'],
 '<PORT_SYMBOL>': ['PORT'],
 '<POSITION_SYMBOL>': ['POSITION'],
 '<PRECEDES_SYMBOL>': ['PRECEDES'],
 '<PRECISION_SYMBOL>': ['PRECISION'],
 '<PREPARE_SYMBOL>': ['PREPARE'],
 '<PRESERVE_SYMBOL>': ['PRESERVE'],
 '<PREV_SYMBOL>': ['PREV'],
 '<PRIMARY_SYMBOL>': ['PRIMARY'],
 '<PRIVILEGES_SYMBOL>': ['PRIVILEGES'],
 '<PROCEDURE_SYMBOL>': ['PROCEDURE'],
 '<PROCESS_SYMBOL>': ['PROCESS'],
 '<PROCESSLIST_SYMBOL>': ['PROCESSLIST'],
 '<PROFILE_SYMBOL>': ['PROFILE'],
 '<PROFILES_SYMBOL>': ['PROFILES'],
 '<PROXY_SYMBOL>': ['PROXY'],
 '<PURGE_SYMBOL>': ['PURGE'],
 '<QUARTER_SYMBOL>': ['QUARTER'],
 '<QUERY_SYMBOL>': ['QUERY'],
 '<QUICK_SYMBOL>': ['QUICK'],
 '<RANGE_SYMBOL>': ['RANGE'],
 '<READS_SYMBOL>': ['READS'],
 '<READ_ONLY_SYMBOL>': ['READ_ONLY'],
 '<READ_SYMBOL>': ['READ'],
 '<REAL_SYMBOL>': ['REAL'],
 '<REBUILD_SYMBOL>': ['REBUILD'],
 '<RECOVER_SYMBOL>': ['RECOVER'],
 '<REDO_BUFFER_SIZE_SYMBOL>': ['REDO_BUFFER_SIZE'],
 '<REDUNDANT_SYMBOL>': ['REDUNDANT'],
 '<REFERENCES_SYMBOL>': ['REFERENCES'],
 '<REGEXP_SYMBOL>': ['REGEXP'],
 '<RELAY_SYMBOL>': ['RELAY'],
 '<RELAYLOG_SYMBOL>': ['RELAYLOG'],
 '<RELAY_LOG_FILE_SYMBOL>': ['RELAY_LOG_FILE'],
 '<RELAY_LOG_POS_SYMBOL>': ['RELAY_LOG_POS'],
 '<RELAY_THREAD_SYMBOL>': ['RELAY_THREAD'],
 '<RELEASE_SYMBOL>': ['RELEASE'],
 '<RELOAD_SYMBOL>': ['RELOAD'],
 '<REMOVE_SYMBOL>': ['REMOVE'],
 '<RENAME_SYMBOL>': ['RENAME'],
 '<REORGANIZE_SYMBOL>': ['REORGANIZE'],
 '<REPAIR_SYMBOL>': ['REPAIR'],
 '<REPEATABLE_SYMBOL>': ['REPEATABLE'],
 '<REPEAT_SYMBOL>': ['REPEAT'],
 '<REPLACE_SYMBOL>': ['REPLACE'],
 '<REPLICATION_SYMBOL>': ['REPLICATION'],
 '<REPLICATE_DO_DB_SYMBOL>': ['REPLICATE_DO_DB'],
 '<REPLICATE_IGNORE_DB_SYMBOL>': ['REPLICATE_IGNORE_DB'],
 '<REPLICATE_DO_TABLE_SYMBOL>': ['REPLICATE_DO_TABLE'],
 '<REPLICATE_IGNORE_TABLE_SYMBOL>': ['REPLICATE_IGNORE_TABLE'],
 '<REPLICATE_WILD_DO_TABLE_SYMBOL>': ['REPLICATE_WILD_DO_TABLE'],
 '<REPLICATE_WILD_IGNORE_TABLE_SYMBOL>': ['REPLICATE_WILD_IGNORE_TABLE'],
 '<REPLICATE_REWRITE_DB_SYMBOL>': ['REPLICATE_REWRITE_DB'],
 '<REQUIRE_SYMBOL>': ['REQUIRE'],
 '<RESET_SYMBOL>': ['RESET'],
 '<RESIGNAL_SYMBOL>': ['RESIGNAL'],
 '<RESTORE_SYMBOL>': ['RESTORE'],
 '<RESTRICT_SYMBOL>': ['RESTRICT'],
 '<RESUME_SYMBOL>': ['RESUME'],
 '<RETURNED_SQLSTATE_SYMBOL>': ['RETURNED_SQLSTATE'],
 '<RETURNS_SYMBOL>': ['RETURNS'],
 '<RETURN_SYMBOL>': ['RETURN'],
 '<REVERSE_SYMBOL>': ['REVERSE'],
 '<REVOKE_SYMBOL>': ['REVOKE'],
 '<RIGHT_SYMBOL>': ['RIGHT'],
 '<ROLLBACK_SYMBOL>': ['ROLLBACK'],
 '<ROLLUP_SYMBOL>': ['ROLLUP'],
 '<ROTATE_SYMBOL>': ['ROTATE'],
 '<ROUTINE_SYMBOL>': ['ROUTINE'],
 '<ROWS_SYMBOL>': ['ROWS'],
 '<ROW_COUNT_SYMBOL>': ['ROW_COUNT'],
 '<ROW_FORMAT_SYMBOL>': ['ROW_FORMAT'],
 '<ROW_SYMBOL>': ['ROW'],
 '<RTREE_SYMBOL>': ['RTREE'],
 '<SAVEPOINT_SYMBOL>': ['SAVEPOINT'],
 '<SCHEDULE_SYMBOL>': ['SCHEDULE'],
 '<SCHEMA_NAME_SYMBOL>': ['SCHEMA_NAME'],
 '<SECOND_MICROSECOND_SYMBOL>': ['SECOND_MICROSECOND'],
 '<SECOND_SYMBOL>': ['SECOND'],
 '<SECURITY_SYMBOL>': ['SECURITY'],
 '<SELECT_SYMBOL>': ['SELECT'],
 '<SEPARATOR_SYMBOL>': ['SEPARATOR'],
 '<SERIALIZABLE_SYMBOL>': ['SERIALIZABLE'],
 '<SERIAL_SYMBOL>': ['SERIAL'],
 '<SESSION_SYMBOL>': ['SESSION'],
 '<SERVER_SYMBOL>': ['SERVER'],
 '<SET_SYMBOL>': ['SET'],
 '<SHARE_SYMBOL>': ['SHARE'],
 '<SHOW_SYMBOL>': ['SHOW'],
 '<SHUTDOWN_SYMBOL>': ['SHUTDOWN'],
 '<SIGNAL_SYMBOL>': ['SIGNAL'],
 '<SIGNED_SYMBOL>': ['SIGNED'],
 '<SIMPLE_SYMBOL>': ['SIMPLE'],
 '<SLAVE_SYMBOL>': ['SLAVE'],
 '<SLOW_SYMBOL>': ['SLOW'],
 '<SMALLINT_SYMBOL>': ['SMALLINT'],
 '<SNAPSHOT_SYMBOL>': ['SNAPSHOT'],
 '<SOCKET_SYMBOL>': ['SOCKET'],
 '<SONAME_SYMBOL>': ['SONAME'],
 '<SOUNDS_SYMBOL>': ['SOUNDS'],
 '<SOURCE_SYMBOL>': ['SOURCE'],
 '<SPATIAL_SYMBOL>': ['SPATIAL'],
 '<SQLEXCEPTION_SYMBOL>': ['SQLEXCEPTION'],
 '<SQLSTATE_SYMBOL>': ['SQLSTATE'],
 '<SQLWARNING_SYMBOL>': ['SQLWARNING'],
 '<SQL_AFTER_GTIDS_SYMBOL>': ['SQL_AFTER_GTIDS'],
 '<SQL_AFTER_MTS_GAPS_SYMBOL>': ['SQL_AFTER_MTS_GAPS'],
 '<SQL_BEFORE_GTIDS_SYMBOL>': ['SQL_BEFORE_GTIDS'],
 '<SQL_BIG_RESULT_SYMBOL>': ['SQL_BIG_RESULT'],
 '<SQL_BUFFER_RESULT_SYMBOL>': ['SQL_BUFFER_RESULT'],
 '<SQL_CALC_FOUND_ROWS_SYMBOL>': ['SQL_CALC_FOUND_ROWS'],
 '<SQL_NO_CACHE_SYMBOL>': ['SQL_NO_CACHE'],
 '<SQL_SMALL_RESULT_SYMBOL>': ['SQL_SMALL_RESULT'],
 '<SQL_SYMBOL>': ['SQL'],
 '<SQL_THREAD_SYMBOL>': ['SQL_THREAD'],
 '<SSL_SYMBOL>': ['SSL'],
 '<STACKED_SYMBOL>': ['STACKED'],
 '<STARTING_SYMBOL>': ['STARTING'],
 '<STARTS_SYMBOL>': ['STARTS'],
 '<START_SYMBOL>': ['START'],
 '<STATS_AUTO_RECALC_SYMBOL>': ['STATS_AUTO_RECALC'],
 '<STATS_PERSISTENT_SYMBOL>': ['STATS_PERSISTENT'],
 '<STATS_SAMPLE_PAGES_SYMBOL>': ['STATS_SAMPLE_PAGES'],
 '<STATUS_SYMBOL>': ['STATUS'],
 '<STDDEV_SAMP_SYMBOL>': ['STDDEV_SAMP'],
 '<STD_SYMBOL>': ['STD'],
 '<STOP_SYMBOL>': ['STOP'],
 '<STORAGE_SYMBOL>': ['STORAGE'],
 '<STORED_SYMBOL>': ['STORED'],
 '<STRAIGHT_JOIN_SYMBOL>': ['STRAIGHT_JOIN'],
 '<STRING_SYMBOL>': ['STRING'],
 '<SUBCLASS_ORIGIN_SYMBOL>': ['SUBCLASS_ORIGIN'],
 '<SUBDATE_SYMBOL>': ['SUBDATE'],
 '<SUBJECT_SYMBOL>': ['SUBJECT'],
 '<SUBPARTITIONS_SYMBOL>': ['SUBPARTITIONS'],
 '<SUBPARTITION_SYMBOL>': ['SUBPARTITION'],
 '<SUBSTRING_SYMBOL>': ['SUBSTRING'],
 '<SUM_SYMBOL>': ['SUM'],
 '<SUPER_SYMBOL>': ['SUPER'],
 '<SUSPEND_SYMBOL>': ['SUSPEND'],
 '<SWAPS_SYMBOL>': ['SWAPS'],
 '<SWITCHES_SYMBOL>': ['SWITCHES'],
 '<SYSDATE_SYMBOL>': ['SYSDATE'],
 '<TABLES_SYMBOL>': ['TABLES'],
 '<TABLESPACE_SYMBOL>': ['TABLESPACE'],
 '<TABLE_SYMBOL>': ['TABLE'],
 '<TABLE_CHECKSUM_SYMBOL>': ['TABLE_CHECKSUM'],
 '<TABLE_NAME_SYMBOL>': ['TABLE_NAME'],
 '<TEMPORARY_SYMBOL>': ['TEMPORARY'],
 '<TEMPTABLE_SYMBOL>': ['TEMPTABLE'],
 '<TERMINATED_SYMBOL>': ['TERMINATED'],
 '<TEXT_SYMBOL>': ['TEXT'],
 '<THAN_SYMBOL>': ['THAN'],
 '<THEN_SYMBOL>': ['THEN'],
 '<TIMESTAMP_SYMBOL>': ['TIMESTAMP'],
 '<TIMESTAMPADD_SYMBOL>': ['TIMESTAMPADD'],
 '<TIMESTAMPDIFF_SYMBOL>': ['TIMESTAMPDIFF'],
 '<TIME_SYMBOL>': ['TIME'],
 '<TINYBLOB_SYMBOL>': ['TINYBLOB'],
 '<TINYINT_SYMBOL>': ['TINYINT'],
 '<TINYTEXT_SYMBOL>': ['TINYTEXT'],
 '<TO_SYMBOL>': ['TO'],
 '<TRAILING_SYMBOL>': ['TRAILING'],
 '<TRANSACTION_SYMBOL>': ['TRANSACTION'],
 '<TRIGGERS_SYMBOL>': ['TRIGGERS'],
 '<TRIGGER_SYMBOL>': ['TRIGGER'],
 '<TRIM_SYMBOL>': ['TRIM'],
 '<TRUE_SYMBOL>': ['TRUE'],
 '<TRUNCATE_SYMBOL>': ['TRUNCATE'],
 '<TYPES_SYMBOL>': ['TYPES'],
 '<TYPE_SYMBOL>': ['TYPE'],
 '<UDF_RETURNS_SYMBOL>': ['UDF_RETURNS'],
 '<UNCOMMITTED_SYMBOL>': ['UNCOMMITTED'],
 '<UNDEFINED_SYMBOL>': ['UNDEFINED'],
 '<UNDOFILE_SYMBOL>': ['UNDOFILE'],
 '<UNDO_BUFFER_SIZE_SYMBOL>': ['UNDO_BUFFER_SIZE'],
 '<UNDO_SYMBOL>': ['UNDO'],
 '<UNICODE_SYMBOL>': ['UNICODE'],
 '<UNINSTALL_SYMBOL>': ['UNINSTALL'],
 '<UNION_SYMBOL>': ['UNION'],
 '<UNIQUE_SYMBOL>': ['UNIQUE'],
 '<UNKNOWN_SYMBOL>': ['UNKNOWN'],
 '<UNLOCK_SYMBOL>': ['UNLOCK'],
 '<UNSIGNED_SYMBOL>': ['UNSIGNED'],
 '<UNTIL_SYMBOL>': ['UNTIL'],
 '<UPDATE_SYMBOL>': ['UPDATE'],
 '<UPGRADE_SYMBOL>': ['UPGRADE'],
 '<USAGE_SYMBOL>': ['USAGE'],
 '<USER_RESOURCES_SYMBOL>': ['USER_RESOURCES'],
 '<USER_SYMBOL>': ['USER'],
 '<USE_FRM_SYMBOL>': ['USE_FRM'],
 '<USE_SYMBOL>': ['USE'],
 '<USING_SYMBOL>': ['USING'],
 '<UTC_DATE_SYMBOL>': ['UTC_DATE'],
 '<UTC_TIMESTAMP_SYMBOL>': ['UTC_TIMESTAMP'],
 '<UTC_TIME_SYMBOL>': ['UTC_TIME'],
 '<VALIDATION_SYMBOL>': ['VALIDATION'],
 '<VALUES_SYMBOL>': ['VALUES'],
 '<VALUE_SYMBOL>': ['VALUE'],
 '<VARBINARY_SYMBOL>': ['VARBINARY'],
 '<VARCHAR_SYMBOL>': ['VARCHAR'],
 '<VARIABLES_SYMBOL>': ['VARIABLES'],
 '<VARIANCE_SYMBOL>': ['VARIANCE'],
 '<VARYING_SYMBOL>': ['VARYING'],
 '<VAR_SAMP_SYMBOL>': ['VAR_SAMP'],
 '<VIEW_SYMBOL>': ['VIEW'],
 '<VIRTUAL_SYMBOL>': ['VIRTUAL'],
 '<WAIT_SYMBOL>': ['WAIT'],
 '<WARNINGS_SYMBOL>': ['WARNINGS'],
 '<WEEK_SYMBOL>': ['WEEK'],
 '<WEIGHT_STRING_SYMBOL>': ['WEIGHT_STRING'],
 '<WHEN_SYMBOL>': ['WHEN'],
 '<WHERE_SYMBOL>': ['WHERE'],
 '<WHILE_SYMBOL>': ['WHILE'],
 '<WITH_SYMBOL>': ['WITH'],
 '<WITHOUT_SYMBOL>': ['WITHOUT'],
 '<WORK_SYMBOL>': ['WORK'],
 '<WRAPPER_SYMBOL>': ['WRAPPER'],
 '<WRITE_SYMBOL>': ['WRITE'],
 '<X509_SYMBOL>': ['X509'],
 '<XA_SYMBOL>': ['XA'],
 '<XID_SYMBOL>': ['XID'],
 '<XML_SYMBOL>': ['XML'],
 '<XOR_SYMBOL>': ['XOR'],
 '<YEAR_MONTH_SYMBOL>': ['YEAR_MONTH'],
 '<YEAR_SYMBOL>': ['YEAR'],
 '<ZEROFILL_SYMBOL>': ['ZEROFILL'],
 '<PERSIST_SYMBOL>': ['PERSIST'],
 '<ROLE_SYMBOL>': ['ROLE'],
 '<ADMIN_SYMBOL>': ['ADMIN'],
 '<INVISIBLE_SYMBOL>': ['INVISIBLE'],
 '<VISIBLE_SYMBOL>': ['VISIBLE'],
 '<EXCEPT_SYMBOL>': ['EXCEPT'],
 '<COMPONENT_SYMBOL>': ['COMPONENT'],
 '<RECURSIVE_SYMBOL>': ['RECURSIVE'],
 '<JSON_OBJECTAGG_SYMBOL>': ['JSON_OBJECTAGG'],
 '<JSON_ARRAYAGG_SYMBOL>': ['JSON_ARRAYAGG'],
 '<OF_SYMBOL>': ['OF'],
 '<SKIP_SYMBOL>': ['SKIP'],
 '<LOCKED_SYMBOL>': ['LOCKED'],
 '<NOWAIT_SYMBOL>': ['NOWAIT'],
 '<GROUPING_SYMBOL>': ['GROUPING'],
 '<PERSIST_ONLY_SYMBOL>': ['PERSIST_ONLY'],
 '<HISTOGRAM_SYMBOL>': ['HISTOGRAM'],
 '<BUCKETS_SYMBOL>': ['BUCKETS'],
 '<REMOTE_SYMBOL>': ['REMOTE'],
 '<CLONE_SYMBOL>': ['CLONE'],
 '<CUME_DIST_SYMBOL>': ['CUME_DIST'],
 '<DENSE_RANK_SYMBOL>': ['DENSE_RANK'],
 '<EXCLUDE_SYMBOL>': ['EXCLUDE'],
 '<FIRST_VALUE_SYMBOL>': ['FIRST_VALUE'],
 '<FOLLOWING_SYMBOL>': ['FOLLOWING'],
 '<GROUPS_SYMBOL>': ['GROUPS'],
 '<LAG_SYMBOL>': ['LAG'],
 '<LAST_VALUE_SYMBOL>': ['LAST_VALUE'],
 '<LEAD_SYMBOL>': ['LEAD'],
 '<NTH_VALUE_SYMBOL>': ['NTH_VALUE'],
 '<NTILE_SYMBOL>': ['NTILE'],
 '<NULLS_SYMBOL>': ['NULLS'],
 '<OTHERS_SYMBOL>': ['OTHERS'],
 '<OVER_SYMBOL>': ['OVER'],
 '<PERCENT_RANK_SYMBOL>': ['PERCENT_RANK'],
 '<PRECEDING_SYMBOL>': ['PRECEDING'],
 '<RANK_SYMBOL>': ['RANK'],
 '<RESPECT_SYMBOL>': ['RESPECT'],
 '<ROW_NUMBER_SYMBOL>': ['ROW_NUMBER'],
 '<TIES_SYMBOL>': ['TIES'],
 '<UNBOUNDED_SYMBOL>': ['UNBOUNDED'],
 '<WINDOW_SYMBOL>': ['WINDOW'],
 '<EMPTY_SYMBOL>': ['EMPTY'],
 '<JSON_TABLE_SYMBOL>': ['JSON_TABLE'],
 '<NESTED_SYMBOL>': ['NESTED'],
 '<ORDINALITY_SYMBOL>': ['ORDINALITY'],
 '<PATH_SYMBOL>': ['PATH'],
 '<HISTORY_SYMBOL>': ['HISTORY'],
 '<REUSE_SYMBOL>': ['REUSE'],
 '<SRID_SYMBOL>': ['SRID'],
 '<THREAD_PRIORITY_SYMBOL>': ['THREAD_PRIORITY'],
 '<RESOURCE_SYMBOL>': ['RESOURCE'],
 '<SYSTEM_SYMBOL>': ['SYSTEM'],
 '<VCPU_SYMBOL>': ['VCPU'],
 '<MASTER_PUBLIC_KEY_PATH_SYMBOL>': ['MASTER_PUBLIC_KEY_PATH'],
 '<GET_MASTER_PUBLIC_KEY_SYMBOL>': ['GET_MASTER_PUBLIC_KEY_SYM'],
 '<RESTART_SYMBOL>': ['RESTART'],
 '<DEFINITION_SYMBOL>': ['DEFINITION'],
 '<DESCRIPTION_SYMBOL>': ['DESCRIPTION'],
 '<ORGANIZATION_SYMBOL>': ['ORGANIZATION'],
 '<REFERENCE_SYMBOL>': ['REFERENCE'],
 '<OPTIONAL_SYMBOL>': ['OPTIONAL'],
 '<SECONDARY_SYMBOL>': ['SECONDARY'],
 '<SECONDARY_ENGINE_SYMBOL>': ['SECONDARY_ENGINE'],
 '<SECONDARY_LOAD_SYMBOL>': ['SECONDARY_LOAD'],
 '<SECONDARY_UNLOAD_SYMBOL>': ['SECONDARY_UNLOAD'],
 '<ACTIVE_SYMBOL>': ['ACTIVE'],
 '<INACTIVE_SYMBOL>': ['INACTIVE'],
 '<LATERAL_SYMBOL>': ['LATERAL'],
 '<RETAIN_SYMBOL>': ['RETAIN'],
 '<OLD_SYMBOL>': ['OLD'],
 '<NETWORK_NAMESPACE_SYMBOL>': ['NETWORK_NAMESPACE'],
 '<ENFORCED_SYMBOL>': ['ENFORCED'],
 '<ARRAY_SYMBOL>': ['ARRAY'],
 '<OJ_SYMBOL>': ['OJ'],
 '<MEMBER_SYMBOL>': ['MEMBER'],
 '<RANDOM_SYMBOL>': ['RANDOM'],
 '<MASTER_COMPRESSION_ALGORITHM_SYMBOL>': ['MASTER_COMPRESSION_ALGORITHM'],
 '<MASTER_ZSTD_COMPRESSION_LEVEL_SYMBOL>': ['MASTER_ZSTD_COMPRESSION_LEVEL'],
 '<PRIVILEGE_CHECKS_USER_SYMBOL>': ['PRIVILEGE_CHECKS_USER'],
 '<MASTER_TLS_CIPHERSUITES_SYMBOL>': ['MASTER_TLS_CIPHERSUITES'],
 '<REQUIRE_ROW_FORMAT_SYMBOL>': ['REQUIRE_ROW_FORMAT'],
 '<PASSWORD_LOCK_TIME_SYMBOL>': ['PASSWORD_LOCK_TIME'],
 '<FAILED_LOGIN_ATTEMPTS_SYMBOL>': ['FAILED_LOGIN_ATTEMPTS'],
 '<REQUIRE_TABLE_PRIMARY_KEY_CHECK_SYMBOL>': ['REQUIRE_TABLE_PRIMARY_KEY_CHECK'],
 '<STREAM_SYMBOL>': ['STREAM'],
 '<OFF_SYMBOL>': ['OFF'],
 '<RETURNING_SYMBOL>': ['RETURNING'],
 '<JSON_VALUE_SYMBOL>': ['JSON_VALUE'],
 '<TLS_SYMBOL>': ['TLS'],
 '<ATTRIBUTE_SYMBOL>': ['ATTRIBUTE'],
 '<ENGINE_ATTRIBUTE_SYMBOL>': ['ENGINE_ATTRIBUTE'],
 '<SECONDARY_ENGINE_ATTRIBUTE_SYMBOL>': ['SECONDARY_ENGINE_ATTRIBUTE'],
 '<SOURCE_CONNECTION_AUTO_FAILOVER_SYMBOL>': ['SOURCE_CONNECTION_AUTO_FAILOVER'],
 '<ZONE_SYMBOL>': ['ZONE'],
 '<REPLICA_SYMBOL>': ['REPLICA'],
 '<REPLICAS_SYMBOL>': ['REPLICAS'],
 '<ASSIGN_GTIDS_TO_ANONYMOUS_TRANSACTIONS_SYMBOL>': ['ASSIGN_GTIDS_TO_ANONYMOUS_TRANSACTIONS'],
 '<GET_SOURCE_PUBLIC_KEY_SYMBOL>': ['GET_SOURCE_PUBLIC_KEY'],
 '<SOURCE_AUTO_POSITION_SYMBOL>': ['SOURCE_AUTO_POSITION'],
 '<SOURCE_BIND_SYMBOL>': ['SOURCE_BIND'],
 '<SOURCE_COMPRESSION_ALGORITHM_SYMBOL>': ['SOURCE_COMPRESSION_ALGORITHM'],
 '<SOURCE_CONNECT_RETRY_SYMBOL>': ['SOURCE_CONNECT_RETRY'],
 '<SOURCE_DELAY_SYMBOL>': ['SOURCE_DELAY'],
 '<SOURCE_HEARTBEAT_PERIOD_SYMBOL>': ['SOURCE_HEARTBEAT_PERIOD'],
 '<SOURCE_HOST_SYMBOL>': ['SOURCE_HOST'],
 '<SOURCE_LOG_FILE_SYMBOL>': ['SOURCE_LOG_FILE'],
 '<SOURCE_LOG_POS_SYMBOL>': ['SOURCE_LOG_POS'],
 '<SOURCE_PASSWORD_SYMBOL>': ['SOURCE_PASSWORD'],
 '<SOURCE_PORT_SYMBOL>': ['SOURCE_PORT'],
 '<SOURCE_PUBLIC_KEY_PATH_SYMBOL>': ['SOURCE_PUBLIC_KEY_PATH'],
 '<SOURCE_RETRY_COUNT_SYMBOL>': ['SOURCE_RETRY_COUNT'],
 '<SOURCE_SSL_SYMBOL>': ['SOURCE_SSL'],
 '<SOURCE_SSL_CA_SYMBOL>': ['SOURCE_SSL_CA'],
 '<SOURCE_SSL_CAPATH_SYMBOL>': ['SOURCE_SSL_CAPATH'],
 '<SOURCE_SSL_CERT_SYMBOL>': ['SOURCE_SSL_CERT'],
 '<SOURCE_SSL_CIPHER_SYMBOL>': ['SOURCE_SSL_CIPHER'],
 '<SOURCE_SSL_CRL_SYMBOL>': ['SOURCE_SSL_CRL'],
 '<SOURCE_SSL_CRLPATH_SYMBOL>': ['SOURCE_SSL_CRLPATH'],
 '<SOURCE_SSL_KEY_SYMBOL>': ['SOURCE_SSL_CRLPATH'],
 '<SOURCE_SSL_VERIFY_SERVER_CERT_SYMBOL>': ['SOURCE_SSL_VERIFY_SERVER_CERT'],
 '<SOURCE_TLS_CIPHERSUITES_SYMBOL>': ['SOURCE_TLS_CIPHERSUITES'],
 '<SOURCE_TLS_VERSION_SYMBOL>': ['SOURCE_TLS_VERSION'],
 '<SOURCE_USER_SYMBOL>': ['SOURCE_USER'],
 '<SOURCE_ZSTD_COMPRESSION_LEVEL_SYMBOL>': ['SOURCE_ZSTD_COMPRESSION_LEVEL'],
 '<ST_COLLECT_SYMBOL>': ['ST_COLLECT'],
 '<KEYRING_SYMBOL>': ['KEYRING'],
 '<AUTHENTICATION_SYMBOL>': ['AUTHENTICATION'],
 '<FACTOR_SYMBOL>': ['FACTOR'],
 '<FINISH_SYMBOL>': ['FINISH'],
 '<INITIATE_SYMBOL>': ['INITIATE'],
 '<REGISTRATION_SYMBOL>': ['REGISTRATION'],
 '<UNREGISTER_SYMBOL>': ['UNREGISTER'],
 '<INITIAL_SYMBOL>': ['INITIAL'],
 '<CHALLENGE_RESPONSE_SYMBOL>': ['CHALLENGE_RESPONSE'],
 '<GTID_ONLY_SYMBOL>': ['GTID_ONLY'],
 '<INTERSECT_SYMBOL>': ['INTERSECT_SYMBOL'],
 '<BULK_SYMBOL>': ['BULK'],
 '<URL_SYMBOL>': ['URL'],
 '<GENERATE_SYMBOL>': ['GENERATE'],
 '<PARSE_TREE_SYMBOL>': ['PARSE_TREE'],
 '<LOG_SYMBOL>': ['LOG'],
 '<GTIDS_SYMBOL>': ['GTIDS'],
 '<PARALLEL_SYMBOL>': ['PARALLEL'],
 '<S3_SYMBOL>': ['S3'],
 '<QUALIFY_SYMBOL>': ['QUALIFY'],
 '<AUTO_SYMBOL>': ['AUTO'],
 '<MANUAL_SYMBOL>': ['MANUAL'],
 '<BERNOULLI_SYMBOL>': ['BERNOULLI'],
 '<TABLESAMPLE_SYMBOL>': ['TABLESAMPLE'],
 '<UNDERSCORE_CHARSET>': ['<UNDERLINE_SYMBOL><[a-z0-9]>+'],
 '<DOLLAR_QUOTED_STRING_TEXT>': ['$<DOLLAR_QUOTE_TAG_CHAR>*$<[0-9a-zA-Z_\\u0080-\\uffff]>*$<DOLLAR_QUOTE_TAG_CHAR>*$'],
 '<IDENTIFIER>': ['<DIGITS>+<[eE]><IDENTIFIER-1>?',
                  '<DIGITS>+<LETTER_WITHOUT_FLOAT_PART><LETTER_WHEN_UNQUOTED>*',
                  '<LETTER_WHEN_UNQUOTED_NO_DIGIT><LETTER_WHEN_UNQUOTED>*'],
 '<IDENTIFIER-1>': ['<LETTER_WHEN_UNQUOTED_NO_DIGIT><LETTER_WHEN_UNQUOTED>*'],
 '<NCHAR_TEXT>': ['<[nN]><SINGLE_QUOTED_TEXT>'],
 '<BACK_TICK>': ['`'],
 '<SINGLE_QUOTE>': ["\\'"],
 '<DOUBLE_QUOTE>': ['"'],
 '<BACK_TICK_QUOTED_ID>': ['<BACK_TICK><BACK_TICK_QUOTED_ID-1>*<BACK_TICK>'],
 '<BACK_TICK_QUOTED_ID-1>': ['<BACK_TICK_QUOTED_ID-2>?<[0-9a-zA-Z_\\u0080-\\uffff]>'],
 '<BACK_TICK_QUOTED_ID-2>': ['\\\\'],
 '<DOUBLE_QUOTED_TEXT>': ['<DOUBLE_QUOTED_TEXT-1>+'],
 '<DOUBLE_QUOTED_TEXT-1>': ['<DOUBLE_QUOTE><DOUBLE_QUOTED_TEXT-2>*<DOUBLE_QUOTE>'],
 '<DOUBLE_QUOTED_TEXT-2>': ['<DOUBLE_QUOTED_TEXT-3>?<[0-9a-zA-Z_\\u0080-\\uffff]>'],
 '<DOUBLE_QUOTED_TEXT-3>': ['\\\\'],
 '<SINGLE_QUOTED_TEXT>': ['<SINGLE_QUOTED_TEXT-1>+'],
 '<SINGLE_QUOTED_TEXT-1>': ['<SINGLE_QUOTE><SINGLE_QUOTED_TEXT-2>*<SINGLE_QUOTE>'],
 '<SINGLE_QUOTED_TEXT-2>': ['<SINGLE_QUOTED_TEXT-3>?<[0-9a-zA-Z_\\u0080-\\uffff]>'],
 '<SINGLE_QUOTED_TEXT-3>': ['\\\\'],
 '<SIMPLE_IDENTIFIER>': ['<SIMPLE_IDENTIFIER-1>+'],
 '<SIMPLE_IDENTIFIER-1>': ['<DIGIT>', '<[a-zA-Z_$]>', '<DOT_SYMBOL>'],
 '<LETTER_WHEN_UNQUOTED>': ['<DIGIT>', '<LETTER_WHEN_UNQUOTED_NO_DIGIT>'],
 '<LETTER_WHEN_UNQUOTED_NO_DIGIT>': ['<[a-zA-Z_$\\u0080-\\uffff]>'],
 '<DOLLAR_QUOTE_TAG_CHAR>': ['<[0-9a-zA-Z_\\u0080-\\uffff]>'],
 '<LETTER_WITHOUT_FLOAT_PART>': ['<[a-df-zA-DF-Z_$\\u0080-\\uffff]>'],
 '<[0-9]>': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 '<[a-zA-Z_$]>': ['a',
                  'b',
                  'c',
                  'd',
                  'e',
                  'f',
                  'g',
                  'h',
                  'i',
                  'j',
                  'k',
                  'l',
                  'm',
                  'n',
                  'o',
                  'p',
                  'q',
                  'r',
                  's',
                  't',
                  'u',
                  'v',
                  'w',
                  'x',
                  'y',
                  'z',
                  'A',
                  'B',
                  'C',
                  'D',
                  'E',
                  'F',
                  'G',
                  'H',
                  'I',
                  'J',
                  'K',
                  'L',
                  'M',
                  'N',
                  'O',
                  'P',
                  'Q',
                  'R',
                  'S',
                  'T',
                  'U',
                  'V',
                  'W',
                  'X',
                  'Y',
                  'Z',
                  '$',
                  '_'],
 '<[01]>': ['0', '1'],
 '<[0-9a-zA-Z_\\u0080-\\uffff]>': ['0',
                                   '1',
                                   '2',
                                   '3',
                                   '4',
                                   '5',
                                   '6',
                                   '7',
                                   '8',
                                   '9',
                                   'a',
                                   'b',
                                   'c',
                                   'd',
                                   'e',
                                   'f',
                                   'g',
                                   'h',
                                   'i',
                                   'j',
                                   'k',
                                   'l',
                                   'm',
                                   'n',
                                   'o',
                                   'p',
                                   'q',
                                   'r',
                                   's',
                                   't',
                                   'u',
                                   'v',
                                   'w',
                                   'x',
                                   'y',
                                   'z',
                                   'A',
                                   'B',
                                   'C',
                                   'D',
                                   'E',
                                   'F',
                                   'G',
                                   'H',
                                   'I',
                                   'J',
                                   'K',
                                   'L',
                                   'M',
                                   'N',
                                   'O',
                                   'P',
                                   'Q',
                                   'R',
                                   'S',
                                   'T',
                                   'U',
                                   'V',
                                   'W',
                                   'X',
                                   'Y',
                                   'Z',
                                   '_'],
 '<[0-9a-fA-F]>': ['0',
                   '1',
                   '2',
                   '3',
                   '4',
                   '5',
                   '6',
                   '7',
                   '8',
                   '9',
                   'a',
                   'b',
                   'c',
                   'd',
                   'e',
                   'f',
                   'A',
                   'B',
                   'C',
                   'D',
                   'E',
                   'F'],
 '<[nN]>': ['n', 'N'],
 '<[a-df-zA-DF-Z_$\\u0080-\\uffff]>': ['a',
                                       'b',
                                       'c',
                                       'd',
                                       'f',
                                       'g',
                                       'h',
                                       'i',
                                       'j',
                                       'k',
                                       'l',
                                       'm',
                                       'n',
                                       'o',
                                       'p',
                                       'q',
                                       'r',
                                       's',
                                       't',
                                       'u',
                                       'v',
                                       'w',
                                       'x',
                                       'y',
                                       'z',
                                       'A',
                                       'B',
                                       'C',
                                       'D',
                                       'F',
                                       'G',
                                       'H',
                                       'I',
                                       'J',
                                       'K',
                                       'L',
                                       'M',
                                       'N',
                                       'O',
                                       'P',
                                       'Q',
                                       'R',
                                       'S',
                                       'T',
                                       'U',
                                       'V',
                                       'W',
                                       'X',
                                       'Y',
                                       'Z',
                                       '$',
                                       '_'],
 '<[a-zA-Z_$\\u0080-\\uffff]>': ['a',
                                 'b',
                                 'c',
                                 'd',
                                 'e',
                                 'f',
                                 'g',
                                 'h',
                                 'i',
                                 'j',
                                 'k',
                                 'l',
                                 'm',
                                 'n',
                                 'o',
                                 'p',
                                 'q',
                                 'r',
                                 's',
                                 't',
                                 'u',
                                 'v',
                                 'w',
                                 'x',
                                 'y',
                                 'z',
                                 'A',
                                 'B',
                                 'C',
                                 'D',
                                 'E',
                                 'F',
                                 'G',
                                 'H',
                                 'I',
                                 'J',
                                 'K',
                                 'L',
                                 'M',
                                 'N',
                                 'O',
                                 'P',
                                 'Q',
                                 'R',
                                 'S',
                                 'T',
                                 'U',
                                 'V',
                                 'W',
                                 'X',
                                 'Y',
                                 'Z',
                                 '$',
                                 '_'],
 '<[a-z0-9]>': ['0',
                '1',
                '2',
                '3',
                '4',
                '5',
                '6',
                '7',
                '8',
                '9',
                'a',
                'b',
                'c',
                'd',
                'e',
                'f',
                'g',
                'h',
                'i',
                'j',
                'k',
                'l',
                'm',
                'n',
                'o',
                'p',
                'q',
                'r',
                's',
                't',
                'u',
                'v',
                'w',
                'x',
                'y',
                'z'],
 '<[eE]>': ['e', 'E']}



# Done

In [ ]:
#%tb